# AInewsbot.ipynb - Automate collecting daily AI news

1. initial scrape of front pages of tech sites
  - Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
  - Save HTML of each URL in htmldata directory  
  - Extract URLs from all files, create a pandas dataframe with url, title, src

2. Filter and clean to AI-related headlines not seen before
  - Use ChatGPT prompt to filter only AI-related headlines by sending a prompt and formatted table of headlines
  - Use SQLite to filter headlines previously seen 
  - remove duplicate URLs and headlines
  - ensure there are pretty source names for each news site

3. Topic analysis, make a list of topics for each headline
  - using a prompt, check each headline against a number of evergreen AI topics, e.g. deepfakes, regulation, AI in education
  - extract free from topics from each headline
  - combine topics into topic list for each headline
  - cluster headlines using dimensionality-reduced embeddings and DBSCAN; ask chatgpt to name each cluster
  - sort headlines by doing a traveling salesman shortest traversal in embedding space

4. Summarize individual news story pages in 3 bullets using a prompt

5. create a large markdown file with all bullet points and topics

6. give the markdown file to ChatGPT and ask it to make a list of most popular and import topics of the day

7. human should make a list of the day's topics, combining the chatgpt response to the quesion and cluster topics and 

8. Put summaries in vector store along with metadata. For each topic, retrieve all associated stories and have chatgpt write a digest of those stories in the given format.

9. assemble stories into first draft of newsletter for rewriting as necessary

todo:

use langgraph for final editing workfow
1. prompt to edit final copy for dupes, combine similar sections, copy edit
2. have a reviewer prompt check if there are any bullet points to move to a different section 
3. if so have an editor prompt remove them , return to 2. until nothing left to move dupes left
4. have a reviewer check each section, identify bullet points that are similar to other bullet points in the section and have identical links. rewrite combining so there is no duplication. 
5. identify sections that are short or similar to other sections and suggest sections that should be combined them
6. have an editor prompt merge short sections, return to 4, until no orphan sections left
7. maybe final copy-edit prompt

Original, alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


In [1]:
import sys
del sys.modules['ainb_webscrape']


KeyError: 'ainb_webscrape'

In [17]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter
import shutil

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import trafilatura

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, get_google_news_redirects, DRIVERS)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, 
                      fetch_openai_summary, trunc_tokens, categorize_headline)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


# Initialize

In [18]:
before_date = '2024-07-17 18:00:00'

In [19]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [20]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-07-18 10:24:10,003 - AInewsbot - INFO - Load 17 sources from sources.yaml
2024-07-18 10:24:10,004 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-18 10:24:10,004 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-18 10:24:10,004 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-18 10:24:10,004 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-18 10:24:10,005 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-18 10:2

20

In [21]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology',
  'sourcename': 'Bloomberg Tech'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=e

In [22]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [23]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]
# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))

2024-07-18 10:24:16,783 - AInewsbot - INFO - 0


# Fetch and save source pages

In [24]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


2024-07-18 10:24:21,181 - AInewsbot - INFO - Saving HTML files using 3 browsers
2024-07-18 10:24:21,195 - AInewsbot - INFO - get_driver - 59284 Initializing webdriver
2024-07-18 10:24:21,196 - AInewsbot - INFO - get_driver - 59284 Initializing webdriver
2024-07-18 10:24:21,196 - AInewsbot - INFO - get_driver - 59284 Initializing webdriver
2024-07-18 10:24:42,267 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-18 10:24:42,268 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-18 10:24:42,269 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-18 10:24:42,270 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-18 10:24:42,271 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-18 10:24:42,272 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-18 10:26:14,045 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-07-18 10:26:14,179 - AInewsbot - INFO - Proces

2024-07-18 10:27:00,021 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Saving Google News - Technology - Artificial intelligence (07_18_2024 10_27_00 AM).html as utf-8
2024-07-18 10:27:00,032 - AInewsbot - INFO - Processing NYT Tech
2024-07-18 10:27:00,035 - AInewsbot - INFO - get_files(Technology - The New York Times) - starting get_files https://www.nytimes.com/section/technology
2024-07-18 10:27:08,331 - AInewsbot - INFO - get_files(Discover and Add New Feedly AI Feeds) - Loading additional infinite scroll items
2024-07-18 10:27:09,339 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrom

2024-07-18 10:27:54,781 - AInewsbot - INFO - get_files(Technology - WSJ.com) - Saving Technology - WSJ.com (07_18_2024 10_27_54 AM).html as utf-8
2024-07-18 10:27:54,782 - AInewsbot - INFO - Quit webdriver
2024-07-18 10:27:59,482 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-18 10:27:59,483 - AInewsbot - INFO - get_files(Technology - The Washington Post) - Saving Technology - The Washington Post (07_18_2024 10_27_59 AM).html as utf-8
2024-07-18 10:27:59,486 - AInewsbot - INFO - Quit webdriver


In [25]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log(f"{sourcename} -> {page}")
    

2024-07-18 10:28:01,912 - AInewsbot - INFO - Saved 17 pages
2024-07-18 10:28:01,915 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (07_18_2024 10_26_26 AM).html
2024-07-18 10:28:01,915 - AInewsbot - INFO - FT Tech -> htmldata/Technology (07_18_2024 10_26_36 AM).html
2024-07-18 10:28:01,916 - AInewsbot - INFO - Hacker News -> htmldata/Hacker News Page 1 (07_18_2024 10_26_47 AM).html
2024-07-18 10:28:01,916 - AInewsbot - INFO - Hacker News 2 -> htmldata/Hacker News Page 2 (07_18_2024 10_26_58 AM).html
2024-07-18 10:28:01,916 - AInewsbot - INFO - HackerNoon -> htmldata/HackerNoon - read, write and learn about any technology (07_18_2024 10_27_09 AM).html
2024-07-18 10:28:01,917 - AInewsbot - INFO - Reddit -> htmldata/top scoring links _ multi (07_18_2024 10_27_42 AM).html
2024-07-18 10:28:01,917 - AInewsbot - INFO - VentureBeat -> htmldata/AI News _ VentureBeat (07_18_2024 10_27_52 AM).html
2024-07-18 10:28:01,917 - AInewsbot - INFO - Bloomberg Tech -> htmldata/Bloomberg Techno

17


# Extract news URLs from saved pages

In [26]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


2024-07-18 10:28:01,925 - AInewsbot - INFO - Parsing html files
2024-07-18 10:28:01,952 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (07_18_2024 10_26_26 AM).html
2024-07-18 10:28:01,952 - AInewsbot - INFO - parse loop - Ars Technica
2024-07-18 10:28:01,975 - AInewsbot - INFO - parse_file - found 252 raw links
2024-07-18 10:28:01,978 - AInewsbot - INFO - parse_file - found 28 filtered links
2024-07-18 10:28:01,978 - AInewsbot - INFO - parse loop - 28 links found
2024-07-18 10:28:01,978 - AInewsbot - INFO - FT Tech -> htmldata/Technology (07_18_2024 10_26_36 AM).html
2024-07-18 10:28:01,978 - AInewsbot - INFO - parse loop - FT Tech
2024-07-18 10:28:02,015 - AInewsbot - INFO - parse_file - found 479 raw links
2024-07-18 10:28:02,020 - AInewsbot - INFO - parse_file - found 121 filtered links
2024-07-18 10:28:02,020 - AInewsbot - INFO - parse loop - 121 links found
2024-07-18 10:28:02,020 - AInewsbot - INFO - Hacker News -> htmldata/Hacker News Page 1 (07_18_2024 10_26_47 AM)

,id,src,title,url
0,0,Ars Technica,One more way to die: Tremors when Vesuvius eru...,https://arstechnica.com/science/2024/07/some-p...
1,1,Ars Technica,Streaming’s bundling obsession ignores the rea...,https://arstechnica.com/gadgets/2024/07/stream...
2,2,Ars Technica,Nintendo World Championships: NES Editiongave ...,https://arstechnica.com/gaming/2024/07/nintend...
3,3,Ars Technica,Valve runs its massive PC gaming ecosystem wit...,https://arstechnica.com/gaming/2024/07/valve-r...
4,4,Ars Technica,Paul Sutter walks us through the future of cli...,https://arstechnica.com/science/2022/04/paul-s...


In [27]:
# # extracts all links from history where isAI=1
# # useful for training dimensionality reduction
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id"
# ai_history_df = pd.read_sql_query(query, conn)
# ai_history_df

# Filter URLs to new AI headlines only

In [28]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df, before_date=before_date)
len(filtered_df)

2024-07-18 10:28:03,672 - AInewsbot - INFO - Existing URLs: 133549
2024-07-18 10:28:03,693 - AInewsbot - INFO - New URLs: 756


756

In [29]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)



You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will closely read the title of each story
to determine if it is primarily about AI based on the semantic meaning of the title and
the keywords and entities mentioned. The input headlines and outptu classifications will
be formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based 

In [30]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


2024-07-18 10:28:03,971 - AInewsbot - INFO - Paginated 16 pages


20

In [31]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


2024-07-18 10:28:03,974 - AInewsbot - INFO - start classify
2024-07-18 10:28:03,977 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,000 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,000 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,000 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,000 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,001 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,001 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,001 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,002 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,002 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,002 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,002 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,003 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,003 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,003 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:28:04,003 - AInewsbot - IN

756


,id,isAI
0,0,False
1,2,False
2,3,False
3,5,False
4,6,False


In [32]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,isAI,date
0,0,Ars Technica,One more way to die: Tremors when Vesuvius eru...,https://arstechnica.com/science/2024/07/some-p...,False,2024-07-18
1,2,Ars Technica,Nintendo World Championships: NES Editiongave ...,https://arstechnica.com/gaming/2024/07/nintend...,False,2024-07-18
2,3,Ars Technica,Valve runs its massive PC gaming ecosystem wit...,https://arstechnica.com/gaming/2024/07/valve-r...,False,2024-07-18
3,5,Ars Technica,Electric eels inspire novel “jelly” batteries ...,https://arstechnica.com/science/2024/07/electr...,False,2024-07-18
4,6,Ars Technica,Five people infected as bird flu appears to go...,https://arstechnica.com/science/2024/07/five-p...,False,2024-07-18


In [33]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-07-18 10:28:14,458 - AInewsbot - INFO - Unmatched response rows: 0
2024-07-18 10:28:14,460 - AInewsbot - INFO - Unmatched source rows: 0


20

In [34]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")

AIdf

2024-07-18 10:28:14,465 - AInewsbot - INFO - Found 202 AI headlines


,id,src,title,url,isAI,date
0,0,Bloomberg Tech,Secretive China Agency Backtracks on Aggressiv...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-18
1,1,Bloomberg Tech,AI Startup Tied to Fake Biden Robocall Aims to...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-18
2,2,Bloomberg Tech,More Than Half of TSMC’s Sales Are Now High-En...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-18
3,3,Bloomberg Tech,Nvidia Backs Networking Software Startup Arrcu...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-18
4,4,Business Insider,Members of the PayPal Mafia include tech titan...,https://www.businessinsider.com/paypal-mafia-m...,True,2024-07-18
...,...,...,...,...,...,...
197,197,VentureBeat,Nvidia and Mistral's new model 'Mistral-NeMo' ...,https://venturebeat.com/ai/nvidia-and-mistrals...,True,2024-07-18
198,198,VentureBeat,From reality to fantasy: Live2Diff AI brings i...,https://venturebeat.com/ai/from-reality-to-fan...,True,2024-07-18
199,199,VentureBeat,Microsoft’s Designer AI image generator comes ...,https://venturebeat.com/ai/microsofts-designer...,True,2024-07-18
200,200,VentureBeat,Artificial Agency raises $16M to bring AI-powe...,https://venturebeat.com/ai/artificial-agency-r...,True,2024-07-18


In [35]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [36]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


2024-07-18 10:28:14,482 - AInewsbot - INFO - Found 198 unique AI headlines


20

In [37]:
# map google news headlines to redirect

AIdf = get_google_news_redirects(AIdf)


2024-07-18 10:28:14,785 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature
2024-07-18 10:28:15,023 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://qz.com/godmother-ai-dr-fei-fei-li-built-1-billion-startup-1851597427
2024-07-18 10:28:15,252 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.tradingview.com/news/barchart:d5ab2bc66094b:0-3-ai-stocks-to-grab-now-for-catch-up-gains/
2024-07-18 10:28:15,521 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://interestingengineering.com/innovation/insect-inspired-navigation-robot
2024-07-18 10:28:15,745 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.nature.com/articles/d41586-024-02354-0
2024-07-18 10:28:16,004 - AInewsbot - INFO - get_google_news_redirect

2024-07-18 10:28:26,096 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology
2024-07-18 10:28:26,392 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.calcalistech.com/ctechnews/article/hkyrovso0
2024-07-18 10:28:26,595 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.prnewswire.com/news-releases/icl-and-lavie-bio-leverage-artificial-intelligence-to-make-significant-advancement-in-development-of-yield-increasing-bio-stimulants-302199316.html
2024-07-18 10:28:26,848 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.businessinsider.com/jpmorgan-ceo-jamie-dimon-adding-ai-focused-jobs-5000-2024-7
2024-07-18 10:28:27,091 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://technode.com/2024/07/18/large-language-models-are-rubbish-at-elementary-level-math/
2024-07-18 10:28:27,335 - AIne

2024-07-18 10:28:36,715 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.cnbc.com/2024/07/17/tinders-new-ai-tool-will-curate-your-dating-profile-pictures-for-you-.html
2024-07-18 10:28:36,956 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://petapixel.com/2024/07/17/tinder-adds-ai-to-help-you-pick-your-best-photos-and-get-more-matches/
2024-07-18 10:28:37,211 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.thesun.co.uk/tech/29291803/tinder-ai-tool-photo-selector-profile-pictures/
2024-07-18 10:28:37,474 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.eurekalert.org/multimedia/1034513
2024-07-18 10:28:37,716 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> https://www.fastcompany.com/91154420/transition-from-solo-acts-to-a-complete-symphony-with-ai
2024-07-18 10:28:37,966 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> http://bre

In [38]:
# must do this after fixing google actualurl
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf.head()

,id,src,title,url,isAI,date,actual_url,hostname
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com
1,1,Google News,'Godmother of AI' Dr. Fei-Fei Li just built a ...,https://news.google.com/articles/CBMiTGh0dHBzO...,True,2024-07-18,https://qz.com/godmother-ai-dr-fei-fei-li-buil...,qz.com
2,2,Feedly AI,1Artificial Intelligence(AI) ETF to Buy With $...,https://www.fool.com/investing/2024/07/18/1-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/1-ai...,www.fool.com
3,3,Google News,3 AI Stocks to Grab Now for 'Catch-Up' Gains,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.tradingview.com/news/barchart:d5ab...,www.tradingview.com
4,4,Feedly AI,3Artificial Intelligence(AI) Stocks With Scorc...,https://www.fool.com/investing/2024/07/18/3-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/3-ai...,www.fool.com


### Get site names and update site names based on URL

In [39]:
# get site_name
conn = sqlite3.connect('articles.db')
c = conn.cursor()
#  and timestamp > '2024-07-01' 
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}

sites_df

,id,hostname,site_name
0,1,247wallst.com,247wallst
1,2,3quarksdaily.com,3 Quarks Daily
2,3,9to5google.com,9to5Google
3,4,9to5mac.com,9to5Mac
4,5,9to5toys.com,9to5Toys
...,...,...,...
744,791,www.thesun.co.uk,The Sun
745,792,newsroom.spotify.com,Spotify Newsroom
746,793,patentlyo.com,Patently-O
747,794,ipwatchdog.com,IPWatchdog


In [40]:
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))
AIdf.loc[AIdf['site_name']==""]

,id,src,title,url,isAI,date,actual_url,hostname,site_name
3,3,Google News,3 AI Stocks to Grab Now for 'Catch-Up' Gains,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.tradingview.com/news/barchart:d5ab...,www.tradingview.com,
31,31,Google News,Artificial intelligence is already an environm...,https://news.google.com/articles/CBMicWh0dHBzO...,True,2024-07-18,https://english.elpais.com/technology/2024-07-...,english.elpais.com,
36,36,Google News,CTERA gets $80m war chest for AI and acquisitions,https://news.google.com/articles/CBMiUWh0dHBzO...,True,2024-07-18,https://blocksandfiles.com/2024/07/17/ctera-ge...,blocksandfiles.com,
42,42,Google News,Discover the Meizu Blue 20 AI: Affordable AI-P...,https://news.google.com/articles/CBMiY2h0dHBzO...,True,2024-07-18,https://www.gizchina.com/2024/07/16/discover-t...,www.gizchina.com,
57,57,Google News,Google's Healthcare Strategy and the Role of AI,https://news.google.com/articles/CBMiWWh0dHBzO...,True,2024-07-18,https://www.medscape.com/viewarticle/googles-h...,www.medscape.com,
67,67,Google News,Hanwha Energy to Invest 2 Trillion Won in Supe...,https://news.google.com/articles/CBMiQmh0dHBzO...,True,2024-07-18,https://www.businesskorea.co.kr/news/articleVi...,www.businesskorea.co.kr,
72,72,Google News,How Machine Learning Is Propelling Structural ...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-18,https://hms.harvard.edu/news/how-machine-learn...,hms.harvard.edu,
85,85,Google News,Large language models are rubbish at elementar...,https://news.google.com/articles/CBMiW2h0dHBzO...,True,2024-07-18,https://technode.com/2024/07/18/large-language...,technode.com,
90,90,Google News,Meizu Blue 20 announced as ~$140-priced AI pho...,https://news.google.com/articles/CBMiSGh0dHBzO...,True,2024-07-18,https://www.gizmochina.com/2024/07/16/meizu-bl...,www.gizmochina.com,
116,116,Google News,NYC will start rolling out AI metal detectors ...,https://news.google.com/articles/CBMiSmh0dHBzO...,True,2024-07-18,https://www.amny.com/transit/nyc-subway-metal-...,www.amny.com,


In [41]:
async def get_site_name(session, row):
    cat_prompt = f"""
based on this url and your knowledge of the Web, what is the name of the site? https://{row.hostname}

return the response as a json object of the form {{"url": "www.yankodesign.com", "site_name": "Yanko Design"}}

    """
    try:
        messages=[
                  {"role": "user", "content": cat_prompt
                  }]

        payload = {"model":  LOWCOST_MODEL,
                   "response_format": {"type": "json_object"},
                   "messages": messages,
                   "temperature": 0
                   }
        response = await fetch_openai(session, payload)
        response_dict = json.loads(response["choices"][0]["message"]["content"])
        return response_dict
    except Exception as exc:
        print(exc)
                
tasks = []
async with aiohttp.ClientSession() as session:
    for row in AIdf.loc[AIdf['site_name']==""].itertuples():
        task = asyncio.create_task(get_site_name(session, row))
        tasks.append(task)
    responses = await asyncio.gather(*tasks)

responses


[{'url': 'www.tradingview.com', 'site_name': 'TradingView'},
 {'url': 'https://english.elpais.com', 'site_name': 'El Pais'},
 {'url': 'https://blocksandfiles.com', 'site_name': 'Blocks and Files'},
 {'url': 'www.gizchina.com', 'site_name': 'GizChina'},
 {'url': 'www.medscape.com', 'site_name': 'Medscape'},
 {'url': 'www.businesskorea.co.kr', 'site_name': 'Business Korea'},
 {'url': 'https://hms.harvard.edu', 'site_name': 'Harvard Medical School'},
 {'url': 'https://technode.com', 'site_name': 'TechNode'},
 {'url': 'www.gizmochina.com', 'site_name': 'GizmoChina'},
 {'url': 'www.amny.com', 'site_name': 'amNewYork'},
 {'url': 'https://aibusiness.com', 'site_name': 'AI Business'},
 {'url': 'https://seanpedersen.github.io', 'site_name': 'Sean Pedersen'},
 {'url': 'https://thehackernews.com', 'site_name': 'The Hacker News'},
 {'url': 'www.wiz.io', 'site_name': 'Wiz'},
 {'url': 'www.rstreet.org', 'site_name': 'R Street Institute'},
 {'url': 'www.tradingview.com', 'site_name': 'TradingView'},


In [42]:
# update site_dict from responses
new_urls = []
for r in responses:
    if r['url'].startswith('https://'):
        r['url'] = r['url'][8:]
    new_urls.append(r['url'])
    sites_dict[r['url']] = r['site_name']
    print(r['url'], r['site_name'])

AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))



www.tradingview.com TradingView
english.elpais.com El Pais
blocksandfiles.com Blocks and Files
www.gizchina.com GizChina
www.medscape.com Medscape
www.businesskorea.co.kr Business Korea
hms.harvard.edu Harvard Medical School
technode.com TechNode
www.gizmochina.com GizmoChina
www.amny.com amNewYork
aibusiness.com AI Business
seanpedersen.github.io Sean Pedersen
thehackernews.com The Hacker News
www.wiz.io Wiz
www.rstreet.org R Street Institute
www.tradingview.com TradingView
thewalrus.ca The Walrus
www.eurekalert.org EurekAlert
foreignpolicy.com Foreign Policy
www.khaleejtimes.com Khaleej Times
daily.jstor.org JSTOR Daily


In [43]:
for url in new_urls:
    sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
    print(url, '->', sites_dict[url])
    conn.execute(sqlstr, (url, sites_dict[url]))
    conn.commit()


www.tradingview.com -> TradingView
english.elpais.com -> El Pais
blocksandfiles.com -> Blocks and Files
www.gizchina.com -> GizChina
www.medscape.com -> Medscape
www.businesskorea.co.kr -> Business Korea
hms.harvard.edu -> Harvard Medical School
technode.com -> TechNode
www.gizmochina.com -> GizmoChina
www.amny.com -> amNewYork
aibusiness.com -> AI Business
seanpedersen.github.io -> Sean Pedersen
thehackernews.com -> The Hacker News
www.wiz.io -> Wiz
www.rstreet.org -> R Street Institute
www.tradingview.com -> TradingView
thewalrus.ca -> The Walrus
www.eurekalert.org -> EurekAlert
foreignpolicy.com -> Foreign Policy
www.khaleejtimes.com -> Khaleej Times
daily.jstor.org -> JSTOR Daily


In [44]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in AIdf.itertuples():
    insert_article(conn, cursor, row.src, row.hostname, row.title,
                   row.url, row.actual_url, row.isAI, row.date)


# Topic analysis
Try to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

### Fit dimensionality reduction model

In [45]:
# # train dimensionality reduction, only need to do this every few months and pickle the model to reflect new topics
# # extracts all links from history where isAI=1
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id desc limit 20000"
# ai_history_df = pd.read_sql_query(query, conn)
# len(ai_history_df)

In [46]:
# embedding_model="text-embedding-3-large"
# embedding_df_list = []
# pages = paginate_df(ai_history_df, maxpagelen=1000, max_input_tokens=8192)

# for p in pages:
#     response = client.embeddings.create(input=[obj['title'] for obj in p],
#                                         model=embedding_model)
#     embedding_df_list.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))

# embedding_df = pd.concat(embedding_df_list, axis=0, ignore_index=True)

# embedding_df.to_pickle("historical_embedding_df.pkl")


In [47]:
# # Initialize the UMAP reducer
# reducer = umap.UMAP(n_components=30)
# # Fit the reducer to the data without transforming
# reducer.fit(embedding_df)
# # Pickle the reducer
# with open('reducer.pkl', 'wb') as f:
#     pickle.dump(reducer, f)
# print("UMAP reducer pickled and saved as 'reducer.pkl'")

In [48]:
# attempt to extract top topics 
print(TOPIC_PROMPT)



You will act as a research assistant to extract topics from news headlines. You will extract topics, entities,
and keywords from news headlines formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 105,'title': "Former Microsoft CEO Steve Ballmer is now just as rich as his former boss Bill Gates. Here's how he spends his billions."},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
]

Output Specification:
You will return a JSON object with the field 'topics' containing a flat list of classification results.
For each headline input, your output will be a JSON object contain

In [49]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

2024-07-18 10:35:28,621 - AInewsbot - INFO - start topic extraction
2024-07-18 10:35:28,624 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:35:28,626 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:35:28,627 - AInewsbot - INFO - sent 50 items 
2024-07-18 10:35:28,627 - AInewsbot - INFO - sent 48 items 
2024-07-18 10:35:45,924 - AInewsbot - INFO - got dict with 48 items 
2024-07-18 10:35:46,637 - AInewsbot - INFO - got dict with 50 items 
2024-07-18 10:35:46,712 - AInewsbot - INFO - got dict with 50 items 
2024-07-18 10:35:49,111 - AInewsbot - INFO - got dict with 50 items 
2024-07-18 10:35:49,114 - AInewsbot - INFO - Processed 198 responses.
2024-07-18 10:35:49,116 - AInewsbot - INFO - end topic extraction


20

In [50]:
topic_df = pd.DataFrame(response)
topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})
print(len(topic_df))
topic_df.head()


198


,id,extracted_topics
0,0,"[AI, biology, proteins, innovation, technology]"
1,1,"[AI, startup, Fei-Fei Li, entrepreneurship]"
2,2,"[AI, ETF, investment, S&P 500, bull market]"
3,3,"[AI, stocks, investment, gains]"
4,4,"[AI, stocks, Wall Street, analysts, investment]"


In [51]:
all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
item_counts = Counter(all_topics)
filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
print(len(filtered_topics))
sorted(filtered_topics)


69


['ai data center',
 'ai features',
 'ai model',
 'ai models',
 'ai search',
 'ai vulnerabilities',
 'ai-enabled war drones',
 'android',
 'anthropic',
 'apple',
 'at&t',
 'biden',
 'bitcoin',
 'brazil',
 'breach',
 'chatgpt',
 'china',
 'ctera',
 'cybersecurity',
 'data security',
 'disney',
 'einstein',
 'elon musk',
 'ethics',
 'eu',
 'expectations',
 'fei-fei li',
 'generative ai',
 'generative ai tools',
 'google',
 'hackers',
 'hanwha energy',
 'healthcare',
 'hp',
 'innovation',
 'investment',
 'large language models',
 'machine learning',
 'meizu blue 20',
 'menlo ventures',
 'meta',
 'microsoft',
 'navigation',
 'nvidia',
 'openai',
 'openelm',
 'personal data',
 'privacy concerns',
 'privacy policy',
 'research',
 'researchers',
 'risk management',
 'robotics',
 'robots',
 'salesforce',
 'samsung',
 'sap',
 'security',
 'startup',
 'stocks',
 'strategy',
 'study',
 'texas',
 'text-to-image diffusion models',
 'tinder',
 'tsmc',
 'ukraine',
 'us-singapore',
 'youtube']

In [52]:
topic_df['extracted_topics'] = topic_df['extracted_topics'].apply(lambda l: [t.title() for t in l if t.lower() in filtered_topics])

In [53]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(CANONICAL_TOPICS)
for c in categories:
    print(c)

AI doom
AMD
Agriculture
Amazon
Anthropic
Apple
Art, Design
Artificial General Intelligence
Authors, Writing
Autonomous vehicles
Baidu
Big Tech
Bill Gates
Books, Publishing
Bubble
ChatGPT
China
Claude
Climate
Cloudflare
Code assistants
Cognitive Science
Cohere
Consciousness
Copilot
Cryptocurrency
Customer service
Cybersecurity
Data
Deals
Deepfakes
Disinformation
Disney
Economics
Education
Elon Musk
Energy
Entertainment
Ethics
European Union
Facial Recognition
Fashion
Finance
Fintech
Food, Drink
Funding
Gaming
Gemini
Gen AI
Geoffrey Hinton
Google
Health, Fitness
Healthcare
History
Hollywood
IPOs
India
Intellectual Property
Japan
Jobs, Careers, Labor Market
Korea
LLMs
Language Models
Laws
Lifestyle, Travel
Machine learning
Manufacturing
Mark Zuckerberg
Mergers and acquisitions
Meta
Microsoft
Military
Music
Mustafa Suleyman
Netflix
Nuclear
Nvidia
Open Source
OpenAI
Opinion
Perplexity
Policy and regulation
Politics
Privacy
Products
RAG
Recommendation systems
Reddit
Reinforcement Learning
Re

In [54]:
[t for t in filtered_topics if t.lower() not in [u.lower() for u in CANONICAL_TOPICS]]

['innovation',
 'startup',
 'fei-fei li',
 'investment',
 'navigation',
 'robotics',
 'biden',
 'risk management',
 'bitcoin',
 'at&t',
 'breach',
 'data security',
 'menlo ventures',
 'youtube',
 'privacy concerns',
 'openelm',
 'ctera',
 'hackers',
 'eu',
 'expectations',
 'ai model',
 'strategy',
 'study',
 'hp',
 'hanwha energy',
 'ai data center',
 'texas',
 'ai search',
 'security',
 'large language models',
 'generative ai',
 'ai models',
 'einstein',
 'meizu blue 20',
 'ai features',
 'generative ai tools',
 'brazil',
 'privacy policy',
 'personal data',
 'researchers',
 'android',
 'tsmc',
 'research',
 'sap',
 'ai vulnerabilities',
 'text-to-image diffusion models',
 'tinder',
 'us-singapore',
 'ukraine',
 'ai-enabled war drones']

In [55]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science
1,1,Google News,'Godmother of AI' Dr. Fei-Fei Li just built a ...,https://news.google.com/articles/CBMiTGh0dHBzO...,True,2024-07-18,https://qz.com/godmother-ai-dr-fei-fei-li-buil...,qz.com,Quartz
2,2,Feedly AI,1Artificial Intelligence(AI) ETF to Buy With $...,https://www.fool.com/investing/2024/07/18/1-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/1-ai...,www.fool.com,The Motley Fool
3,3,Google News,3 AI Stocks to Grab Now for 'Catch-Up' Gains,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.tradingview.com/news/barchart:d5ab...,www.tradingview.com,TradingView
4,4,Feedly AI,3Artificial Intelligence(AI) Stocks With Scorc...,https://www.fool.com/investing/2024/07/18/3-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/3-ai...,www.fool.com,The Motley Fool
...,...,...,...,...,...,...,...,...,...
193,193,Google News,Why business leaders view AI as an opportunity...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-18,https://fortune.com/2024/07/17/ai-business-lea...,fortune.com,Fortune
194,194,Feedly AI,"World's first ""MRI for cars"" launched in Hawai...",https://www.techspot.com/news/103863-world-fir...,True,2024-07-18,https://www.techspot.com/news/103863-world-fir...,www.techspot.com,TechSpot
195,195,Google News,You can now chat with Google's Gemini AI even ...,https://news.google.com/articles/CBMiigFodHRwc...,True,2024-07-18,https://www.techradar.com/computing/artificial...,www.techradar.com,TechRadar
196,196,Feedly AI,iPhone 16 Pro Max Leak Reveals Mind-Blowing 10...,https://www.geeky-gadgets.com/iphone-16-pro-ma...,True,2024-07-18,https://www.geeky-gadgets.com/iphone-16-pro-ma...,www.geeky-gadgets.com,Geeky Gadgets


In [56]:
# maybe try to set timeout in categorize_headline
catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_headline(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        
catdict

2024-07-18 10:35:49,187 - AInewsbot - INFO - Categorizing headline 1 of 198
2024-07-18 10:35:49,189 - AInewsbot - INFO - 'ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature'
2024-07-18 10:35:50,700 - AInewsbot - INFO - ['Machine learning', 'Meta', 'Science']
2024-07-18 10:35:50,701 - AInewsbot - INFO - Categorizing headline 2 of 198
2024-07-18 10:35:50,702 - AInewsbot - INFO - 'Godmother of AI' Dr. Fei-Fei Li just built a $1 billion startup
2024-07-18 10:35:54,437 - AInewsbot - INFO - ['Machine learning', 'Venture Capital']
2024-07-18 10:35:54,438 - AInewsbot - INFO - Categorizing headline 3 of 198
2024-07-18 10:35:54,439 - AInewsbot - INFO - 1Artificial Intelligence(AI) ETF to Buy With $40 During the S&P 500 Bull Market
2024-07-18 10:35:57,035 - AInewsbot - INFO - ['Finance', 'Stocks']
2024-07-18 10:35:57,038 - AInewsbot - INFO - Categorizing headline 4 of 198
2024-07-18 10:35:57,038 - AInewsbot - INFO - 3 AI Stocks to Grab Now 

2024-07-18 10:36:39,445 - AInewsbot - INFO - Apple Denies Using YouTube Videos To Train Its AI Features After Report Claims Tim Cook's Company Used Creators' Content Without Consent
2024-07-18 10:36:40,541 - AInewsbot - INFO - ['Apple', 'Big Tech', 'Ethics', 'Privacy']
2024-07-18 10:36:40,542 - AInewsbot - INFO - Categorizing headline 28 of 198
2024-07-18 10:36:40,543 - AInewsbot - INFO - Apple Trained Its AI on YouTube Transcripts Without Permission, Report Says
2024-07-18 10:36:42,001 - AInewsbot - INFO - ['Apple', 'Big Tech', 'Data', 'Ethics', 'Language Models', 'Machine learning', 'Privacy']
2024-07-18 10:36:42,003 - AInewsbot - INFO - Categorizing headline 29 of 198
2024-07-18 10:36:42,005 - AInewsbot - INFO - Apple says its OpenELM model doesn't power any AI features, including Apple Intelligence, after a report said Apple used YouTube subtitles to train the model
2024-07-18 10:36:46,299 - AInewsbot - INFO - ['Apple', 'Big Tech', 'Data', 'Ethics', 'Language Models', 'Machine lear

2024-07-18 10:37:33,069 - AInewsbot - INFO - Categorizing headline 53 of 198
2024-07-18 10:37:33,070 - AInewsbot - INFO - From reality to fantasy: Live2Diff AI brings instant video stylization to life
2024-07-18 10:37:34,711 - AInewsbot - INFO - ['Art, Design', 'Gen AI', 'Machine learning', 'Products', 'Science', 'Streaming']
2024-07-18 10:37:34,713 - AInewsbot - INFO - Categorizing headline 54 of 198
2024-07-18 10:37:34,714 - AInewsbot - INFO - GenAI sinks into the 'trough of disillusionment'. GenAI faces growing skepticism as it struggles to deliver on high expectations.
2024-07-18 10:37:35,837 - AInewsbot - INFO - ['AI doom', 'Bubble', 'Gen AI', 'Language Models', 'Machine learning', 'Review']
2024-07-18 10:37:35,838 - AInewsbot - INFO - Categorizing headline 55 of 198
2024-07-18 10:37:35,838 - AInewsbot - INFO - German AI startup DeepL releases its new translation model with support for German, Japanese, and Simplified Chinese to and from English, available in DeepL Pro
2024-07-18 

2024-07-18 10:38:08,263 - AInewsbot - INFO - Categorizing headline 79 of 198
2024-07-18 10:38:08,264 - AInewsbot - INFO - ICL and Lavie Bio Leverage Artificial Intelligence to Make Significant Advancement in Development of Yield Increasing Bio-Stimulants
2024-07-18 10:38:09,872 - AInewsbot - INFO - ['Agriculture', 'Deals', 'Gen AI', 'Machine learning', 'Meta', 'Products', 'Science']
2024-07-18 10:38:09,874 - AInewsbot - INFO - Categorizing headline 80 of 198
2024-07-18 10:38:09,875 - AInewsbot - INFO - I made a personalized workout plan with Claude AI  heres how to do it
2024-07-18 10:38:11,048 - AInewsbot - INFO - ['Claude', 'Gen AI', 'Health, Fitness', 'Machine learning', 'Products', 'Recommendation systems', 'Review', 'Robots']
2024-07-18 10:38:11,049 - AInewsbot - INFO - Categorizing headline 81 of 198
2024-07-18 10:38:11,050 - AInewsbot - INFO - Italy's antitrust agency launches a probe into Google and Alphabet over alleged unfair commercial practices related to consent requests G

2024-07-18 10:38:46,886 - AInewsbot - INFO - ['Big Tech', 'Data', 'Ethics', 'European Union', 'Gen AI', 'Intellectual Property', 'LLMs', 'Language Models', 'Laws', 'Machine learning', 'Meta', 'Policy and regulation', 'Privacy', 'Products', 'Robots']
2024-07-18 10:38:46,901 - AInewsbot - INFO - Categorizing headline 103 of 198
2024-07-18 10:38:46,903 - AInewsbot - INFO - Microsoft SpreadsheetLLM AI specifically designed for spreadsheets
2024-07-18 10:38:48,180 - AInewsbot - INFO - ['Big Tech', 'Data', 'Deals', 'LLMs', 'Machine learning', 'Meta', 'Microsoft', 'Products', 'Review']
2024-07-18 10:38:48,180 - AInewsbot - INFO - Categorizing headline 104 of 198
2024-07-18 10:38:48,181 - AInewsbot - INFO - Microsoft brings controversial AI app to iPhone and Android devices
2024-07-18 10:38:49,354 - AInewsbot - INFO - ['Big Tech', 'Copilot', 'Ethics', 'Facial Recognition', 'Language Models', 'Machine learning', 'Microsoft', 'Privacy', 'Products']
2024-07-18 10:38:49,355 - AInewsbot - INFO - Ca

2024-07-18 10:39:27,208 - AInewsbot - INFO - Categorizing headline 127 of 198
2024-07-18 10:39:27,208 - AInewsbot - INFO - Perplexity, Google, and the battle for AI search supremacy
2024-07-18 10:39:36,420 - AInewsbot - INFO - ['Big Tech', 'Google', 'Language Models', 'Machine learning', 'Perplexity']
2024-07-18 10:39:36,423 - AInewsbot - INFO - Categorizing headline 128 of 198
2024-07-18 10:39:36,424 - AInewsbot - INFO - Playground Ventures Invests in Ethical AI Firm
2024-07-18 10:39:38,365 - AInewsbot - INFO - ['Deals', 'Ethics', 'Funding', 'Venture Capital']
2024-07-18 10:39:38,366 - AInewsbot - INFO - Categorizing headline 129 of 198
2024-07-18 10:39:38,369 - AInewsbot - INFO - Playing the Long Game: How AI Can Optimize Enablement in Longtail Software Partnerships
2024-07-18 10:39:44,581 - AInewsbot - INFO - ['Opinion']
2024-07-18 10:39:44,585 - AInewsbot - INFO - Categorizing headline 130 of 198
2024-07-18 10:39:44,587 - AInewsbot - INFO - Rare earths in the red and an overhaul fo

2024-07-18 10:42:47,040 - AInewsbot - INFO - ['Economics', 'Manufacturing', 'Products', 'Semiconductor chips', 'Taiwan']
2024-07-18 10:42:47,042 - AInewsbot - INFO - Categorizing headline 154 of 198
2024-07-18 10:42:47,043 - AInewsbot - INFO - TSMC rides AI demand to raise revenue forecast, says no to US joint venture
2024-07-18 10:42:48,722 - AInewsbot - INFO - ['Deals', 'Economics', 'Manufacturing', 'Semiconductor chips', 'Taiwan']
2024-07-18 10:42:48,725 - AInewsbot - INFO - Categorizing headline 155 of 198
2024-07-18 10:42:48,725 - AInewsbot - INFO - TSMC second-quarter profit beats expectations as AI chip boom continues
2024-07-18 10:42:49,709 - AInewsbot - INFO - ['Economics', 'Manufacturing', 'Semiconductor chips', 'Taiwan']
2024-07-18 10:42:49,711 - AInewsbot - INFO - Categorizing headline 156 of 198
2024-07-18 10:42:49,712 - AInewsbot - INFO - TSMC smashed second quarter expectations due to AI chip demand
2024-07-18 10:42:51,211 - AInewsbot - INFO - ['Economics', 'Gen AI', 'Ma

2024-07-18 10:43:25,146 - AInewsbot - INFO - ['Deals', 'Politics']
2024-07-18 10:43:25,151 - AInewsbot - INFO - Categorizing headline 181 of 198
2024-07-18 10:43:25,152 - AInewsbot - INFO - Ukraine Builds AI Drone Fleets to Combat Russian Forces
2024-07-18 10:43:26,269 - AInewsbot - INFO - ['Autonomous vehicles', 'Machine learning', 'Military', 'Robots', 'Russia']
2024-07-18 10:43:26,271 - AInewsbot - INFO - Categorizing headline 182 of 198
2024-07-18 10:43:26,272 - AInewsbot - INFO - Ukraine Goes All-In on Ground Robots
2024-07-18 10:43:27,563 - AInewsbot - INFO - ['Military', 'Robots']
2024-07-18 10:43:27,565 - AInewsbot - INFO - Categorizing headline 183 of 198
2024-07-18 10:43:27,565 - AInewsbot - INFO - Ukraine Rushes to Create AI-Enabled War Drones
2024-07-18 10:43:29,187 - AInewsbot - INFO - ['AI doom', 'Gen AI', 'Machine learning', 'Military', 'Science']
2024-07-18 10:43:29,197 - AInewsbot - INFO - Categorizing headline 184 of 198
2024-07-18 10:43:29,198 - AInewsbot - INFO - Uk

{0: ['Machine learning', 'Meta', 'Science'],
 1: ['Machine learning', 'Venture Capital'],
 2: ['Finance', 'Stocks'],
 3: ['Finance', 'Stocks'],
 4: ['Finance', 'Stocks'],
 5: ['Autonomous vehicles',
  'Baidu',
  'Machine learning',
  'Meta',
  'Products',
  'Robots',
  'Science',
  'Testing',
  'Transportation'],
 6: ['Healthcare', 'Machine learning', 'Science'],
 7: ['Machine learning', 'Meta', 'OpenAI', 'Robots'],
 8: ['Disinformation', 'Ethics', 'Machine learning', 'Politics'],
 9: ['AI doom',
  'Big Tech',
  'Cognitive Science',
  'Data',
  'Gen AI',
  'Machine learning',
  'Meta',
  'OpenAI',
  'Science',
  'Singularity'],
 10: ['Cryptocurrency', 'Finance', 'Fintech', 'Machine learning'],
 11: ['Big Tech', 'Economics', 'Products', 'Semiconductor chips', 'Stocks'],
 12: ['Big Tech',
  'Data',
  'Ethics',
  'European Union',
  'Machine learning',
  'Meta',
  'Policy and regulation',
  'Privacy'],
 13: ['Big Tech',
  'Cybersecurity',
  'Data',
  'Ethics',
  'Machine learning',
  'Pol

In [57]:
topic_df['assigned_topics'] = topic_df['id'].apply(lambda id: catdict.get(id, ""))
topic_df

,id,extracted_topics,assigned_topics
0,0,[Innovation],"[Machine learning, Meta, Science]"
1,1,"[Startup, Fei-Fei Li]","[Machine learning, Venture Capital]"
2,2,[Investment],"[Finance, Stocks]"
3,3,"[Stocks, Investment]","[Finance, Stocks]"
4,4,"[Stocks, Investment]","[Finance, Stocks]"
...,...,...,...
193,193,[],"[Machine learning, Opinion]"
194,194,[],"[Machine learning, Products, Science, Testing,..."
195,195,[Google],"[Big Tech, Gemini, Gen AI, Google, Language Mo..."
196,196,[],"[Apple, Big Tech, Gen AI, Machine learning, Pr..."


In [62]:
lcategories = set([c.lower() for c in categories])

In [63]:
def clean_topics(row):
    topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]
    return combined

topic_df["topics"] = topic_df.apply(clean_topics, axis=1)
topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)
topic_df

,id,extracted_topics,assigned_topics,topics,topic_str
0,0,[Innovation],"[Machine learning, Meta, Science]","[Innovation, Machine Learning, Meta, Science]","Innovation, Machine Learning, Meta, Science"
1,1,"[Startup, Fei-Fei Li]","[Machine learning, Venture Capital]","[Fei-Fei Li, Machine Learning, Startup, Ventur...","Fei-Fei Li, Machine Learning, Startup, Venture..."
2,2,[Investment],"[Finance, Stocks]","[Finance, Investment, Stocks]","Finance, Investment, Stocks"
3,3,"[Stocks, Investment]","[Finance, Stocks]","[Finance, Investment, Stocks]","Finance, Investment, Stocks"
4,4,"[Stocks, Investment]","[Finance, Stocks]","[Finance, Investment, Stocks]","Finance, Investment, Stocks"
...,...,...,...,...,...
193,193,[],"[Machine learning, Opinion]","[Machine Learning, Opinion]","Machine Learning, Opinion"
194,194,[],"[Machine learning, Products, Science, Testing,...","[Machine Learning, Products, Science, Testing,...","Machine Learning, Products, Science, Testing, ..."
195,195,[Google],"[Big Tech, Gemini, Gen AI, Google, Language Mo...","[Big Tech, Gemini, Gen AI, Google, Language Mo...","Big Tech, Gemini, Gen AI, Google, Language Mod..."
196,196,[],"[Apple, Big Tech, Gen AI, Machine learning, Pr...","[Apple, Big Tech, Gen AI, Machine Learning, Pr...","Apple, Big Tech, Gen AI, Machine Learning, Pro..."


In [65]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str_x,title_topic_str,topic_str_y
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science,"Innovation, Machine Learning, Meta, Science",'ChatGPT moment for biology': Ex-Meta scientis...,"Innovation, Machine Learning, Meta, Science"
1,1,Google News,'Godmother of AI' Dr. Fei-Fei Li just built a ...,https://news.google.com/articles/CBMiTGh0dHBzO...,True,2024-07-18,https://qz.com/godmother-ai-dr-fei-fei-li-buil...,qz.com,Quartz,"Fei-Fei Li, Machine Learning, Startup, Venture...",'Godmother of AI' Dr. Fei-Fei Li just built a ...,"Fei-Fei Li, Machine Learning, Startup, Venture..."
2,2,Feedly AI,1Artificial Intelligence(AI) ETF to Buy With $...,https://www.fool.com/investing/2024/07/18/1-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/1-ai...,www.fool.com,The Motley Fool,"Finance, Investment, Stocks",1Artificial Intelligence(AI) ETF to Buy With $...,"Finance, Investment, Stocks"
3,3,Google News,3 AI Stocks to Grab Now for 'Catch-Up' Gains,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.tradingview.com/news/barchart:d5ab...,www.tradingview.com,TradingView,"Finance, Investment, Stocks",3 AI Stocks to Grab Now for 'Catch-Up' Gains (...,"Finance, Investment, Stocks"
4,4,Feedly AI,3Artificial Intelligence(AI) Stocks With Scorc...,https://www.fool.com/investing/2024/07/18/3-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/3-ai...,www.fool.com,The Motley Fool,"Finance, Investment, Stocks",3Artificial Intelligence(AI) Stocks With Scorc...,"Finance, Investment, Stocks"
...,...,...,...,...,...,...,...,...,...,...,...,...
193,193,Google News,Why business leaders view AI as an opportunity...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-18,https://fortune.com/2024/07/17/ai-business-lea...,fortune.com,Fortune,"Machine Learning, Opinion",Why business leaders view AI as an opportunity...,"Machine Learning, Opinion"
194,194,Feedly AI,"World's first ""MRI for cars"" launched in Hawai...",https://www.techspot.com/news/103863-world-fir...,True,2024-07-18,https://www.techspot.com/news/103863-world-fir...,www.techspot.com,TechSpot,"Machine Learning, Products, Science, Testing, ...","World's first ""MRI for cars"" launched in Hawai...","Machine Learning, Products, Science, Testing, ..."
195,195,Google News,You can now chat with Google's Gemini AI even ...,https://news.google.com/articles/CBMiigFodHRwc...,True,2024-07-18,https://www.techradar.com/computing/artificial...,www.techradar.com,TechRadar,"Big Tech, Gemini, Gen AI, Google, Language Mod...",You can now chat with Google's Gemini AI even ...,"Big Tech, Gemini, Gen AI, Google, Language Mod..."
196,196,Feedly AI,iPhone 16 Pro Max Leak Reveals Mind-Blowing 10...,https://www.geeky-gadgets.com/iphone-16-pro-ma...,True,2024-07-18,https://www.geeky-gadgets.com/iphone-16-pro-ma...,www.geeky-gadgets.com,Geeky Gadgets,"Apple, Big Tech, Gen AI, Machine Learning, Pro...",iPhone 16 Pro Max Leak Reveals Mind-Blowing 10...,"Apple, Big Tech, Gen AI, Machine Learning, Pro..."


In [75]:
try:  # for idempotency
    AIdf = AIdf.drop(columns=["title_topic_str"])
except:
    pass
try:  # for idempotency
    AIdf = AIdf.drop(columns=["topic_str"])
except:
    pass

In [76]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science
1,1,Google News,'Godmother of AI' Dr. Fei-Fei Li just built a ...,https://news.google.com/articles/CBMiTGh0dHBzO...,True,2024-07-18,https://qz.com/godmother-ai-dr-fei-fei-li-buil...,qz.com,Quartz
2,2,Feedly AI,1Artificial Intelligence(AI) ETF to Buy With $...,https://www.fool.com/investing/2024/07/18/1-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/1-ai...,www.fool.com,The Motley Fool
3,3,Google News,3 AI Stocks to Grab Now for 'Catch-Up' Gains,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.tradingview.com/news/barchart:d5ab...,www.tradingview.com,TradingView
4,4,Feedly AI,3Artificial Intelligence(AI) Stocks With Scorc...,https://www.fool.com/investing/2024/07/18/3-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/3-ai...,www.fool.com,The Motley Fool
...,...,...,...,...,...,...,...,...,...
193,193,Google News,Why business leaders view AI as an opportunity...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-18,https://fortune.com/2024/07/17/ai-business-lea...,fortune.com,Fortune
194,194,Feedly AI,"World's first ""MRI for cars"" launched in Hawai...",https://www.techspot.com/news/103863-world-fir...,True,2024-07-18,https://www.techspot.com/news/103863-world-fir...,www.techspot.com,TechSpot
195,195,Google News,You can now chat with Google's Gemini AI even ...,https://news.google.com/articles/CBMiigFodHRwc...,True,2024-07-18,https://www.techradar.com/computing/artificial...,www.techradar.com,TechRadar
196,196,Feedly AI,iPhone 16 Pro Max Leak Reveals Mind-Blowing 10...,https://www.geeky-gadgets.com/iphone-16-pro-ma...,True,2024-07-18,https://www.geeky-gadgets.com/iphone-16-pro-ma...,www.geeky-gadgets.com,Geeky Gadgets


In [77]:
AIdf = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="inner")
AIdf['title_topic_str'] = AIdf.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science,"Innovation, Machine Learning, Meta, Science",'ChatGPT moment for biology': Ex-Meta scientis...
1,1,Google News,'Godmother of AI' Dr. Fei-Fei Li just built a ...,https://news.google.com/articles/CBMiTGh0dHBzO...,True,2024-07-18,https://qz.com/godmother-ai-dr-fei-fei-li-buil...,qz.com,Quartz,"Fei-Fei Li, Machine Learning, Startup, Venture...",'Godmother of AI' Dr. Fei-Fei Li just built a ...
2,2,Feedly AI,1Artificial Intelligence(AI) ETF to Buy With $...,https://www.fool.com/investing/2024/07/18/1-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/1-ai...,www.fool.com,The Motley Fool,"Finance, Investment, Stocks",1Artificial Intelligence(AI) ETF to Buy With $...
3,3,Google News,3 AI Stocks to Grab Now for 'Catch-Up' Gains,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.tradingview.com/news/barchart:d5ab...,www.tradingview.com,TradingView,"Finance, Investment, Stocks",3 AI Stocks to Grab Now for 'Catch-Up' Gains (...
4,4,Feedly AI,3Artificial Intelligence(AI) Stocks With Scorc...,https://www.fool.com/investing/2024/07/18/3-ai...,True,2024-07-18,https://www.fool.com/investing/2024/07/18/3-ai...,www.fool.com,The Motley Fool,"Finance, Investment, Stocks",3Artificial Intelligence(AI) Stocks With Scorc...
...,...,...,...,...,...,...,...,...,...,...,...
193,193,Google News,Why business leaders view AI as an opportunity...,https://news.google.com/articles/CBMiVWh0dHBzO...,True,2024-07-18,https://fortune.com/2024/07/17/ai-business-lea...,fortune.com,Fortune,"Machine Learning, Opinion",Why business leaders view AI as an opportunity...
194,194,Feedly AI,"World's first ""MRI for cars"" launched in Hawai...",https://www.techspot.com/news/103863-world-fir...,True,2024-07-18,https://www.techspot.com/news/103863-world-fir...,www.techspot.com,TechSpot,"Machine Learning, Products, Science, Testing, ...","World's first ""MRI for cars"" launched in Hawai..."
195,195,Google News,You can now chat with Google's Gemini AI even ...,https://news.google.com/articles/CBMiigFodHRwc...,True,2024-07-18,https://www.techradar.com/computing/artificial...,www.techradar.com,TechRadar,"Big Tech, Gemini, Gen AI, Google, Language Mod...",You can now chat with Google's Gemini AI even ...
196,196,Feedly AI,iPhone 16 Pro Max Leak Reveals Mind-Blowing 10...,https://www.geeky-gadgets.com/iphone-16-pro-ma...,True,2024-07-18,https://www.geeky-gadgets.com/iphone-16-pro-ma...,www.geeky-gadgets.com,Geeky Gadgets,"Apple, Big Tech, Gen AI, Machine Learning, Pro...",iPhone 16 Pro Max Leak Reveals Mind-Blowing 10...


In [78]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(AIdf.loc[AIdf["topic_str"]==""][['title']])


,title
129,Rare earths in the red and an overhaul for AI infrastructure
188,What happens when AI gets the beauty treatment
190,"Where the creative industry is heading, and how to survive the next 15 years"


### Semantic sort

In [79]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

# sort of a traveling salesman sort
log(f"Sort with agglomerative cluster sort")
sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf['sort_order'] = sorted_indices

# do dimensionality reduction on embedding_df and cluster analysis
log(f"Perform dimensionality reduction")
with open("reducer.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
reduced_data = reducer.transform(embedding_df)
log(f"Cluster with DBSCAN")
dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
AIdf['cluster'] = dbscan.fit_predict(reduced_data)
AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999
    
# sort first by clusters found by DBSCAN, then by semantic ordering
AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

AIdf


2024-07-18 10:49:41,794 - AInewsbot - INFO - Fetching embeddings for 198 headlines
2024-07-18 10:49:42,493 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 10:49:42,983 - AInewsbot - INFO - Sort with agglomerative cluster sort
2024-07-18 10:49:43,057 - AInewsbot - INFO - Perform dimensionality reduction
2024-07-18 10:49:47,428 - AInewsbot - INFO - Cluster with DBSCAN


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science,"Innovation, Machine Learning, Meta, Science",'ChatGPT moment for biology': Ex-Meta scientis...,66,0
1,1,Feedly AI,AI & Drug Discovery With Harren Jhoti: Vanguar...,https://www.bloomberg.com/news/audio/2024-07-1...,True,2024-07-18,https://www.bloomberg.com/news/audio/2024-07-1...,www.bloomberg.com,Bloomberg,"Healthcare, Machine Learning, Science",AI & Drug Discovery With Harren Jhoti: Vanguar...,94,0
2,2,Google News,How Machine Learning Is Propelling Structural ...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-18,https://hms.harvard.edu/news/how-machine-learn...,hms.harvard.edu,Harvard Medical School,"Data, Machine Learning, Science",How Machine Learning Is Propelling Structural ...,156,0
3,3,Google News,nference Establishes AI Initiative with Takeda...,https://news.google.com/articles/CBMiowFodHRwc...,True,2024-07-18,https://www.businesswire.com/news/home/2024071...,www.businesswire.com,Business Wire,"Deals, Health, Fitness, Healthcare, Machine Le...",nference Establishes AI Initiative with Takeda...,168,0
4,4,Google News,56-gram drone gets ant-inspired AI eyes to nav...,https://news.google.com/articles/CBMiTmh0dHBzO...,True,2024-07-18,https://interestingengineering.com/innovation/...,interestingengineering.com,Interesting Engineering,"Autonomous Vehicles, Baidu, Machine Learning, ...",56-gram drone gets ant-inspired AI eyes to nav...,22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,193,Google News,Ruled by robots: Research reveals people prefe...,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.newswise.com/articles/ruled-by-rob...,www.newswise.com,Newswise,"AI Doom, Artificial General Intelligence, Gen ...",Ruled by robots: Research reveals people prefe...,190,999
194,194,Google News,Maths test stumps AI models: which number is b...,https://news.google.com/articles/CBMicGh0dHBzO...,True,2024-07-18,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"AI Models, Cognitive Science, Education, Gen A...",Maths test stumps AI models: which number is b...,192,999
195,195,Reddit,GenAI sinks into the 'trough of disillusionmen...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-18,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit,"AI Doom, Bubble, Expectations, Gen AI, Languag...",GenAI sinks into the 'trough of disillusionmen...,194,999
196,196,HackerNoon,The Chosen One: Consistent Characters in Text-...,https://hackernoon.com/the-chosen-one-consiste...,True,2024-07-18,https://hackernoon.com/the-chosen-one-consiste...,hackernoon.com,Hacker Noon,Text-To-Image Diffusion Models,The Chosen One: Consistent Characters in Text-...,196,999


In [80]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):

    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses. 
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided 
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
            response = await fetch_openai(session, payload)
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}
        

# show clusters
cluster_topics = []
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    async with aiohttp.ClientSession() as session:

        for i in range(30):
            tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
            if len(tmpdf) ==0:
                break
            display(tmpdf)
            title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
            cluster_topic = await write_topic_name(session, title_topic_str_list)
            cluster_topics.append(cluster_topic)
            print(cluster_topic)

    


,id,title_topic_str
0,0,"'ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature' (Topics: Innovation, Machine Learning, Meta, Science)"
1,1,"AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care (Topics: Healthcare, Machine Learning, Science)"
2,2,"How Machine Learning Is Propelling Structural Biology (Topics: Data, Machine Learning, Science)"
3,3,"nference Establishes AI Initiative with Takeda to Advance Precision Medicine in Inflammatory Bowel Disease (Topics: Deals, Health, Fitness, Healthcare, Machine Learning, Science)"


{'topic_title': 'AI in Healthcare & Precision Medicine'}


,id,title_topic_str
4,4,"56-gram drone gets ant-inspired AI eyes to navigate autonomously (Topics: Autonomous Vehicles, Baidu, Machine Learning, Meta, Navigation, Products, Robots, Science, Testing, Transportation)"
5,5,"Tiny drone ""CrazyFlie"" weighs only 56 grams (IMAGE) (Topics: Manufacturing, Open Source, Products, Robots, Science, Testing)"
6,6,"Bug brains could help drone swarms find their way home (Topics: Autonomous Vehicles, Cognitive Science, Navigation, Robots, Science)"


{'topic_title': 'Autonomous Drones'}


,id,title_topic_str
7,7,"Andreessen Horowitz-Backed 'Godmother Of AI' Fei-Fei Li Quietly Builds $1B Startup In 4 Months (Topics: Deals, Fei-Fei Li, Funding, Investment, Machine Learning, Startup, Venture Capital)"
8,8,"Artificial Agency raises $16M to bring AI-powered behavior to games (Topics: Deals, Funding, Gaming, Gen AI, Machine Learning, Rag, Reinforcement Learning, Venture Capital)"
9,9,"Godmother of A.I. Fei-Fei Li Has Created a $1B Startup in Less Than 4 Months (Topics: Fei-Fei Li, Funding, Startup, Venture Capital)"
10,10,"Hybrid cloud data platform CTERA bags $80 million, mostly in secondary funding (Topics: Ctera, Data, Deals, Funding, Products, Venture Capital)"
11,11,"The godmother of AI has a new startup already worth $1 billion (Topics: Funding, OpenAI, Startup)"
12,12,"CTERA gets $80m war chest for AI and acquisitions (Topics: Ctera, Deals, Funding, Investment, Mergers And Acquisitions, Venture Capital)"
13,13,"Ctera Co-Founder On New $80M Growth Funding, Plans For AI Intelligence In File Services (Topics: Ctera, Data, Deals, Funding, Venture Capital)"
14,14,"Anthropic and Menlo Ventures Partner on $100 Million AI Fund (Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital)"


...

{'topic_title': 'AI Impact and Ethics'}


,id,title_topic_str
69,69,The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Related Work (Topics: Text-To-Image Diffusion Models)
70,70,"The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Abstract and Introduction (Topics: OpenAI, Text-To-Image Diffusion Models)"
71,71,"The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Implementation Details (Topics: Machine Learning, Text-To-Image Diffusion Models)"
72,72,"The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Limitations and Conclusions (Topics: Machine Learning, OpenAI, Review, Science, Text-To-Image Diffusion Models)"
73,73,"The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Experiments (Topics: Cohere, Language Models, Machine Learning, OpenAI, Science, Text-To-Image Diffusion Models)"
74,74,"The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Societal Impact (Topics: Machine Learning, Society, Culture, Text-To-Image Diffusion Models)"
75,75,"The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Additional Experiments (Topics: Machine Learning, OpenAI, Science, Text-To-Image Diffusion Models)"


{'topic_title': 'Text-To-Image Diffusion Models'}


,id,title_topic_str
76,76,"ChatGPT has eyes (Topics: Chatgpt, OpenAI)"
77,77,"The Fastest Way to Lose a Court Case? Use ChatGPT (Topics: Chatgpt, Code Assistants, Cohere, Gen AI, Language Models, Llms, OpenAI)"
78,78,"The AI doctor will see you now: ChatGPT dominates medical exam (Topics: Chatgpt, Language Models, Machine Learning, OpenAI)"


{'topic_title': 'ChatGPT in AI Dominance'}


In [ ]:
# we could extract top words using tfidf, something like 
# vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = vectorizer.fit_transform(documents)
# feature_names = vectorizer.get_feature_names_out()
# topics = []
# for i in range(n_topics):
#     # Get the documents in this cluster
#     cluster_docs = [doc for doc, label in zip(documents, cluster_labels) if label == i]
#     cluster_metadatas = [meta for meta, label in zip(metadatas, cluster_labels) if label == i]

#     # Get the top words for this cluster based on TF-IDF scores
#     tfidf_scores = tfidf_matrix[cluster_labels == i].sum(axis=0).A1
#     top_word_indices = tfidf_scores.argsort()[-n_words_per_topic:][::-1]
#     top_words = [feature_names[index] for index in top_word_indices]


In [81]:
cluster_topic_list = [obj['topic_title'] for obj in cluster_topics]
cluster_topic_list

['AI in Healthcare & Precision Medicine',
 'Autonomous Drones',
 'AI Startup Funding',
 'Claude AI for Data Analysis',
 'AI Startup Funding',
 'AI in Big Tech Products',
 'AI-Powered Smartphones',
 'Big Tech Antitrust Investigations',
 'Ukrainian Military Robotics',
 'Large Language Models',
 'AI-Powered Customer Service Agents',
 'AI Ethics and Regulation',
 'AI Art by Microsoft',
 'AI Chip Demand Surge',
 'AI and Nvidia Partnerships',
 'AI Impact and Ethics',
 'Text-To-Image Diffusion Models',
 'ChatGPT in AI Dominance']

In [82]:
AIdf['cluster_name'] = AIdf['cluster'].apply(lambda i: cluster_topic_list[i] if i<len(cluster_topic_list) else "")


# Save and email headlines


In [83]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.site_name}]({row.actual_url})")
    html_str += f'{row.Index}.<a href="{row.actual_url}">{row.title} - {row.site_name}</a><br />\n'


2024-07-18 10:50:15,669 - AInewsbot - INFO - [0. 'ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature' - Live Science](https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature)
2024-07-18 10:50:15,672 - AInewsbot - INFO - [1. AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)
2024-07-18 10:50:15,673 - AInewsbot - INFO - [2. How Machine Learning Is Propelling Structural Biology - Harvard Medical School](https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology)
2024-07-18 10:50:15,673 - AInewsbot - INFO - [3. nference Establishes AI Initiative with Takeda to Advance Precision Medicine in Inflammatory Bowel Disease - Business Wire](https://www.businesswire.com/news/home/2024

2024-07-18 10:50:15,682 - AInewsbot - INFO - [34. Ukraine Rushes to Create AI-Enabled War Drones - Newsmax](https://www.newsmax.com/world/globaltalk/ukraine-war-drones/2024/07/18/id/1173032/)
2024-07-18 10:50:15,682 - AInewsbot - INFO - [35. Ukraine rushes to create AI-enabled war drones - Reuters](https://www.reuters.com/technology/artificial-intelligence/ukraine-rushes-create-ai-enabled-war-drones-2024-07-18/)
2024-07-18 10:50:15,682 - AInewsbot - INFO - [36. Ukraine Builds AI Drone Fleets to Combat Russian Forces - PCMag](https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces)
2024-07-18 10:50:15,683 - AInewsbot - INFO - [37. Ukraine Goes All-In on Ground Robots - Foreign Policy](https://foreignpolicy.com/2024/07/17/ukraine-russia-war-ground-robots-combat/)
2024-07-18 10:50:15,683 - AInewsbot - INFO - [38. From basement to battlefield: Ukrainian startups create low-cost robots to fight Russia - Associated Press News](https://apnews.com/article/ukrain

2024-07-18 10:50:15,689 - AInewsbot - INFO - [69. The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Related Work - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-related-work)
2024-07-18 10:50:15,690 - AInewsbot - INFO - [70. The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Abstract and Introduction - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-abstract-and-introduction)
2024-07-18 10:50:15,690 - AInewsbot - INFO - [71. The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Implementation Details - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-implementation-details)
2024-07-18 10:50:15,691 - AInewsbot - INFO - [72. The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Limitations and Conclusions - Hacker Noon](https://hac

2024-07-18 10:50:15,697 - AInewsbot - INFO - [104. 3Artificial Intelligence(AI) Stocks With Scorching-Hot Upside of Up to 100%, According to Select Wall Street Analysts - The Motley Fool](https://www.fool.com/investing/2024/07/18/3-ai-stocks-scorching-hot-upside-100-wall-street/)
2024-07-18 10:50:15,698 - AInewsbot - INFO - [105. Tinder Adds AI to Help You Pick Your Best Photos and Get More Matches - PetaPixel](https://petapixel.com/2024/07/17/tinder-adds-ai-to-help-you-pick-your-best-photos-and-get-more-matches/)
2024-07-18 10:50:15,698 - AInewsbot - INFO - [106. Japanese media say AI search infringes copyright, urge legal reform - Reddit](https://www.reddit.com/r/technology/comments/1e5zuiv/japanese_media_say_ai_search_infringes_copyright/)
2024-07-18 10:50:15,699 - AInewsbot - INFO - [107. Why business leaders view AI as an opportunity to take toil out of our work - Fortune](https://fortune.com/2024/07/17/ai-business-leaders-workplace-workday-mckinsey-lenovo/)
2024-07-18 10:50:15,69

2024-07-18 10:50:15,712 - AInewsbot - INFO - [138. Watch Steve Jobs describe the future and AI a year before the Mac - Apple Insider](https://appleinsider.com/articles/24/07/18/watch-steve-jobs-describe-the-future-and-ai-a-year-before-the-mac)
2024-07-18 10:50:15,713 - AInewsbot - INFO - [139. What Musks Trump support means for X - The Washington Post](https://www.washingtonpost.com/politics/2024/07/17/musk-trump-endorsement-x-moderation-bias/)
2024-07-18 10:50:15,713 - AInewsbot - INFO - [140. United States and Singapore Sign SOI to Strengthen Data, Analytics and Artificial Intelligence Cooperation - Department of Defense](https://www.defense.gov/News/Releases/Release/Article/3839100/united-states-and-singapore-sign-soi-to-strengthen-data-analytics-and-artificia/)
2024-07-18 10:50:15,713 - AInewsbot - INFO - [141. What happens when AI gets the beauty treatment - Khaleej Times](https://www.khaleejtimes.com/lifestyle/what-happens-when-ai-gets-the-beauty-treatment)
2024-07-18 10:50:15,71

2024-07-18 10:50:15,729 - AInewsbot - INFO - [170. Meta AI on WhatsApp is getting this basic Maths question wrong and why it may not be alone - Times of India](https://timesofindia.indiatimes.com/technology/tech-news/meta-ai-on-whatsapp-is-getting-this-basic-maths-question-wrong/articleshow/111802337.cms)
2024-07-18 10:50:15,729 - AInewsbot - INFO - [171. How to Plan for Your Next Career Move in Data Science and Machine Learning - Towards Data Science](https://towardsdatascience.com/how-to-plan-for-your-next-career-move-in-data-science-and-machine-learning-36c9d6f8cce6)
2024-07-18 10:50:15,729 - AInewsbot - INFO - [172. HPs new AI computer raises the stakes in the battle of tech hardware - Semafor](https://www.semafor.com/article/07/17/2024/hps-new-ai-computer-raises-the-stakes-in-the-battle-of-tech-hardware)
2024-07-18 10:50:15,729 - AInewsbot - INFO - [173. Hanwha Energy builds green AI data center in Texas, eyes $7.5 billion tech revenue - DigiTimes](https://www.digitimes.com/news/a

In [84]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [85]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-18 10:50:18,004 - AInewsbot - INFO - Sending headlines email


# Save individual pages 

In [86]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.actual_url, row.title))


2024-07-18 10:50:22,684 - AInewsbot - INFO - Queuing URLs for scraping


In [87]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


2024-07-18 10:50:23,345 - AInewsbot - INFO - fetching 198 pages using 4 browsers
2024-07-18 10:50:23,348 - AInewsbot - INFO - get_driver - 59284 Initializing webdriver
2024-07-18 10:50:23,354 - AInewsbot - INFO - get_driver - 59284 Initializing webdriver
2024-07-18 10:50:23,354 - AInewsbot - INFO - get_driver - 59284 Initializing webdriver
2024-07-18 10:50:23,355 - AInewsbot - INFO - get_driver - 59284 Initializing webdriver
2024-07-18 10:50:50,613 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-18 10:50:50,629 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-18 10:50:50,629 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-18 10:50:50,629 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-18 10:50:50,659 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-18 10:50:50,660 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-18 10:50:50,661 - AInewsbot - INFO 

2024-07-18 10:54:01,042 - AInewsbot - INFO - get_url(Artificial Agency raises $16M to bring AI-powered behavior to games) - Saving Artificial_Agency_raises__16M_to_bring_AI-powered_behavior_to_games_20240718_105401.html as utf-8
2024-07-18 10:54:01,044 - AInewsbot - INFO - Processing https://blocksandfiles.com/2024/07/17/ctera-gets-funding-for-ai-and-acquisitions/
2024-07-18 10:54:01,045 - AInewsbot - INFO - get_url(https://blocksandfiles.com/2024/07/17/ctera-gets-funding-for-ai-and-acquisitions/) - starting get_url https://blocksandfiles.com/2024/07/17/ctera-gets-funding-for-ai-and-acquisitions/
2024-07-18 10:54:08,257 - AInewsbot - INFO - get_url(Godmother of A.I. Fei-Fei Li Has Created a $1B Startup in Less Than 4 Months) - Saving Godmother_of_A_I__Fei-Fei_Li_Has_Created_a__1B_Startup_in_Less_Than_4_Months_20240718_105408.html as utf-8
2024-07-18 10:54:08,262 - AInewsbot - INFO - Processing https://www.crn.com/news/storage/2024/ctera-ceo-on-new-80m-growth-funding-plans-for-ai-intell

2024-07-18 10:54:35,795 - AInewsbot - INFO - get_url(https://www.scmp.com/tech/big-tech/article/3270977/chinas-foldable-smartphone-market-heats-samsung-bytedance-ai-pact-xiaomi-launch) - starting get_url https://www.scmp.com/tech/big-tech/article/3270977/chinas-foldable-smartphone-market-heats-samsung-bytedance-ai-pact-xiaomi-launch
2024-07-18 10:54:43,185 - AInewsbot - INFO - get_url(Samsung's AI Makes Images That Are Uncanny Real) - Saving Samsung_s_AI_Makes_Images_That_Are_Uncanny_Real_20240718_105443.html as utf-8
2024-07-18 10:54:43,191 - AInewsbot - INFO - Processing https://www.gizchina.com/2024/07/16/discover-the-meizu-blue-20-ai-affordable-ai-powered-smartphone/
2024-07-18 10:54:43,192 - AInewsbot - INFO - get_url(https://www.gizchina.com/2024/07/16/discover-the-meizu-blue-20-ai-affordable-ai-powered-smartphone/) - starting get_url https://www.gizchina.com/2024/07/16/discover-the-meizu-blue-20-ai-affordable-ai-powered-smartphone/
2024-07-18 10:54:47,486 - AInewsbot - INFO - ge

2024-07-18 10:55:25,297 - AInewsbot - INFO - get_url(European Commission Asking About Google-Samsung AI Deal) - Saving European_Commission_Asking_About_Google-Samsung_AI_Deal_20240718_105525.html as utf-8
2024-07-18 10:55:25,302 - AInewsbot - INFO - Processing https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces
2024-07-18 10:55:25,310 - AInewsbot - INFO - get_url(https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces) - starting get_url https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces
2024-07-18 10:55:29,759 - AInewsbot - INFO - get_url(Microsoft faces UK antitrust probe after hiring Inflection AI founders and employees) - Saving Microsoft_faces_UK_antitrust_probe_after_hiring_Inflection_AI_founders_and_employees_20240718_105529.html as utf-8
2024-07-18 10:55:29,771 - AInewsbot - INFO - Processing https://foreignpolicy.com/2024/07/17/ukraine-russia-war-ground-robots-combat/
2024-

2024-07-18 10:56:01,471 - AInewsbot - INFO - get_url(https://www.reuters.com/technology/artificial-intelligence/meta-decides-suspend-its-generative-ai-tools-brazil-2024-07-17/) - starting get_url https://www.reuters.com/technology/artificial-intelligence/meta-decides-suspend-its-generative-ai-tools-brazil-2024-07-17/
2024-07-18 10:56:04,969 - AInewsbot - INFO - get_url(Meet Salesforce's AI-Powered Customer Service Agent, Einstein) - Saving Meet_Salesforce_s_AI-Powered_Customer_Service_Agent__Einstein_20240718_105604.html as utf-8
2024-07-18 10:56:05,030 - AInewsbot - INFO - Processing https://www.techtimes.com/articles/306734/20240718/meta-s-new-multimodal-ai-model-launch-eu-amidst-regulator.htm
2024-07-18 10:56:05,053 - AInewsbot - INFO - get_url(https://www.techtimes.com/articles/306734/20240718/meta-s-new-multimodal-ai-model-launch-eu-amidst-regulator.htm) - starting get_url https://www.techtimes.com/articles/306734/20240718/meta-s-new-multimodal-ai-model-launch-eu-amidst-regulator.

...

2024-07-18 11:02:17,776 - AInewsbot - INFO - get_url(https://www.laptopmag.com/software/creative-media-apps/microsoft-brings-controversial-ai-app-to-iphone-and-android-devices) - starting get_url https://www.laptopmag.com/software/creative-media-apps/microsoft-brings-controversial-ai-app-to-iphone-and-android-devices
2024-07-18 11:02:21,995 - AInewsbot - INFO - get_url(TTT models might be the next frontier in generative AI) - Saving TTT_models_might_be_the_next_frontier_in_generative_AI_20240718_110221.html as utf-8
2024-07-18 11:02:22,107 - AInewsbot - INFO - Processing https://www.freethink.com/robots-ai/ai-search-engines-llms
2024-07-18 11:02:22,115 - AInewsbot - INFO - get_url(https://www.freethink.com/robots-ai/ai-search-engines-llms) - starting get_url https://www.freethink.com/robots-ai/ai-search-engines-llms
2024-07-18 11:02:34,355 - AInewsbot - INFO - get_url(A short history of AI, and what it is (and isnt)) - Saving A_short_history_of_AI__and_what_it_is__and_isnt__20240718_11

2024-07-18 11:03:15,779 - AInewsbot - INFO - Processing https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai
2024-07-18 11:03:15,780 - AInewsbot - INFO - get_url(https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai) - starting get_url https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai
2024-07-18 11:03:17,039 - AInewsbot - INFO - get_url(Kaspersky gives US customers six months of free updates as a parting giftUpdatedSo long, farewell, do svidaniya, goodbyeSecurity20 hrs|11) - Saving Kaspersky_gives_US_customers_six_months_of_free_updates_as_a_parting_giftUpdatedSo_long__farewell__do_svidaniya__goodbyeSecurity20_hrs_11_20240718_110317.html as utf-8
2024-07-18 11:03:17,062 - AInewsbot - INFO - Processing https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-cap

2024-07-18 11:04:00,182 - AInewsbot - INFO - get_url(Apple Denies Using YouTube Videos To Train Its AI Features After Report Claims Tim Cook's Company Used Creators' Content Without Consent) - Saving Apple_Denies_Using_YouTube_Videos_To_Train_Its_AI_Features_After_Report_Claims_Tim_Cook_s_Company_Used_Creators__Content_Without_Consent_20240718_110400.html as utf-8
2024-07-18 11:04:00,187 - AInewsbot - INFO - Processing https://towardsdatascience.com/how-to-plan-for-your-next-career-move-in-data-science-and-machine-learning-36c9d6f8cce6
2024-07-18 11:04:00,187 - AInewsbot - INFO - get_url(https://towardsdatascience.com/how-to-plan-for-your-next-career-move-in-data-science-and-machine-learning-36c9d6f8cce6) - starting get_url https://towardsdatascience.com/how-to-plan-for-your-next-career-move-in-data-science-and-machine-learning-36c9d6f8cce6
2024-07-18 11:04:05,829 - AInewsbot - INFO - get_url(Microsoft has let its AI-powered Designer app out of preview mode) - Saving Microsoft_has_let_

2024-07-18 11:04:47,136 - AInewsbot - INFO - get_url(https://www.bloomberg.com/news/articles/2024-07-18/nvidia-backs-networking-software-startup-arrcus-s-latest-round) - starting get_url https://www.bloomberg.com/news/articles/2024-07-18/nvidia-backs-networking-software-startup-arrcus-s-latest-round
2024-07-18 11:04:49,073 - AInewsbot - INFO - get_url(HP's upcoming laptop to feature Ryzen AI 300 processor with 55 TOPS NPU, surpassing HX 370 SKU in speed) - Saving HP_s_upcoming_laptop_to_feature_Ryzen_AI_300_processor_with_55_TOPS_NPU__surpassing_HX_370_SKU_in_speed_20240718_110449.html as utf-8
2024-07-18 11:04:49,080 - AInewsbot - INFO - get_url(How ToxMods AI impacted toxicity in Call of Duty voice chat | case study) - Saving How_ToxMods_AI_impacted_toxicity_in_Call_of_Duty_voice_chat___case_study_20240718_110449.html as utf-8
2024-07-18 11:04:49,094 - AInewsbot - INFO - Processing https://www.reddit.com/r/technews/comments/1e67t8h/after_breach_senators_ask_why_att_stores_call/
2024-

2024-07-18 11:05:50,555 - AInewsbot - INFO - get_url(Research Confirms Google AIO Keyword Trends) - Saving Research_Confirms_Google_AIO_Keyword_Trends_20240718_110550.html as utf-8
2024-07-18 11:05:50,579 - AInewsbot - INFO - Processing https://www.reddit.com/r/technology/comments/1e5plln/genai_sinks_into_the_trough_of_disillusionment/
2024-07-18 11:05:50,594 - AInewsbot - INFO - get_url(https://www.reddit.com/r/technology/comments/1e5plln/genai_sinks_into_the_trough_of_disillusionment/) - starting get_url https://www.reddit.com/r/technology/comments/1e5plln/genai_sinks_into_the_trough_of_disillusionment/
2024-07-18 11:05:51,186 - AInewsbot - INFO - get_url(I made a personalized workout plan with Claude AI  heres how to do it) - Saving I_made_a_personalized_workout_plan_with_Claude_AI__heres_how_to_do_it_20240718_110551.html as utf-8
2024-07-18 11:05:51,266 - AInewsbot - INFO - Processing https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-refe

In [88]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'actual_url', 'title', 'path']
pages_df

,id,actual_url,title,path
0,6,https://www.popsci.com/technology/drones-ants-...,Bug brains could help drone swarms find their ...,htmldata/Bug_brains_could_help_drone_swarms_fi...
1,10,https://www.calcalistech.com/ctechnews/article...,Hybrid cloud data platform CTERA bags $80 mill...,htmldata/Hybrid_cloud_data_platform_CTERA_bags...
2,15,https://www.geeky-gadgets.com/ai-data-analysis...,How to use Claude 3.5 Sonnet AI for Data Analysis,htmldata/How_to_use_Claude_3_5_Sonnet_AI_for_D...
3,18,https://cointelegraph.com/news/anthropic-partn...,Anthropic partners with Menlo for $100M fund i...,htmldata/Anthropic_partners_with_Menlo_for__10...
4,23,https://www.theregister.com/2024/07/18/samsung...,Samsung buys UK AI startup to give its product...,htmldata/Samsung_buys_UK_AI_startup_to_give_it...
...,...,...,...,...
193,178,https://videocardz.com/newz/hps-upcoming-lapto...,HP's upcoming laptop to feature Ryzen AI 300 p...,htmldata/HP_s_upcoming_laptop_to_feature_Ryzen...
194,183,https://www.reddit.com/r/technews/comments/1e6...,"After breach, senators ask why AT&T stores cal...",htmldata/After_breach__senators_ask_why_AT_T_s...
195,187,https://www.digitaltrends.com/mobile/google-ge...,Google Gemini is now a lot more helpful on And...,htmldata/Google_Gemini_is_now_a_lot_more_helpf...
196,191,https://www.searchenginejournal.com/research-c...,Research Confirms Google AIO Keyword Trends,htmldata/Research_Confirms_Google_AIO_Keyword_...


In [89]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [90]:
AIdf

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science,"Innovation, Machine Learning, Meta, Science",'ChatGPT moment for biology': Ex-Meta scientis...,66,0,AI in Healthcare & Precision Medicine,htmldata/_ChatGPT_moment_for_biology___Ex-Meta...
1,1,Feedly AI,AI & Drug Discovery With Harren Jhoti: Vanguar...,https://www.bloomberg.com/news/audio/2024-07-1...,True,2024-07-18,https://www.bloomberg.com/news/audio/2024-07-1...,www.bloomberg.com,Bloomberg,"Healthcare, Machine Learning, Science",AI & Drug Discovery With Harren Jhoti: Vanguar...,94,0,AI in Healthcare & Precision Medicine,htmldata/AI___Drug_Discovery_With_Harren_Jhoti...
2,2,Google News,How Machine Learning Is Propelling Structural ...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-18,https://hms.harvard.edu/news/how-machine-learn...,hms.harvard.edu,Harvard Medical School,"Data, Machine Learning, Science",How Machine Learning Is Propelling Structural ...,156,0,AI in Healthcare & Precision Medicine,htmldata/How_Machine_Learning_Is_Propelling_St...
3,3,Google News,nference Establishes AI Initiative with Takeda...,https://news.google.com/articles/CBMiowFodHRwc...,True,2024-07-18,https://www.businesswire.com/news/home/2024071...,www.businesswire.com,Business Wire,"Deals, Health, Fitness, Healthcare, Machine Le...",nference Establishes AI Initiative with Takeda...,168,0,AI in Healthcare & Precision Medicine,htmldata/nference_Establishes_AI_Initiative_wi...
4,4,Google News,56-gram drone gets ant-inspired AI eyes to nav...,https://news.google.com/articles/CBMiTmh0dHBzO...,True,2024-07-18,https://interestingengineering.com/innovation/...,interestingengineering.com,Interesting Engineering,"Autonomous Vehicles, Baidu, Machine Learning, ...",56-gram drone gets ant-inspired AI eyes to nav...,22,1,Autonomous Drones,htmldata/56-gram_drone_gets_ant-inspired_AI_ey...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,193,Google News,Ruled by robots: Research reveals people prefe...,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.newswise.com/articles/ruled-by-rob...,www.newswise.com,Newswise,"AI Doom, Artificial General Intelligence, Gen ...",Ruled by robots: Research reveals people prefe...,190,999,,htmldata/Ruled_by_robots__Research_reveals_peo...
194,194,Google News,Maths test stumps AI models: which number is b...,https://news.google.com/articles/CBMicGh0dHBzO...,True,2024-07-18,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"AI Models, Cognitive Science, Education, Gen A...",Maths test stumps AI models: which number is b...,192,999,,htmldata/Maths_test_stumps_AI_models__which_nu...
195,195,Reddit,GenAI sinks into the 'trough of disillusionmen...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-18,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit,"AI Doom, Bubble, Expectations, Gen AI, Languag...",GenAI sinks into the 'trough of disillusionmen...,194,999,,htmldata/GenAI_sinks_into_the__trough_of_disil...
196,196,HackerNoon,The Chosen One: Consistent Characters in Text-...,https://hackernoon.com/the-chosen-one-consiste...,True,2024-07-18,https://hackernoon.com/the-chosen-one-consiste...,hackernoon.com,Hacker Noon,Text-To-Image Diffusion Models,The Chosen One: Consistent Characters in Text-...,196,999,,htmldata/The_Chosen_One__Consistent_Characters...


# Summarize individual pages

In [91]:
print(SUMMARIZE_SYSTEM_PROMPT)


You are a summarization assistant.
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will focus on the top 3 points of the text and keep the response concise."
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will output Markdown format.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.



In [92]:
print(SUMMARIZE_USER_PROMPT)


Summarize the main points of the following text concisely in at most 3 bullet points:



In [93]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on 3.5 and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


2024-07-18 11:06:23,620 - AInewsbot - INFO - Starting summarize
2024-07-18 11:06:24,889 - AInewsbot - INFO - fetch_all_summaries - Page title: 'ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature' | Live Science
Social card title: 'ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature'
Social card description: The ESM3 model can 'write' new proteins from scratch, opening up new possibilities for synthetic biology.

2024-07-18 11:06:25,063 - AInewsbot - INFO - fetch_all_summaries - Page title: AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg
Social card title: AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg
Social card description: Understand every aspect of the global economy – and know how to make your next move.

2024-07-18 11:06:25,096 - AInewsbot - INFO - fetch_all_summaries - Page title: How Machine Learning Is Propell

2024-07-18 11:06:26,104 - AInewsbot - INFO - fetch_all_summaries - Page title: Anthropic, Menlo Ventures launch $100 million Anthology Fund for AI
Social card title: Anthropic launches $100 million AI fund with Menlo Ventures, ramping up competition with OpenAI
Social card description: Anthropic and one of its biggest investors are teaming up to back early-stage startups, and get them using the AI company's technology. 

2024-07-18 11:06:26,176 - AInewsbot - INFO - fetch_all_summaries - Page title: Anthropic follows OpenAI to launch $100 million AI startup fund
Social card title: Anthropic is following OpenAI to launch its own $100 million AI startup fund
Social card description: Anthropic partnered with Menlo Ventures to fund startups focused on AI with societal benefits

2024-07-18 11:06:26,220 - AInewsbot - INFO - fetch_all_summaries - Page title: Samsung's AI Makes Images That Are Uncanny Real
Social card title: Samsung's AI Makes Images That Are Uncanny Real
Social card descriptio

2024-07-18 11:06:29,177 - AInewsbot - INFO - fetch_all_summaries - Page title: When To Use Large Language Models—And When Not To
Social card title: Council Post: When To Use Large Language Models—And When Not To
Social card description: Not every problem will be solved by LLMs. Courage. Vision. Wisdom. Empathy. The world needs these human traits—and a great copilot to steer us toward a promising future.

2024-07-18 11:06:29,355 - AInewsbot - INFO - fetch_all_summaries - Page title: Large language models are rubbish at elementary level math · TechNode
Social card title: Large language models are rubbish at elementary level math · TechNode
Social card description: “9.11 and 9.9, which one is bigger?” Questions as simple as this confuse large language models including OpenAI’s GPT-4o, Moonshot-created Kimi, and

2024-07-18 11:06:29,443 - AInewsbot - INFO - fetch_all_summaries - Page title: Overcoming the limits of current LLM

2024-07-18 11:06:29,531 - AInewsbot - INFO - fetch_all_summari

2024-07-18 11:06:32,401 - AInewsbot - INFO - fetch_all_summaries - Page title: TSMC Q2 2024 earnings
Social card title: TSMC second-quarter profit beats expectations as AI chip boom continues
Social card description: TSMC on Thursday beat revenue and profit expectations in the second quarter, as demand for advanced chips used in AI applications continue to surge.

2024-07-18 11:06:32,543 - AInewsbot - INFO - fetch_all_summaries - Page title: TSMC posts Q2 earnings beat thanks to AI chip demand from Nvidia, Apple
Social card title: TSMC smashed earnings expectations thanks to AI chip demand
Social card description: The Taiwanese chipmaker predicted in April that its second-quarter sales could rise up to 30% on strong demand from Nvidia and Apple

2024-07-18 11:06:32,637 - AInewsbot - INFO - fetch_all_summaries - Page title: TSMC Q2 2024 earnings
Social card title: TSMC second-quarter profit beats expectations as AI chip boom continues
Social card description: TSMC on Thursday beat reven

2024-07-18 11:06:35,510 - AInewsbot - INFO - fetch_all_summaries - Page title: Self-Regulation Won’t Prevent Problematic Political Uses of Generative AI : technology
Social card title: Self-Regulation Won’t Prevent Problematic Political Uses of Generative AI
Social card description: Posted in r/technology by u/AtroScolo • 88 points and 20 comments

2024-07-18 11:06:35,555 - AInewsbot - INFO - fetch_all_summaries - Page title: SAP AI Core Vulnerabilities Expose Customer Data to Cyber Attacks
Social card title: SAP AI Core Vulnerabilities Expose Customer Data to Cyber Attacks
Social card description: Discover how SAP AI Core vulnerabilities could expose customer data and lead to supply chain attacks. Learn about the 'SAPwned' flaws and their impact

2024-07-18 11:06:35,582 - AInewsbot - INFO - fetch_all_summaries - Page title: Menlo Ventures and Anthropic team up to drop $100M on AI startups.
Social card title: Menlo Ventures and Anthropic team up to drop $100M on AI startups.
Social car

...

2024-07-18 11:06:39,369 - AInewsbot - INFO - fetch_all_summaries - Page title: Watch Steve Jobs describe the future & AI a year before the Mac
Social card title: Watch Steve Jobs describe the future and AI a year before the Mac
Social card description: The Steve Jobs Archive has released a fascinating video of a 28-year-old Jobs explaining computers to a skeptical crowd in 1983.

2024-07-18 11:06:39,437 - AInewsbot - INFO - fetch_all_summaries - Page title: What Elon Musk’s endorsement of Donald Trump means for X in 2024 - The Washington Post
Social card title: Analysis | What Musk’s Trump support means for X
Social card description: Conservatives long accused Twitter of liberal bias. What happens now that its owner openly backs Trump?

2024-07-18 11:06:39,610 - AInewsbot - INFO - fetch_all_summaries - Page title: United States and Singapore Sign SOI to Strengthen Data, Analytics and Artificial Intelligence Cooperation > U.S. Department of Defense > Release
Social card title: United St

2024-07-18 11:06:42,596 - AInewsbot - INFO - fetch_all_summaries - Page title: Orgs Are Finally Making Moves to Mitigate GenAI Risks
Social card title: Orgs Are Finally Making Moves to Mitigate GenAI Risks
Social card description: With AI use ramping up rapidly, a growing number of enterprise security teams have begun putting controls in place to protect sensitive data from accidental exposure and leaks.

2024-07-18 11:06:42,673 - AInewsbot - INFO - fetch_all_summaries - Page title: Gigabyte releases AI software to help train your own AI — AI TOP utility taps Gigabyte motherboards, GPUs, SSDs, and power supplies to fine-tune local AI model training | Tom's Hardware
Social card title: Gigabyte releases AI software to help train your own AI — AI TOP utility taps Gigabyte motherboards, GPUs, SSDs, and power supplies to fine-tune local AI model training
Social card description: The tool can locally train AI models with up to 236B parameters.

2024-07-18 11:06:42,734 - AInewsbot - INFO - fe

2024-07-18 11:06:44,196 - AInewsbot - INFO - fetch_all_summaries - Page title: Grayscale Launches New Artificial Intelligence (AI) Fund Featuring 5 Key Protocols | Bitcoinist.com
Social card title: Grayscale Launches New Artificial Intelligence (AI) Fund Featuring 5 Key Protocols | Bitcoinist.com
Social card description: Grayscale has launched its Decentralized AI Fund, to provide investors exposure to protocols within the AI sector of the crypto ecosystem.

2024-07-18 11:06:44,249 - AInewsbot - INFO - fetch_all_summaries - Page title: HP's upcoming laptop to feature Ryzen AI 300 processor with 55 TOPS NPU, surpassing HX 370 SKU in speed - VideoCardz.com
Social card title: HP's upcoming laptop to feature Ryzen AI 300 processor with 55 TOPS NPU, surpassing HX 370 SKU in speed - VideoCardz.com
Social card description: HP reveals OmniBook Ultra with the fastest NPU on the market, another variant of AMD Ryzen AI 300 incoming? HP has issued a press release claiming to have the fastest Neura

2024-07-18 11:06:49,267 - AInewsbot - INFO - fetch_all_summaries - Page title: The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: References | HackerNoon
Social card title: The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: References | HackerNoon
Social card description: In this study, researchers introduce an iterative procedure that, at each stage, identifies a coherent set of images sharing a similar identity.

2024-07-18 11:06:49,445 - AInewsbot - INFO - fetch_all_summaries - Page title: ElevenLabs, whose tech was used for the fake Biden robocalls, partners with AI detection company | TechSpot
Social card title: ElevenLabs, whose tech was used for the fake Biden robocalls, partners with AI detection company
Social card description: In January, a 39-second robocall went out to voters in New Hampshire telling them not to vote in the Democratic primary election, but to "save their votes"...

2024-07-18 11:06:55,479 - AInewsbot - INFO - Rece

(0, {'id': 'chatcmpl-9mN0T4hsQ06wKI54OyAeuZG4EKxWT', 'object': 'chat.completion', 'created': 1721315209, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '- Ex-Meta scientists developed the ESM3 model, an AI model that can create proteins not found in nature, opening up new possibilities for synthetic biology.\n- The ESM3 model is similar to OpenAI\'s GPT-4 and is trained on 2.78 billion proteins, enabling it to predict missing information about protein sequences, structures, and functions.\n- ESM3 can generate new proteins with specific functions, such as creating a novel green fluorescent protein called "esmGPF," which would take 500 million years of evolution to achieve naturally.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1228, 'completion_tokens': 114, 'total_tokens': 1342}, 'system_fingerprint': None})


In [94]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

198

In [95]:
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))


In [96]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science,"Innovation, Machine Learning, Meta, Science",'ChatGPT moment for biology': Ex-Meta scientis...,66,0,AI in Healthcare & Precision Medicine,htmldata/_ChatGPT_moment_for_biology___Ex-Meta...
1,1,Feedly AI,AI & Drug Discovery With Harren Jhoti: Vanguar...,https://www.bloomberg.com/news/audio/2024-07-1...,True,2024-07-18,https://www.bloomberg.com/news/audio/2024-07-1...,www.bloomberg.com,Bloomberg,"Healthcare, Machine Learning, Science",AI & Drug Discovery With Harren Jhoti: Vanguar...,94,0,AI in Healthcare & Precision Medicine,htmldata/AI___Drug_Discovery_With_Harren_Jhoti...
2,2,Google News,How Machine Learning Is Propelling Structural ...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-18,https://hms.harvard.edu/news/how-machine-learn...,hms.harvard.edu,Harvard Medical School,"Data, Machine Learning, Science",How Machine Learning Is Propelling Structural ...,156,0,AI in Healthcare & Precision Medicine,htmldata/How_Machine_Learning_Is_Propelling_St...
3,3,Google News,nference Establishes AI Initiative with Takeda...,https://news.google.com/articles/CBMiowFodHRwc...,True,2024-07-18,https://www.businesswire.com/news/home/2024071...,www.businesswire.com,Business Wire,"Deals, Health, Fitness, Healthcare, Machine Le...",nference Establishes AI Initiative with Takeda...,168,0,AI in Healthcare & Precision Medicine,htmldata/nference_Establishes_AI_Initiative_wi...
4,4,Google News,56-gram drone gets ant-inspired AI eyes to nav...,https://news.google.com/articles/CBMiTmh0dHBzO...,True,2024-07-18,https://interestingengineering.com/innovation/...,interestingengineering.com,Interesting Engineering,"Autonomous Vehicles, Baidu, Machine Learning, ...",56-gram drone gets ant-inspired AI eyes to nav...,22,1,Autonomous Drones,htmldata/56-gram_drone_gets_ant-inspired_AI_ey...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,193,Google News,Ruled by robots: Research reveals people prefe...,https://news.google.com/articles/CBMiZWh0dHBzO...,True,2024-07-18,https://www.newswise.com/articles/ruled-by-rob...,www.newswise.com,Newswise,"AI Doom, Artificial General Intelligence, Gen ...",Ruled by robots: Research reveals people prefe...,190,999,,htmldata/Ruled_by_robots__Research_reveals_peo...
194,194,Google News,Maths test stumps AI models: which number is b...,https://news.google.com/articles/CBMicGh0dHBzO...,True,2024-07-18,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"AI Models, Cognitive Science, Education, Gen A...",Maths test stumps AI models: which number is b...,192,999,,htmldata/Maths_test_stumps_AI_models__which_nu...
195,195,Reddit,GenAI sinks into the 'trough of disillusionmen...,https://www.reddit.com/r/technology/comments/1...,True,2024-07-18,https://www.reddit.com/r/technology/comments/1...,www.reddit.com,Reddit,"AI Doom, Bubble, Expectations, Gen AI, Languag...",GenAI sinks into the 'trough of disillusionmen...,194,999,,htmldata/GenAI_sinks_into_the__trough_of_disil...
196,196,HackerNoon,The Chosen One: Consistent Characters in Text-...,https://hackernoon.com/the-chosen-one-consiste...,True,2024-07-18,https://hackernoon.com/the-chosen-one-consiste...,hackernoon.com,Hacker Noon,Text-To-Image Diffusion Models,The Chosen One: Consistent Characters in Text-...,196,999,,htmldata/The_Chosen_One__Consistent_Characters...


In [97]:
# make text for email and also collect data for vector store
markdown_str = ''
vectorstore_list = []
metadata_list=[]
for i, row in enumerate(AIdf.itertuples()):
    topics = []
    if row.cluster_name:
        topics.append(row.cluster_name)
    if row.topic_str:
        topics.append(row.topic_str)
    topic_str = ", ".join(topics)

    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
    # simpler version for vector store
    vectorstore_list.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
    metadata_list.append({'id': row.id, 'title': row.title, 'url': row.actual_url, 'site': row.site_name})
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. 'ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature' - Live Science](https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature)  

 AI in Healthcare & Precision Medicine, Innovation, Machine Learning, Meta, Science  

- Ex-Meta scientists developed the ESM3 model, an AI model that can create proteins not found in nature, opening up new possibilities for synthetic biology.
- The ESM3 model is similar to OpenAI's GPT-4 and is trained on 2.78 billion proteins, enabling it to predict missing information about protein sequences, structures, and functions.
- ESM3 can generate new proteins with specific functions, such as creating a novel green fluorescent protein called "esmGPF," which would take 500 million years of evolution to achieve naturally. 



[2. AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)  

 AI in Healthcare & Precision Medicine, Healthcare, Machine Learning, Science  

- Artificial Intelligence is revolutionizing drug development, particularly in the early stages of drug discovery.
- Discussion between Sam Fazeli and Harren Jhoti explores the role of machine learning, AI, and AlphaFold 3 in drug discovery.
- Astex, co-founded by Harren Jhoti in 1999, has made significant contributions to oncology drugs through fragment-based drug discovery, leading to the development of drugs like Kisqali for breast cancer. 



[3. How Machine Learning Is Propelling Structural Biology - Harvard Medical School](https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology)  

 AI in Healthcare & Precision Medicine, Data, Machine Learning, Science  

- Structural biologist Lucas Farnung studies how molecular machines regulate gene expression during transcription.
- Farnung uses visualization techniques such as specific microscopes to study molecular machines involved in transcription.
- Artificial intelligence, specifically machine learning, is revolutionizing structural biology by predicting protein interactions, accelerating research, and enabling nonbiased testing of protein interactions. 



[4. nference Establishes AI Initiative with Takeda to Advance Precision Medicine in Inflammatory Bowel Disease - Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)  

 AI in Healthcare & Precision Medicine, Deals, Health, Fitness, Healthcare, Machine Learning, Science  

- nference and Takeda have established a research alliance to leverage AI for optimizing patient care in inflammatory bowel disease (IBD).
- The collaboration, named Project SUCCINCT, will use nference's AI platform to analyze health records and personalize treatment options for IBD patients.
- Takeda and nference aim to advance precision medicine by utilizing AI to improve diagnostic and treatment journeys for patients with IBD. 



[5. 56-gram drone gets ant-inspired AI eyes to navigate autonomously - Interesting Engineering](https://interestingengineering.com/innovation/insect-inspired-navigation-robot)  

 Autonomous Drones, Autonomous Vehicles, Baidu, Machine Learning, Meta, Navigation, Products, Robots, Science, Testing, Transportation  

- Researchers have developed an ant-inspired navigation system for a 56-gram drone that combines visual homing with odometry for autonomous navigation, aiming at applications like warehouse stock monitoring and gas leak detection.
- Tiny drones weighing from ten to a few hundred grams have potential for real-world applications due to their lightweight design for safe navigation, especially in narrow areas, and their potential for use in scenarios like pest detection in greenhouses.
- The bio-inspired navigation approach for small drones involves capturing snapshots of surroundings, comparing visuals, and correcting odometry drift, making it memory-efficient and suitable for tasks like returning to a starting point in various indoor conditions. 



...

[194. Ruled by robots: Research reveals people prefer AI | Newswise - Newswise](https://www.newswise.com/articles/ruled-by-robots-research-reveals-people-prefer-ai-to-make-decisions)  

 AI Doom, Artificial General Intelligence, Gen AI, Machine Learning, Research, Robots, Science  

- A study revealed that people prefer AI over humans for redistributive decisions.
- Participants favored algorithms over humans regardless of potential discrimination concerns.
- While there was a preference for AI, participants were less satisfied with AI decisions compared to human decisions in terms of fairness. 



[195. Maths test stumps AI models: which number is bigger, 9.90 or 9.11? - South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3270851/maths-test-stumps-ai-models-which-number-bigger-990-or-911)  

 AI Models, Cognitive Science, Education, Gen AI, Machine Learning, OpenAI, Perplexity, Robots, Testing  

- Large language models struggle with basic maths knowledge.
- AI models can only predict answers based on training data.
- AI technology's evolution is highlighted by the math issue faced by models. 



[196. GenAI sinks into the 'trough of disillusionment'. GenAI faces growing skepticism as it struggles to deliver on high expectations. - Reddit](https://www.reddit.com/r/technology/comments/1e5plln/genai_sinks_into_the_trough_of_disillusionment/)  

 AI Doom, Bubble, Expectations, Gen AI, Language Models, Machine Learning, Review  

- GenAI is facing growing skepticism and struggling to meet high expectations.
- The subreddit r/technology is a platform for sharing and discussing technological developments.
- Submissions on r/technology must adhere to specific guidelines related to content and behavior. 



[197. The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: References - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-references)  

 Text-To-Image Diffusion Models  

- Study introduces an iterative procedure to identify images sharing similar identities
- Authors include researchers from Google, Hebrew University of Jerusalem, Tel Aviv University, and Reichman University
- Several references to related works in text-to-image diffusion models and personalization techniques 



[198. ElevenLabs, whose tech was used for the fake Biden robocalls, partners with AI-detection company - TechSpot](https://www.techspot.com/news/103867-elevenlabs-whose-tech-used-fake-biden-robocalls-partners.html)  

 Biden, Cybersecurity, Deals, Disinformation, Machine Learning, Politics, Scams  

- ElevenLabs, known for its AI tech used in fake Biden robocalls, partners with Reality Defender, an AI detection company, for deepfake combat during the election year.
- The partnership brings mutual benefits as Reality Defender gains access to ElevenLabs' voice cloning data, enhancing detection capabilities, while ElevenLabs can utilize Reality Defender's tools to prevent misuse of its products.
- ElevenLabs' technology allows bridging language gaps, restoring lost voices, and creating more human digital interactions, emphasizing the importance of ethical use in the face of increasingly sophisticated deepfake threats. 



In [98]:
display(Markdown( vectorstore_list[16]))

[Anthropics Claude 3.5 Sonnet ranks number 1 for business and finance in S&P AI Benchmarks by Kensho - Amazon Web Services](https://aws.amazon.com/blogs/machine-learning/anthropic-claude-3-5-sonnet-ranks-number-1-for-business-and-finance-in-sp-ai-benchmarks-by-kensho/)

Topics: Anthropic, Claude, Finance, Fintech, Machine Learning, Products, Stocks 

- Anthropic’s Claude 3.5 Sonnet is ranked number 1 for business and finance in S&P AI Benchmarks by Kensho, showcasing its capabilities in challenging financial and business tasks.
- S&P AI Benchmarks by Kensho focus on evaluating language models for finance and business based on domain knowledge, quantity extraction, and quantitative reasoning, offering a leaderboard for model performance comparison.
- Anthropic’s Claude 3.5 Sonnet excels in tasks such as quantitative reasoning, quantity extraction, and domain knowledge, demonstrating its strength in generating insights and calculations in the financial domain.



In [99]:
print(metadata_list[16])


{'id': 16, 'title': 'Anthropics Claude 3.5 Sonnet ranks number 1 for business and finance in S&P AI Benchmarks by Kensho', 'url': 'https://aws.amazon.com/blogs/machine-learning/anthropic-claude-3-5-sonnet-ranks-number-1-for-business-and-finance-in-sp-ai-benchmarks-by-kensho/', 'site': 'Amazon Web Services'}


In [100]:
# Create Document objects with the paragraphs and corresponding metadata
docs = [Document(page_content=paragraph, metadata=meta) 
        for paragraph, meta in zip(vectorstore_list, metadata_list)]
len(docs)

198

In [101]:
print(docs[16])


page_content='[Anthropics Claude 3.5 Sonnet ranks number 1 for business and finance in S&P AI Benchmarks by Kensho - Amazon Web Services](https://aws.amazon.com/blogs/machine-learning/anthropic-claude-3-5-sonnet-ranks-number-1-for-business-and-finance-in-sp-ai-benchmarks-by-kensho/)\n\nTopics: Anthropic, Claude, Finance, Fintech, Machine Learning, Products, Stocks \n\n- Anthropic’s Claude 3.5 Sonnet is ranked number 1 for business and finance in S&P AI Benchmarks by Kensho, showcasing its capabilities in challenging financial and business tasks.\n- S&P AI Benchmarks by Kensho focus on evaluating language models for finance and business based on domain knowledge, quantity extraction, and quantitative reasoning, offering a leaderboard for model performance comparison.\n- Anthropic’s Claude 3.5 Sonnet excels in tasks such as quantitative reasoning, quantity extraction, and domain knowledge, demonstrating its strength in generating insights and calculations in the financial domain.\n\n' me

In [102]:
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_openai"
try:
    del vectorstore
except Exception as e:
    log(f"{e}")

try:
    shutil.rmtree(persist_directory)
    log(f"Directory '{persist_directory}' and all its contents have been removed successfully.")
except Exception as e:
    log(f"Remove directory error: {e}")
        
embeddings_openAI = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma.from_documents(docs, embeddings_openAI)


2024-07-18 11:06:55,825 - AInewsbot - INFO - name 'vectorstore' is not defined
2024-07-18 11:06:55,826 - AInewsbot - INFO - Remove directory error: name 'persist_directory' is not defined
2024-07-18 11:06:56,475 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-18 11:06:57,880 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [103]:
# Perform a similarity search
query = "What is the latest with openai?"
results = vectorstore.similarity_search_with_score(query, 
                                        k=20,
                                       )  # k is the number of results to return
# Print the results
urldict = {}
for doc, score in results:
    if urldict.get(doc.metadata['url']):
        continue
    urldict[doc.metadata['url']] = 1
    if score < 1.25:
        print(f"Score:   {score}")
        print(f"Content: {doc.page_content}\n")
        print(f"Metadata: {doc.metadata}\n")
        print("---")

2024-07-18 11:06:58,955 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Score:   0.9842485785484314
Content: [OpenAI used a game to help AI models explain themselves better - VentureBeat](https://venturebeat.com/ai/openai-used-a-game-to-help-ai-models-explain-themselves-better/)

Topics: AI Models, Artificial General Intelligence, Big Tech, Gen AI, Language Models, Machine Learning, OpenAI, Reinforcement Learning, Science 

- OpenAI has developed an algorithm to improve the legibility and explainability of large language models (LLMs) like GPT-4, aiming to foster trust in AI systems, especially in critical fields.
- The algorithm is based on a Prover-Verifier Game where a powerful "prover" AI model tries to persuade a less powerful "verifier" model in providing correct answers, enhancing the models' ability to explain their reasoning.
- By engaging the models in rounds of the game and training them to be more transparent and accurate in their responses, OpenAI's method aims to enhance the correctness and understandability of AI outputs, crucial for trust a

In [104]:
# # or use local embeddings with sentence_transformers
# # Initialize your embedding model
# embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the vector store with a persist_directory
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_huggingface"
# vectorstore_hf = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings_hf,
#     persist_directory=persist_directory
# )

# # Perform a similarity search
# query = "What is the latest with OpenAI?"
# results = vectorstore_hf.similarity_search(query, k=10)  # k is the number of results to return

# # Print the results
# for doc in results:
#     print(f"Content: {doc.page_content}\n")
#     print(f"Metadata: {doc.metadata}\n")
#     print("---")
    


In [105]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [106]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [107]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-18 11:06:59,113 - AInewsbot - INFO - Sending bullet points email


# Ask ChatGPT for top categories

In [108]:
print(TOP_CATEGORIES_PROMPT)

You will act as a research assistant identifying the top stories and topics
of today's news. I will provide a list of today's news stories about AI and summary bullet points in markdown
format. You are tasked with identifying the top 10-20 stories and topics of today's news. For each top story
or topic, you will create a short title and respond with a list of titles formatted as a JSON object.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Categories of

In [109]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


2024-07-18 11:07:49,365 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [110]:
suggested_categories = list(json.loads(response.choices[0].message.content).values())[0]
suggested_categories

['Meta AI model for protein creation',
 'AI in drug discovery',
 'Machine learning in structural biology',
 'nference and Takeda AI partnership',
 'Ant-inspired AI for drones',
 "Fei-Fei Li's $1B AI startup",
 "Artificial Agency's $16M funding",
 "CTERA's $80M funding",
 "Anthropic's $100M AI fund",
 "Samsung's AI image generation",
 'EU antitrust probe on AI deals',
 "Ukraine's AI-enabled war drones",
 'Large language models limitations',
 'Salesforce AI service agent',
 "Meta's AI regulatory issues",
 'Nvidia and Mistral AI model',
 'TSMC AI chip demand surge',
 'AI search engines',
 'AI in healthcare strategy',
 'AI in cybersecurity']

In [111]:
cluster_topic_list

['AI in Healthcare & Precision Medicine',
 'Autonomous Drones',
 'AI Startup Funding',
 'Claude AI for Data Analysis',
 'AI Startup Funding',
 'AI in Big Tech Products',
 'AI-Powered Smartphones',
 'Big Tech Antitrust Investigations',
 'Ukrainian Military Robotics',
 'Large Language Models',
 'AI-Powered Customer Service Agents',
 'AI Ethics and Regulation',
 'AI Art by Microsoft',
 'AI Chip Demand Surge',
 'AI and Nvidia Partnerships',
 'AI Impact and Ethics',
 'Text-To-Image Diffusion Models',
 'ChatGPT in AI Dominance']

In [112]:
# human category edit 
my_cats = [
 'AI in drug discovery',
 'Machine learning in structural biology',
 'nference and Takeda AI partnership',
 'Ant-inspired AI for drones',
 "Fei-Fei Li's $1B AI startup",
 "Artificial Agency's $16M funding",
 "CTERA's $80M funding",
 "Anthropic's $100M AI fund",
 "Samsung's AI image generation",
 'EU antitrust probe on AI deals',
 'Salesforce AI service agent',
 "Meta's AI regulatory issues",
 'Nvidia and Mistral AI model',
 'TSMC AI chip demand surge',
 'AI search engines',
 'AI in cybersecurity',
 'AI in Healthcare & Precision Medicine',
 'Autonomous Drones',
 'AI Startup Funding',
 'Claude AI for Data Analysis',
 'AI Startup Funding',
 'AI Smartphones',
 'Big Tech Antitrust Investigations',
 'Large Language Models',
 'AI-Powered Customer Service Agents',
 'AI Ethics and Regulation',
 'AI Art by Microsoft',
 'AI Chip Demand Surge',
 'Text-To-Image Diffusion Models',
 'ChatGPT in AI Dominance'
]

In [113]:
md_str = ""
doc_list = []
docid_list = []
similarity_cutoff = 1.25
for cat in my_cats:
    docstr = f"# {cat} \n\n"
    # Perform a similarity search
    results = vectorstore.similarity_search_with_score(cat, 
                                                       k=10,
                                                      )
    if results:
        # Print the results
        urldict = {}
        for doc, score in results:
            if urldict.get(doc.metadata['url']):
                continue
            urldict[doc.metadata['url']] = 1    
            if score > similarity_cutoff:
                break
            docstr += f"{doc.page_content}\n"
            docid_list.append(doc.metadata['id'])
        doc_list.append(docstr)
        md_str += docstr
        
        
display(Markdown(md_str))
            


2024-07-18 11:11:24,823 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:25,050 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:25,271 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:25,465 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:25,660 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:25,839 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:26,018 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:26,231 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-18 11:11:26,416 - httpx - INFO - HTTP Request: POST https://api.openai.c

# AI in drug discovery 

[AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)

Topics: Healthcare, Machine Learning, Science 

- Artificial Intelligence is revolutionizing drug development, particularly in the early stages of drug discovery.
- Discussion between Sam Fazeli and Harren Jhoti explores the role of machine learning, AI, and AlphaFold 3 in drug discovery.
- Astex, co-founded by Harren Jhoti in 1999, has made significant contributions to oncology drugs through fragment-based drug discovery, leading to the development of drugs like Kisqali for breast cancer.


[ICL and Lavie Bio Leverage Artificial Intelligence to Make Significant Advancement in Development of Yield Increasing Bio-Stimulants - PR Newswire](https://www.prnewswire.com/news-releases/icl-and-lavie-bio-leverage-artificial-intelligence-to-make-significant-advancement-in-development-of-yield-increasing-bio-stimulants-302199316.html)

Topics: Agriculture, Deals, Gen AI, Machine Learning, Meta, Products, Science 

- ICL and Lavie Bio have used artificial intelligence to identify more than a dozen novel microbes for row crops facing extreme weather conditions, aiming to increase yields by 5% to 10%.
- The collaboration's AI-driven program has achieved a high success rate in identifying microbial candidates, paving the way for field trials in the U.S. and Brazil by the end of 2024.
- The use of AI and advanced biological solutions is crucial to addressing the economic impact of climate change on global row crops caused by extreme weather events like droughts, emphasizing the need for innovative solutions in agriculture.


[Playing the Long Game: How AI Can Optimize Enablement in Longtail Software Partnerships - Hacker Noon](https://hackernoon.com/playing-the-long-game-how-ai-can-optimize-enablement-in-longtail-software-partnerships)

Topics: Opinion 

- Larger partners receive personalized support, while smaller long-tail partners often face disparities in enablement processes, hindering growth for both.
- Generative AI can offer tier-one treatment affordably to long-tail partners by providing necessary support.
- Over 50% of SaaS licenses remain unused for more than 90 days, highlighting efficiency challenges that AI can help address.


[3 AI Stocks to Grab Now for 'Catch-Up' Gains - TradingView](https://www.tradingview.com/news/barchart:d5ab2bc66094b:0-3-ai-stocks-to-grab-now-for-catch-up-gains/)

Topics: Finance, Investment, Stocks 

- Analyst Ben Reitzes highlights three AI stocks for potential gains: Apple Inc. AAPL, Advanced Micro Devices, Inc. AMD, and Intel Corporation INTC.
- Apple's recent innovations in AI, strong financial performance, and shareholder rewards position it for growth.
- Advanced Micro Devices and Intel Corporation are introducing AI technologies to challenge Nvidia, with high growth projections and potential for stock price increases.


[AI can predict tipping points before they happen - The Economist](https://www.economist.com/science-and-technology/2024/07/17/ai-can-predict-tipping-points-before-they-happen)

Topics: AI Doom, Big Tech, Cognitive Science, Data, Gen AI, Machine Learning, Meta, OpenAI, Risk Management, Science, Singularity 

- AI has been developed to predict tipping points in complex systems before they occur, with potential applications in various fields such as economics and epidemiology.
- Computer scientists in China utilized machine-learning algorithms to accurately predict critical transitions, offering solutions for real-world issues like floods and power outages.
- The ability to foresee tipping points using AI could provide valuable time for prevention and mitigation strategies in advance.


[nference Establishes AI Initiative with Takeda to Advance Precision Medicine in Inflammatory Bowel Disease - Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)

Topics: Deals, Health, Fitness, Healthcare, Machine Learning, Science 

- nference and Takeda have established a research alliance to leverage AI for optimizing patient care in inflammatory bowel disease (IBD).
- The collaboration, named Project SUCCINCT, will use nference's AI platform to analyze health records and personalize treatment options for IBD patients.
- Takeda and nference aim to advance precision medicine by utilizing AI to improve diagnostic and treatment journeys for patients with IBD.


[AI & robotics briefing: What is AI, really? - Nature](https://www.nature.com/articles/d41586-024-02354-0)

Topics: Machine Learning, Meta, OpenAI, Robotics, Robots 

- AI algorithm reveals sperm whales' phonetic alphabet, enhancing understanding of their communication
- AI lie detector outperforms human accuracy, leading to concerns about reliance on technology
- AI ultrasound software in Uganda improves access to pregnancy scans, potentially reducing stillbirths and complications


[The AI doctor will see you now: ChatGPT dominates medical exam - CNBC](https://www.cnbc.com/video/2024/07/18/the-ai-doctor-will-see-you-now-chatgpt-dominates-medical-exam.html)

Topics: Chatgpt, Language Models, Machine Learning, OpenAI 

- Former FDA Commissioner Dr. Scott Gottlieb discusses AI large language models excelling in a medical exam for doctors
- ChatGPT stands out in the medical exam for a medical license
- AI-powered healthcare is the future of healthcare


[Why business leaders view AI as an opportunity to take toil out of our work - Fortune](https://fortune.com/2024/07/17/ai-business-leaders-workplace-workday-mckinsey-lenovo/)

Topics: Machine Learning, Opinion 

- AI tools can enhance productivity at work and allow employees to focus on more meaningful tasks, leading to increased joy and productivity for individuals.
- Business leaders at Fortune's Brainstorm Tech conference believe that AI can bring about unprecedented productivity gains for knowledge workers by automating tasks, such as summarizing meetings or updating code, and helping to eliminate biases in processes like hiring.
- While concerns exist about job displacement due to AI adoption, there is a significant opportunity for workplaces to optimize internal processes, implement controls, and leverage AI to increase productivity and efficiency.


[See the 9-slide pitch deck that startup Thoughtful AI used to raise $20 million from Drive Capital for its AI agents - Business Insider](https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-capital-series-a-2024-7)

Topics: Funding, Gen AI, Machine Learning, Products, Robots, Venture Capital 

- Thoughtful AI raised $20 million in Series A funding to launch AI agents for healthcare clinics and is focusing on automating revenue cycle management tasks.
- The startup identified healthcare as its key market after realizing the ROI was highest in that industry, leading to a strategic shift in its target clients.
- Thoughtful AI's AI agents can autonomously handle 100% of revenue cycle management workflows, helping healthcare clinics save costs on hiring and increasing efficiency.


# Machine learning in structural biology 

[How Machine Learning Is Propelling Structural Biology - Harvard Medical School](https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology)

Topics: Data, Machine Learning, Science 

- Structural biologist Lucas Farnung studies how molecular machines regulate gene expression during transcription.
- Farnung uses visualization techniques such as specific microscopes to study molecular machines involved in transcription.
- Artificial intelligence, specifically machine learning, is revolutionizing structural biology by predicting protein interactions, accelerating research, and enabling nonbiased testing of protein interactions.


['ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature' - Live Science](https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature)

Topics: Innovation, Machine Learning, Meta, Science 

- Ex-Meta scientists developed the ESM3 model, an AI model that can create proteins not found in nature, opening up new possibilities for synthetic biology.
- The ESM3 model is similar to OpenAI's GPT-4 and is trained on 2.78 billion proteins, enabling it to predict missing information about protein sequences, structures, and functions.
- ESM3 can generate new proteins with specific functions, such as creating a novel green fluorescent protein called "esmGPF," which would take 500 million years of evolution to achieve naturally.


# nference and Takeda AI partnership 

[nference Establishes AI Initiative with Takeda to Advance Precision Medicine in Inflammatory Bowel Disease - Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)

Topics: Deals, Health, Fitness, Healthcare, Machine Learning, Science 

- nference and Takeda have established a research alliance to leverage AI for optimizing patient care in inflammatory bowel disease (IBD).
- The collaboration, named Project SUCCINCT, will use nference's AI platform to analyze health records and personalize treatment options for IBD patients.
- Takeda and nference aim to advance precision medicine by utilizing AI to improve diagnostic and treatment journeys for patients with IBD.


[TSMC rides AI demand to raise revenue forecast, says no to US joint venture - Reuters](https://www.reuters.com/technology/tsmc-set-report-strong-profit-stock-pressured-by-trump-comments-2024-07-18/)

Topics: Deals, Economics, Manufacturing, Semiconductor Chips, Taiwan, Tsmc 

- TSMC raised its full-year revenue forecast due to high demand for AI chips, benefiting from the global AI boom amid pandemic-led electronics demand slowdown.
- The company plans to continue expanding overseas, including in the U.S., Japan, and Germany, rejecting the possibility of a joint venture in the U.S. despite potential political pressure.
- TSMC expects tight supply for leading-edge nodes like 3nm and 5nm, with strong smartphone and AI-related demand supporting future business growth.


[CTERA gets $80m war chest for AI and acquisitions - Blocks and Files](https://blocksandfiles.com/2024/07/17/ctera-gets-funding-for-ai-and-acquisitions/)

Topics: Ctera, Deals, Funding, Investment, Mergers And Acquisitions, Venture Capital 

- CTERA has raised $80 million in funding from PSG Equity for AI expansion and acquisitions in the hybrid cloud data market.
- The focus is on leveraging unstructured data for AI and hybrid cloud file storage growth, with the aim to capture market opportunities and enhance customer value.
- The funding is split between primary funding for business growth and secondary funding to buy out existing shareholders, with an emphasis on utilizing data intelligence services to address challenges in AI data utilization.


[Ctera Co-Founder On New $80M Growth Funding, Plans For AI Intelligence In File Services - CRN](https://www.crn.com/news/storage/2024/ctera-ceo-on-new-80m-growth-funding-plans-for-ai-intelligence-in-file-services)

Topics: Ctera, Data, Deals, Funding, Venture Capital 

- Ctera secured an $80 million growth investment round to expand global presence and enhance AI capabilities.
- The company is focusing on developing Ctera Data Intelligence to connect existing data platforms into AI systems.
- The competitive landscape in hybrid cloud data services is growing, reflected in recent investments in similar companies like Nasuni.


[TSMC Hikes Revenue Outlook to Reflect Heated AI Demand - Yahoo Finance](https://finance.yahoo.com/news/tsmc-profit-surpasses-estimates-ai-053450831.html)

Topics: Machine Learning, Manufacturing, Products, Semiconductor Chips, Taiwan, Tsmc 

- TSMC raised its revenue outlook for 2024 due to strong AI demand, with sales expected to grow more than previously guided.
- The company's confidence in the high AI spending is reflected in its revenue forecast and capital spending outlook, driven by global AI infrastructure investments.
- TSMC's shift in focus towards AI is paying off, with net income rising, high-performance computing contributing over half of revenue, and expectations to meet or exceed gross margin targets.


[AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)

Topics: Healthcare, Machine Learning, Science 

- Artificial Intelligence is revolutionizing drug development, particularly in the early stages of drug discovery.
- Discussion between Sam Fazeli and Harren Jhoti explores the role of machine learning, AI, and AlphaFold 3 in drug discovery.
- Astex, co-founded by Harren Jhoti in 1999, has made significant contributions to oncology drugs through fragment-based drug discovery, leading to the development of drugs like Kisqali for breast cancer.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


[US-Singapore boost artificial intelligence ties, after small nation made earlier agreement with China - Breaking Defense](http://breakingdefense.com/2024/07/us-singapore-boost-artificial-intelligence-ties-after-small-nation-made-earlier-agreement-with-china/)

Topics: China, Deals, Politics, Us-Singapore 

- Singapore signed an agreement with the United States to boost AI efforts in the military sector, aiming to enhance data sharing for better understanding of international shipping and maritime activities.
- Singapore has a strong AI sector, with significant government-supported R&D spending and a well-developed startup ecosystem.
- Prior to the agreement with the US, Singapore had also entered into an AI agreement with China focused on civilian AI development and data sharing, posing challenges in aligning cooperation between the US and China in the digital and cyber realms.


[EU probes Microsoft-OpenAI and Google-Samsung AI deals - Artificial Intelligence News](https://www.artificialintelligence-news.com/news/eu-probes-microsoft-openai-and-google-samsung-ai-deals/)

Topics: Big Tech, Deals, Eu, European Union, Google, Machine Learning, Mergers And Acquisitions, Microsoft, OpenAI, Policy And Regulation, Review, Samsung 

- The European Union has intensified antitrust scrutiny on AI deals involving tech giants like Microsoft-OpenAI and Google-Samsung, amid concerns of potential monopolistic power and anti-competitive practices.
- Microsoft's multibillion-dollar partnership with OpenAI and Google's collaboration with Samsung in AI technologies have drawn regulatory attention due to their implications on market dominance and competition.
- The EU Commission is investigating these partnerships to determine if exclusivity clauses could harm competition in the AI market, signaling a growing focus on fostering fair competition and ensuring smaller competitors have opportunities to compete and innovate.


[TSMC raises hopes of a prolonged AI boom - Financial Times](https://www.ft.com/content/24c8ec54-5a9d-4e0a-b7c7-6f01fa53d049)

Topics: Semiconductor Chips, Taiwan, Tsmc 

- TSMC raises hopes for a prolonged AI boom
- Complete digital access to quality FT journalism with expert analysis
- Includes global news, expert opinion, curated newsletters, exclusive analysis


# Ant-inspired AI for drones 

[56-gram drone gets ant-inspired AI eyes to navigate autonomously - Interesting Engineering](https://interestingengineering.com/innovation/insect-inspired-navigation-robot)

Topics: Autonomous Vehicles, Baidu, Machine Learning, Meta, Navigation, Products, Robots, Science, Testing, Transportation 

- Researchers have developed an ant-inspired navigation system for a 56-gram drone that combines visual homing with odometry for autonomous navigation, aiming at applications like warehouse stock monitoring and gas leak detection.
- Tiny drones weighing from ten to a few hundred grams have potential for real-world applications due to their lightweight design for safe navigation, especially in narrow areas, and their potential for use in scenarios like pest detection in greenhouses.
- The bio-inspired navigation approach for small drones involves capturing snapshots of surroundings, comparing visuals, and correcting odometry drift, making it memory-efficient and suitable for tasks like returning to a starting point in various indoor conditions.


[Bug brains could help drone swarms find their way home - Popular Science](https://www.popsci.com/technology/drones-ants-memory/)

Topics: Autonomous Vehicles, Cognitive Science, Navigation, Robots, Science 

- Researchers at Delft University of Technology explored using the navigation techniques of ants, specifically their "snapshot model," to help small drones navigate autonomously with minimal memory capacity.
- The insect-inspired drones were successfully able to navigate through an obstacle course using a small amount of memory and a basic microcontroller for visual computing.
- These bug-inspired drones have the potential to navigate autonomously in remote or GPS-inaccessible areas, offering applications in agriculture, stock tracking, search and rescue operations, and military uses, without the need for complex sensors or external infrastructure.


[Ukraine rushes to create AI-enabled war drones - Reuters](https://www.reuters.com/technology/artificial-intelligence/ukraine-rushes-create-ai-enabled-war-drones-2024-07-18/)

Topics: AI-Enabled War Drones, Gen AI, Machine Learning, Military, Robots, Science, Ukraine 

- Ukraine is rushing to develop AI-enabled drones to gain a technological edge in warfare, with startups focusing on visual systems, terrain mapping, and interconnected drone swarms.
- AI drone control systems are necessary to combat signal jamming from electronic warfare systems and increase hit rates, with drones being developed to lock onto targets through cameras and operate autonomously to bypass jamming effects.
- The development of cost-effective AI targeting systems for drones using tools like Raspberry Pi is crucial for mass deployment along the front line, where thousands of drones are used weekly, to counter the Russian threat and protect drone pilots.


[Ukraine Builds AI Drone Fleets to Combat Russian Forces - PCMag](https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces)

Topics: Autonomous Vehicles, Machine Learning, Military, Robots, Russia, Ukraine 

- Ukrainian tech startups like Vyriy and Swarmer are developing AI-powered drones to combat Russia's ongoing invasion, with Swarmer's drones capable of fleet deployment for strikes and surveillance missions without continuous human involvement.
- AI-powered drones are resistant to Russian signal jamming, providing a strategic advantage over human-operated drones on the battlefield and aiming to eventually operate with no human connection.
- The use of autonomous drones in the war raises ethical concerns but offers the potential to reduce the human cost of the conflict, with reports indicating AI drones have already been employed in targeting Russian energy sources and plans for autonomous weaponized helicopters are in progress.


[Ukraine Rushes to Create AI-Enabled War Drones - Newsmax](https://www.newsmax.com/world/globaltalk/ukraine-war-drones/2024/07/18/id/1173032/)

Topics: AI Doom, AI-Enabled War Drones, Gen AI, Machine Learning, Military, Science, Ukraine 

- Ukrainian startups are developing AI systems to operate drones for warfare, aiming to gain a technological edge in battle and counter Russian signal jamming. 
- AI-enabled drone development includes visual systems for target identification, terrain mapping for navigation, and complex programs for drones to operate in interconnected swarms. 
- Automation in drone operations using AI not only improves efficiency but also protects drone pilots who are prime targets for enemy fire, with the use of systems like Swarmer's Styx directing a network of drones autonomously.


[Researchers create insect-inspired autonomous navigation strategy for tiny, lightweight robots - TechXplore](https://techxplore.com/news/2024-07-insect-autonomous-strategy-tiny-lightweight.html)

Topics: Researchers, Robots, Science 

- TU Delft researchers have developed an insect-inspired autonomous navigation strategy for tiny robots based on how ants recognize their environment, enabling robots to return home after long trajectories with minimal resources.
- Tiny, lightweight robots have the potential for applications like monitoring stock in warehouses or detecting gas leaks, as they are safe, can navigate narrow areas, and be deployed in large numbers.
- The proposed navigation strategy allows small robots to cover distances up to 100 meters with minimal memory usage, enabling practical real-world applications without the need for external infrastructure like GPS or wireless beacons.


[AI & robotics briefing: What is AI, really? - Nature](https://www.nature.com/articles/d41586-024-02354-0)

Topics: Machine Learning, Meta, OpenAI, Robotics, Robots 

- AI algorithm reveals sperm whales' phonetic alphabet, enhancing understanding of their communication
- AI lie detector outperforms human accuracy, leading to concerns about reliance on technology
- AI ultrasound software in Uganda improves access to pregnancy scans, potentially reducing stillbirths and complications


[Tiny drone "CrazyFlie" weighs only 56 grams (IMAGE) - EurekAlert](https://www.eurekalert.org/multimedia/1034513)

Topics: Manufacturing, Open Source, Products, Robots, Science, Testing 

- A tiny drone named "CrazyFlie" weighs only 56 grams.
- Developed by Delft University of Technology.
- The drone is capable of navigating back to its starting location using an insect-inspired navigation strategy.


[From basement to battlefield: Ukrainian startups create low-cost robots to fight Russia - Associated Press News](https://apnews.com/article/ukraine-russia-drones-robots-artificial-intelligence-b07de714659ef859e16c51d7cda4256b)

Topics: Military, Products, Robots, Russia 

- Ukrainian startups are creating low-cost robots to aid in battlefield tasks like mine clearance, evacuations, and equipment carrying, leveraging innovation in secret workshops.
- Defense startups in Ukraine are producing economical unmanned vehicles, like the Odyssey, priced at $35,000, to counteract Russia's resources in the conflict.
- The government is supporting the development of drones and unmanned technologies, with concerns raised by experts regarding the proliferation and impact of low-cost drones and AI in warfare.


[Combining Web3, AI and robotics to defeat world soccer champions - Cointelegraph](https://cointelegraph.com/news/combining-web3-ai-and-robotics-to-defeat-world-soccer-champions)

Topics: Machine Learning, Robotics, Robots, Sports 

- Foxsy AI combines robotics and blockchain technology to create a scalable framework for autonomous systems, aiming to have humanoid robots defeat world soccer champions by 2040 through collaborative efforts.
- The project is based on the RoboCup mission for humanoid robots to win a soccer game against world champions by 2050, with Foxsy AI confident in achieving this by 2040. They leverage insights from human thinking to advance AI and robotics.
- Foxsy AI's native token, FOXSY, is launched on the MultiversX network, offering a community-focused approach with token incentives and user-friendly products to support creative strategies in AI soccer capabilities.


# Fei-Fei Li's $1B AI startup 

['Godmother of AI' Dr. Fei-Fei Li just built a $1 billion startup - Quartz](https://qz.com/godmother-ai-dr-fei-fei-li-built-1-billion-startup-1851597427)

Topics: Fei-Fei Li, Machine Learning, Startup, Venture Capital 

- Dr. Fei-Fei Li built a $1 billion startup called World Labs focusing on human-like visual data processing and advanced reasoning technology.
- She is renowned for her contributions to AI such as inventing ImageNet and consults with policymakers on setting up guardrails for technology.
- Dr. Fei-Fei Li is optimistic about AI's potential for creating positive impacts, contrary to prevailing pessimism in the field.


[Andreessen Horowitz-Backed 'Godmother Of AI' Fei-Fei Li Quietly Builds $1B Startup In 4 Months - Benzinga](https://www.benzinga.com/news/24/07/39833376/andreessen-horowitz-backed-godmother-of-ai-fei-fei-li-quietly-builds-1b-startup-in-4-months)

Topics: Deals, Fei-Fei Li, Funding, Investment, Machine Learning, Startup, Venture Capital 

- Fei-Fei Li, known as the "godmother of AI," has rapidly founded a billion-dollar startup named World Labs with significant funding from investors like Andreessen Horowitz and Radical Ventures.
- World Labs focuses on developing "spatial intelligence" in AI, aiming to process visual data like humans, amid increasing investor interest in AI technologies.
- The swift valuation of World Labs reflects the growing investment and competition in the AI sector, with notable funds being launched and major investments like SoftBank Group's $300 million funding in Skild AI.


[Godmother of A.I. Fei-Fei Li Has Created a $1B Startup in Less Than 4 Months - Observer](https://observer.com/2024/07/ai-godmother-fei-fei-li-1b-spatial-intelligence-startup/)

Topics: Fei-Fei Li, Funding, Startup, Venture Capital 

- Fei-Fei Li's startup, World Labs, reached a $1 billion valuation within four months of launching, with investments from firms like Andreessen Horowitz and Radical Ventures.
- The concept of spatial intelligence is being pursued by Li in her new venture, aiming to advance AI beyond current capabilities by linking perception with action in AI systems.
- The startup, World Labs, has successfully raised about $100 million in funding through two rounds since its launch in April.


[The godmother of AI has a new startup already worth $1 billion - The Verge](https://www.theverge.com/2024/7/17/24200496/ai-fei-fei-li-world-labs-andreessen-horowitz-radical-ventures)

Topics: Funding, OpenAI, Startup 

- Fei-Fei Li, known as the "godmother of AI," founded the startup World Labs, already valued at over $1 billion within four months.
- World Labs aims to develop AI capable of advanced reasoning through human-like processing of visual data, with a focus on understanding the three-dimensional physical world.
- The startup has received significant funding rounds, with backing from investors like Andreessen Horowitz and Radical Ventures, and could impact fields such as robotics, augmented reality, virtual reality, and computer vision.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


[Menlo Ventures and Anthropic team up to drop $100M on AI startups. - Ben's Bites](https://bensbites.beehiiv.com/p/menlo-ventures-anthropic-team-drop-100m-ai-startups)

Topics: Anthropic, Deals, Funding, Gen AI, Machine Learning, Menlo Ventures, Venture Capital 

- Menlo Ventures and Anthropic launched a $100M Anthology Fund to invest in AI startups, serving as matchmakers for the AI companies and VC money.
- The fund focuses on investing in AI startups from seed to expansion stage, with particular interest in AI infrastructure, cutting-edge AI apps, consumer AI, trust and safety tools, and AI for social good.
- Startups receiving investments not only get cash but also benefit from access to AI models, networking opportunities with AI experts, and cutting-edge technology, potentially birthing the next big innovation in AI.


[Anthropic and Menlo Ventures Partner on $100 Million AI Fund - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-17/anthropic-and-menlo-ventures-partner-on-100-million-ai-fund)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures have partnered on a $100 million AI fund targeting startups using Anthropic's AI models, with investments starting at $100,000. 
- The fund, managed by Menlo Ventures partner Matt Murphy, aims to help AI startups in various sectors like healthcare and cybersecurity, providing funding, advice, and networking opportunities.
- Anthropic will not take stakes in companies receiving investments but offers credits and access to AI models to portfolio startups, in addition to potentially considering them as acquisition targets in the future.


[Anthropic launches $100 million AI fund with Menlo Ventures, ramping up competition with OpenAI - CNBC](https://www.cnbc.com/2024/07/17/anthropic-menlo-ventures-launch-100-million-anthology-fund-for-ai.html)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures collaborate to establish the $100 million Anthology Fund to support early-stage startups and integrate the AI company's technology.
- The Anthology Fund is inspired by the successful iFund partnership between Apple and Kleiner Perkins, aiming to support developers and provide insights into technological needs and advancements.
- Venture capitalists are enhancing their support for AI startups amidst the rapid pace of AI development, with Anthropic offering resources like coaching, meetups, and direct communication to improve its products and assist startups in the ecosystem.


[Anthropic is following OpenAI to launch its own $100 million AI startup fund - Quartz](https://qz.com/anthropic-menlo-ventures-100-million-ai-startup-fund-o-1851596353)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, OpenAI, Sam Altman, Venture Capital 

- Anthropic partners with Menlo Ventures to launch a $100 million fund for AI startups with societal benefits.
- The fund aims to support startups working on AI safety tools and apps in fields like healthcare, education, and energy.
- Anthropic's co-founders are former OpenAI engineers who left the company over concerns about scaling up AI tech too quickly.


[Missed Out on Nvidia? Billionaires Are Buying These 2 Artificial Intelligence (AI) Stocks Hand Over Fist. - Yahoo Finance](https://finance.yahoo.com/news/missed-nvidia-billionaires-buying-2-081500476.html)

Topics: Finance, Nvidia, Stocks 

- Nvidia created significant value for investors with a ninefold increase in market capitalization over the last 18 months.
- Billionaire fund managers invested in Amazon and Oracle as potential AI opportunities post-Nvidia's success.
- Amazon and Oracle have showcased strong AI implementations within their businesses, indicating potential for positive returns for investors.


# Artificial Agency's $16M funding 

[Artificial Agency raises $16M to bring AI-powered behavior to games - VentureBeat](https://venturebeat.com/ai/artificial-agency-raises-16m-to-bring-ai-powered-behavior-to-games/)

Topics: Deals, Funding, Gaming, Gen AI, Machine Learning, Rag, Reinforcement Learning, Venture Capital 

- Artificial Agency, an AI startup, raised $16 million in funding for its flagship product, an AI-powered behavior engine that allows game developers to incorporate generative AI seamlessly into various game aspects, promising more personalized and engaging gaming experiences.
- The engine enables game developers to add improvisation to scripted interactions, create full artificial players, and control game elements to enhance player immersion and entertainment, with the aim of reducing churn and keeping players engaged.
- The company, based in Edmonton, is collaborating with AAA studios to develop the engine, which is expected to be widely available for studios in 2025, offering a customizable and extensible solution for game design and development.


[See the 9-slide pitch deck that startup Thoughtful AI used to raise $20 million from Drive Capital for its AI agents - Business Insider](https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-capital-series-a-2024-7)

Topics: Funding, Gen AI, Machine Learning, Products, Robots, Venture Capital 

- Thoughtful AI raised $20 million in Series A funding to launch AI agents for healthcare clinics and is focusing on automating revenue cycle management tasks.
- The startup identified healthcare as its key market after realizing the ROI was highest in that industry, leading to a strategic shift in its target clients.
- Thoughtful AI's AI agents can autonomously handle 100% of revenue cycle management workflows, helping healthcare clinics save costs on hiring and increasing efficiency.


[Anthropic and Menlo Ventures Partner on $100 Million AI Fund - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-17/anthropic-and-menlo-ventures-partner-on-100-million-ai-fund)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures have partnered on a $100 million AI fund targeting startups using Anthropic's AI models, with investments starting at $100,000. 
- The fund, managed by Menlo Ventures partner Matt Murphy, aims to help AI startups in various sectors like healthcare and cybersecurity, providing funding, advice, and networking opportunities.
- Anthropic will not take stakes in companies receiving investments but offers credits and access to AI models to portfolio startups, in addition to potentially considering them as acquisition targets in the future.


[Anthropic launches $100 million AI fund with Menlo Ventures, ramping up competition with OpenAI - CNBC](https://www.cnbc.com/2024/07/17/anthropic-menlo-ventures-launch-100-million-anthology-fund-for-ai.html)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures collaborate to establish the $100 million Anthology Fund to support early-stage startups and integrate the AI company's technology.
- The Anthology Fund is inspired by the successful iFund partnership between Apple and Kleiner Perkins, aiming to support developers and provide insights into technological needs and advancements.
- Venture capitalists are enhancing their support for AI startups amidst the rapid pace of AI development, with Anthropic offering resources like coaching, meetups, and direct communication to improve its products and assist startups in the ecosystem.


[Menlo Ventures and Anthropic team up to drop $100M on AI startups. - Ben's Bites](https://bensbites.beehiiv.com/p/menlo-ventures-anthropic-team-drop-100m-ai-startups)

Topics: Anthropic, Deals, Funding, Gen AI, Machine Learning, Menlo Ventures, Venture Capital 

- Menlo Ventures and Anthropic launched a $100M Anthology Fund to invest in AI startups, serving as matchmakers for the AI companies and VC money.
- The fund focuses on investing in AI startups from seed to expansion stage, with particular interest in AI infrastructure, cutting-edge AI apps, consumer AI, trust and safety tools, and AI for social good.
- Startups receiving investments not only get cash but also benefit from access to AI models, networking opportunities with AI experts, and cutting-edge technology, potentially birthing the next big innovation in AI.


[Anthropic partners with Menlo for $100M fund investing in AI startups - Cointelegraph](https://cointelegraph.com/news/anthropic-partners-menlo-100m-fund-investing-ai-startups)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic partners with Menlo for a $100M fund to invest in AI startups, focusing on generative AI technology and various AI development areas.
- Anthropic's Anthology Fund supports AI entrepreneurs with product access, research, credits, and Menlo's additional support.
- Anthropic, known for AI chatbot Claude, was founded in 2021 by former OpenAI employees and has received investments from tech giants like Amazon and Google.


[Anthropic is following OpenAI to launch its own $100 million AI startup fund - Quartz](https://qz.com/anthropic-menlo-ventures-100-million-ai-startup-fund-o-1851596353)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, OpenAI, Sam Altman, Venture Capital 

- Anthropic partners with Menlo Ventures to launch a $100 million fund for AI startups with societal benefits.
- The fund aims to support startups working on AI safety tools and apps in fields like healthcare, education, and energy.
- Anthropic's co-founders are former OpenAI engineers who left the company over concerns about scaling up AI tech too quickly.


[The godmother of AI has a new startup already worth $1 billion - The Verge](https://www.theverge.com/2024/7/17/24200496/ai-fei-fei-li-world-labs-andreessen-horowitz-radical-ventures)

Topics: Funding, OpenAI, Startup 

- Fei-Fei Li, known as the "godmother of AI," founded the startup World Labs, already valued at over $1 billion within four months.
- World Labs aims to develop AI capable of advanced reasoning through human-like processing of visual data, with a focus on understanding the three-dimensional physical world.
- The startup has received significant funding rounds, with backing from investors like Andreessen Horowitz and Radical Ventures, and could impact fields such as robotics, augmented reality, virtual reality, and computer vision.


[Andreessen Horowitz-Backed 'Godmother Of AI' Fei-Fei Li Quietly Builds $1B Startup In 4 Months - Benzinga](https://www.benzinga.com/news/24/07/39833376/andreessen-horowitz-backed-godmother-of-ai-fei-fei-li-quietly-builds-1b-startup-in-4-months)

Topics: Deals, Fei-Fei Li, Funding, Investment, Machine Learning, Startup, Venture Capital 

- Fei-Fei Li, known as the "godmother of AI," has rapidly founded a billion-dollar startup named World Labs with significant funding from investors like Andreessen Horowitz and Radical Ventures.
- World Labs focuses on developing "spatial intelligence" in AI, aiming to process visual data like humans, amid increasing investor interest in AI technologies.
- The swift valuation of World Labs reflects the growing investment and competition in the AI sector, with notable funds being launched and major investments like SoftBank Group's $300 million funding in Skild AI.


[Godmother of A.I. Fei-Fei Li Has Created a $1B Startup in Less Than 4 Months - Observer](https://observer.com/2024/07/ai-godmother-fei-fei-li-1b-spatial-intelligence-startup/)

Topics: Fei-Fei Li, Funding, Startup, Venture Capital 

- Fei-Fei Li's startup, World Labs, reached a $1 billion valuation within four months of launching, with investments from firms like Andreessen Horowitz and Radical Ventures.
- The concept of spatial intelligence is being pursued by Li in her new venture, aiming to advance AI beyond current capabilities by linking perception with action in AI systems.
- The startup, World Labs, has successfully raised about $100 million in funding through two rounds since its launch in April.


# CTERA's $80M funding 

[CTERA gets $80m war chest for AI and acquisitions - Blocks and Files](https://blocksandfiles.com/2024/07/17/ctera-gets-funding-for-ai-and-acquisitions/)

Topics: Ctera, Deals, Funding, Investment, Mergers And Acquisitions, Venture Capital 

- CTERA has raised $80 million in funding from PSG Equity for AI expansion and acquisitions in the hybrid cloud data market.
- The focus is on leveraging unstructured data for AI and hybrid cloud file storage growth, with the aim to capture market opportunities and enhance customer value.
- The funding is split between primary funding for business growth and secondary funding to buy out existing shareholders, with an emphasis on utilizing data intelligence services to address challenges in AI data utilization.


[Hybrid cloud data platform CTERA bags $80 million, mostly in secondary funding - Calcalistech](https://www.calcalistech.com/ctechnews/article/hkyrovso0)

Topics: Ctera, Data, Deals, Funding, Products, Venture Capital 

- CTERA, an Israeli company, raised $80 million in funding, with a significant portion as secondary funding used for veteran investors to exercise their holdings, and the rest directly into the company.
- CTERA's software platform enables distributed file storage in private and public cloud environments, with clients including large corporations globally, and strategic partnerships with IBM, HPE, and Hitachi Vantara.
- The funding will be used to accelerate growth for CTERA, a profitable company growing by about 25% annually, focused on reaching revenues of tens of millions of dollars and developing new AI products.


[Ctera Co-Founder On New $80M Growth Funding, Plans For AI Intelligence In File Services - CRN](https://www.crn.com/news/storage/2024/ctera-ceo-on-new-80m-growth-funding-plans-for-ai-intelligence-in-file-services)

Topics: Ctera, Data, Deals, Funding, Venture Capital 

- Ctera secured an $80 million growth investment round to expand global presence and enhance AI capabilities.
- The company is focusing on developing Ctera Data Intelligence to connect existing data platforms into AI systems.
- The competitive landscape in hybrid cloud data services is growing, reflected in recent investments in similar companies like Nasuni.


[How a cold reach-out from a VC algorithm kickstarted fertility startup Legacy's fundraising push - Business Insider](https://www.businessinsider.com/male-fertility-startup-legacy-vc-ai-funding-2024-7)

Topics: Data, Deals, Funding, Machine Learning, Venture Capital 

- Male fertility startup Legacy secured $7.5 million in funding from investors like Samsung Next Ventures, with the help of VC firm TRAC's algorithm-driven investment approach.
- The use of AI in venture capital is showcased through TRAC's selection of Legacy for investment, exemplifying how AI is enhancing and changing traditional VC processes.
- Legacy, focusing on male fertility services, aims to normalize sperm freezing and enhance its offerings with the new funding, including launching health reports based on sperm quality and securing contracts with the US Department of Veterans Affairs.


[See the 9-slide pitch deck that startup Thoughtful AI used to raise $20 million from Drive Capital for its AI agents - Business Insider](https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-capital-series-a-2024-7)

Topics: Funding, Gen AI, Machine Learning, Products, Robots, Venture Capital 

- Thoughtful AI raised $20 million in Series A funding to launch AI agents for healthcare clinics and is focusing on automating revenue cycle management tasks.
- The startup identified healthcare as its key market after realizing the ROI was highest in that industry, leading to a strategic shift in its target clients.
- Thoughtful AI's AI agents can autonomously handle 100% of revenue cycle management workflows, helping healthcare clinics save costs on hiring and increasing efficiency.


[Menlo Ventures and Anthropic team up to drop $100M on AI startups. - Ben's Bites](https://bensbites.beehiiv.com/p/menlo-ventures-anthropic-team-drop-100m-ai-startups)

Topics: Anthropic, Deals, Funding, Gen AI, Machine Learning, Menlo Ventures, Venture Capital 

- Menlo Ventures and Anthropic launched a $100M Anthology Fund to invest in AI startups, serving as matchmakers for the AI companies and VC money.
- The fund focuses on investing in AI startups from seed to expansion stage, with particular interest in AI infrastructure, cutting-edge AI apps, consumer AI, trust and safety tools, and AI for social good.
- Startups receiving investments not only get cash but also benefit from access to AI models, networking opportunities with AI experts, and cutting-edge technology, potentially birthing the next big innovation in AI.


# Anthropic's $100M AI fund 

[Anthropic and Menlo Ventures Partner on $100 Million AI Fund - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-17/anthropic-and-menlo-ventures-partner-on-100-million-ai-fund)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures have partnered on a $100 million AI fund targeting startups using Anthropic's AI models, with investments starting at $100,000. 
- The fund, managed by Menlo Ventures partner Matt Murphy, aims to help AI startups in various sectors like healthcare and cybersecurity, providing funding, advice, and networking opportunities.
- Anthropic will not take stakes in companies receiving investments but offers credits and access to AI models to portfolio startups, in addition to potentially considering them as acquisition targets in the future.


[Anthropic is following OpenAI to launch its own $100 million AI startup fund - Quartz](https://qz.com/anthropic-menlo-ventures-100-million-ai-startup-fund-o-1851596353)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, OpenAI, Sam Altman, Venture Capital 

- Anthropic partners with Menlo Ventures to launch a $100 million fund for AI startups with societal benefits.
- The fund aims to support startups working on AI safety tools and apps in fields like healthcare, education, and energy.
- Anthropic's co-founders are former OpenAI engineers who left the company over concerns about scaling up AI tech too quickly.


[Anthropic launches $100 million AI fund with Menlo Ventures, ramping up competition with OpenAI - CNBC](https://www.cnbc.com/2024/07/17/anthropic-menlo-ventures-launch-100-million-anthology-fund-for-ai.html)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures collaborate to establish the $100 million Anthology Fund to support early-stage startups and integrate the AI company's technology.
- The Anthology Fund is inspired by the successful iFund partnership between Apple and Kleiner Perkins, aiming to support developers and provide insights into technological needs and advancements.
- Venture capitalists are enhancing their support for AI startups amidst the rapid pace of AI development, with Anthropic offering resources like coaching, meetups, and direct communication to improve its products and assist startups in the ecosystem.


[Anthropic partners with Menlo for $100M fund investing in AI startups - Cointelegraph](https://cointelegraph.com/news/anthropic-partners-menlo-100m-fund-investing-ai-startups)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic partners with Menlo for a $100M fund to invest in AI startups, focusing on generative AI technology and various AI development areas.
- Anthropic's Anthology Fund supports AI entrepreneurs with product access, research, credits, and Menlo's additional support.
- Anthropic, known for AI chatbot Claude, was founded in 2021 by former OpenAI employees and has received investments from tech giants like Amazon and Google.


[Menlo Ventures and Anthropic team up to drop $100M on AI startups. - Ben's Bites](https://bensbites.beehiiv.com/p/menlo-ventures-anthropic-team-drop-100m-ai-startups)

Topics: Anthropic, Deals, Funding, Gen AI, Machine Learning, Menlo Ventures, Venture Capital 

- Menlo Ventures and Anthropic launched a $100M Anthology Fund to invest in AI startups, serving as matchmakers for the AI companies and VC money.
- The fund focuses on investing in AI startups from seed to expansion stage, with particular interest in AI infrastructure, cutting-edge AI apps, consumer AI, trust and safety tools, and AI for social good.
- Startups receiving investments not only get cash but also benefit from access to AI models, networking opportunities with AI experts, and cutting-edge technology, potentially birthing the next big innovation in AI.


[The godmother of AI has a new startup already worth $1 billion - The Verge](https://www.theverge.com/2024/7/17/24200496/ai-fei-fei-li-world-labs-andreessen-horowitz-radical-ventures)

Topics: Funding, OpenAI, Startup 

- Fei-Fei Li, known as the "godmother of AI," founded the startup World Labs, already valued at over $1 billion within four months.
- World Labs aims to develop AI capable of advanced reasoning through human-like processing of visual data, with a focus on understanding the three-dimensional physical world.
- The startup has received significant funding rounds, with backing from investors like Andreessen Horowitz and Radical Ventures, and could impact fields such as robotics, augmented reality, virtual reality, and computer vision.


[Andreessen Horowitz-Backed 'Godmother Of AI' Fei-Fei Li Quietly Builds $1B Startup In 4 Months - Benzinga](https://www.benzinga.com/news/24/07/39833376/andreessen-horowitz-backed-godmother-of-ai-fei-fei-li-quietly-builds-1b-startup-in-4-months)

Topics: Deals, Fei-Fei Li, Funding, Investment, Machine Learning, Startup, Venture Capital 

- Fei-Fei Li, known as the "godmother of AI," has rapidly founded a billion-dollar startup named World Labs with significant funding from investors like Andreessen Horowitz and Radical Ventures.
- World Labs focuses on developing "spatial intelligence" in AI, aiming to process visual data like humans, amid increasing investor interest in AI technologies.
- The swift valuation of World Labs reflects the growing investment and competition in the AI sector, with notable funds being launched and major investments like SoftBank Group's $300 million funding in Skild AI.


[Grayscale Launches NewArtificial Intelligence(AI) Fund Featuring 5 Key Protocols - Bitcoinist](https://bitcoinist.com/grayscale-launches-artificial-intelligence-fund/)

Topics: Cryptocurrency, Funding, Products 

- Grayscale launched the Decentralized AI Fund to provide investors exposure to AI protocols in the crypto ecosystem.
- The fund includes protocols for decentralized AI services, solutions to centralized AI-related issues, and infrastructure critical for AI technology development.
- The announcement of the fund had a significant impact on the native tokens of the included protocols, with varying price movements observed following the announcement.


[Godmother of A.I. Fei-Fei Li Has Created a $1B Startup in Less Than 4 Months - Observer](https://observer.com/2024/07/ai-godmother-fei-fei-li-1b-spatial-intelligence-startup/)

Topics: Fei-Fei Li, Funding, Startup, Venture Capital 

- Fei-Fei Li's startup, World Labs, reached a $1 billion valuation within four months of launching, with investments from firms like Andreessen Horowitz and Radical Ventures.
- The concept of spatial intelligence is being pursued by Li in her new venture, aiming to advance AI beyond current capabilities by linking perception with action in AI systems.
- The startup, World Labs, has successfully raised about $100 million in funding through two rounds since its launch in April.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


# Samsung's AI image generation 

[Samsung's AI Makes Images That Are Uncanny Real - Analytics Insight](https://www.analyticsinsight.net/ampstories/artificial-intelligence/samsungs-ai-makes-images-that-are-uncanny-real)

Topics: Big Tech, Korea, Machine Learning, Products, Samsung, Science 

- Samsung's latest AI technology can transform sketches into hyper-realistic images with uncanny realism.
- The new Z Fold 6 integrates this AI, allowing users to turn rough sketches into detailed visuals in seconds.
- Users can sketch directly on the Z Fold 6 and witness their drawings come to life with incredible accuracy.


[Samsung Galaxy AI thinks Im fat, and its not wrong - SamMobile](https://www.sammobile.com/opinion/samsung-galaxy-ai-portrait-studio-thinks-i-am-fat/)

Topics: Big Tech, Data, Gen AI, Language Models, Machine Learning, Products, Recommendation Systems, Review, Samsung 

- One UI 6.1.1's Portrait Studio feature on Samsung Galaxy AI accurately detects double chins and neck fat, potentially motivating individuals to lose weight.
- The AI's renders can vary in accuracy, sometimes making individuals look quite different from their actual appearance, leading to some disappointment.
- Users can't edit the renders created by Galaxy AI, but can strategically take photos from specific angles to avoid highlighting certain features like double chins.


[Microsofts Designer AI image generator comes to iOS, Android with new features - VentureBeat](https://venturebeat.com/ai/microsofts-designer-ai-image-generator-comes-to-ios-android-with-new-features/)

Topics: Android, Art, Design, Big Tech, Gen AI, Machine Learning, Microsoft, Products 

- Microsoft's Designer AI image generator, powered by OpenAI's DALL-E 3 image generation model, is now available as a free app on Android, iOS, and Windows devices.
- New capabilities for Designer include the ability to restyle existing images, prompts to help users start on a blank canvas, and categories like avatars, emojis, stickers, greeting cards, and invitations.
- Microsoft plans to integrate Designer into the Windows 11 Photos app and offers a Copilot Pro subscription for advanced features like 100 daily boosts for image creation and editing.


[Samsung buys UK AI startup to give its products the personal touchOxford Semantic could help your fridge and smartphone pick up on your proclivitiesAI + ML9 hrs|11 - The Register](https://www.theregister.com/2024/07/18/samsung_acquires_oxford_semantic/)

Topics: Big Tech, Customer Service, Deals, Machine Learning, Mergers And Acquisitions, Products, Recommendation Systems, Samsung, Smart Home, Uk 

- Samsung acquired UK knowledge graph startup Oxford Semantic Technologies to enhance its AI capabilities and offer more personalized experiences on its devices.
- Oxford Semantic specializes in knowledge graphs that store and organize data like humans, enabling rapid information retrieval and recommendations. 
- Samsung plans to integrate Oxford Semantic's technology across its range of products, such as mobiles, televisions, and home appliances.


[Smartphone growth continues to surge in Q2 2024 results thanks to AI - Android Central](https://www.androidcentral.com/phones/smartphone-growth-continues-to-surge-in-q2-2024-results-thanks-to-ai)

Topics: Big Tech, Products 

- The smartphone market saw a rebound in the second quarter of 2024, with IDC and Canalys reporting overall growth.
- Samsung led in shipments in Q2 2024, with 19.8% market share, partially attributed to its AI push.
- AI features are driving smartphone market excitement and growth, with Samsung's Galaxy AI suite being a key factor in its success.


[Gigabyte releases AI software to help train your own AI  AI TOP utility taps Gigabyte motherboards, GPUs, SSDs, and power supplies to fine-tune local AI model training - Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai)

Topics: AI Model, Data, Gen AI, Machine Learning, Meta, Products, Taiwan, Testing 

- Gigabyte released AI TOP utility for local AI model training with up to 236B parameters, previously requiring remote data centers.
- The utility works with Gigabyte hardware, including motherboards, SSDs, GPUs, and power supplies, and supports over 70 LLM models on Hugging Face.
- Users can fine-tune AI models locally for enhanced data privacy, without needing AI programming skills, through a user-friendly graphical interface provided by Gigabyte.


[San Jose-based DreamBig, which develops chiplet platforms and was founded in 2019, raised a $75M equity round co-led by Samsung Catalyst Fund and Sehat Sutardja - Crunchbase News](https://news.crunchbase.com/semiconductors-and-5g/dreambig-semiconductor-chip-startup-nvidia-halo/)

Topics: Deals, Funding, Korea, Manufacturing, Products, Samsung, Semiconductor Chips, Venture Capital 

- DreamBig Semiconductor raised $75 million in an equity round with investment from Samsung Catalyst Fund and specializes in chiplet platforms for applications like generative AI.
- The semiconductor industry is benefiting from the current AI trend, with other notable funding rounds including Halo Industries, Black Semiconductor, Etched.ai, and Celestial AI.
- In 2024, VC-backed chip startups have raised nearly $5.7 billion in 210 deals, showing strong venture funding support in the industry.


[Microsoft has let its AI-powered Designer app out of preview mode - Mashable](https://sea.mashable.com/tech/33474/microsoft-has-let-its-ai-powered-designer-app-out-of-preview-mode)

Topics: Art, Design, Big Tech, Microsoft, Products 

- Microsoft's AI-powered Designer app is now available for free to all users with a Microsoft account, allowing for template-based image editing, creation of greeting cards, invitations, and basic designs.
- The app features a Restyle Image function for creating AI-powered avatars in different styles like pop art and cubism, similar to offerings from other tech giants.
- Users can integrate Designer with Microsoft 365 software, access Boosts for faster AI-powered image editing, and Microsoft emphasizes responsible AI use in its blog post.


[Mastering Photoshops Generative AI for Perfect Edits - Fstoppers](https://fstoppers.com/photoshop/mastering-photoshops-generative-ai-perfect-edits-673085)

Topics: Art, Design, Gen AI, Generative AI, Machine Learning, Products 

- Photoshop's generative AI can save time but may yield unexpected results; knowing techniques is crucial.
- Anthony Morganti's video demonstrates using Photoshop's generative AI to remove objects from photos.
- Tips include expanding selections, avoiding detailed selections that confuse AI, using the Remove tool for simple backgrounds, and using the Quick Mask tool for precise selections.


[Microsoft brings controversial AI app to iPhone and Android devices - Laptop Mag](https://www.laptopmag.com/software/creative-media-apps/microsoft-brings-controversial-ai-app-to-iphone-and-android-devices)

Topics: Android, Big Tech, Copilot, Ethics, Facial Recognition, Language Models, Machine Learning, Microsoft, Privacy, Products 

- Microsoft announced the release of a free mobile app for Designer, offering AI-enhanced graphic design and image editing capabilities similar to Adobe Express or Canva.
- The app integrates seamlessly with Microsoft Word, PowerPoint, and Photos, supports over 80 languages, and is available on Windows, iOS, and Android devices.
- Despite a controversial history involving the generation of inappropriate content, Microsoft has enhanced security measures and now offers Designer as a powerful and intuitive graphic design tool on multiple platforms.


# EU antitrust probe on AI deals 

[EU probes Microsoft-OpenAI and Google-Samsung AI deals - Artificial Intelligence News](https://www.artificialintelligence-news.com/news/eu-probes-microsoft-openai-and-google-samsung-ai-deals/)

Topics: Big Tech, Deals, Eu, European Union, Google, Machine Learning, Mergers And Acquisitions, Microsoft, OpenAI, Policy And Regulation, Review, Samsung 

- The European Union has intensified antitrust scrutiny on AI deals involving tech giants like Microsoft-OpenAI and Google-Samsung, amid concerns of potential monopolistic power and anti-competitive practices.
- Microsoft's multibillion-dollar partnership with OpenAI and Google's collaboration with Samsung in AI technologies have drawn regulatory attention due to their implications on market dominance and competition.
- The EU Commission is investigating these partnerships to determine if exclusivity clauses could harm competition in the AI market, signaling a growing focus on fostering fair competition and ensuring smaller competitors have opportunities to compete and innovate.


[EU antitrust regulators want to know if Google and Samsung's chatbot deal hinders rivals - Reuters](https://www.reuters.com/technology/artificial-intelligence/eu-antitrust-regulators-want-know-if-google-samsungs-chatbot-deal-hinders-rivals-2024-07-17/)

Topics: Big Tech, Deals, Eu, European Union, Google, Policy And Regulation, Robots, Samsung 

- EU antitrust regulators are investigating if Google and Samsung's AI deal hinders rival chatbots on Samsung smartphones.
- The European Commission is gathering information to understand the effects of the deal, specifically focusing on potential anti-competitive practices.
- Industry participants are being asked about limitations on other AI systems, interoperability with other chatbots, and unsuccessful attempts to secure deals for pre-installing chatbots on devices.


[Microsoft faces UK antitrust probe after hiring Inflection AI founders and employees - TechCrunch](https://techcrunch.com/2024/07/16/microsoft-faces-uk-antitrust-probe-after-hiring-inflection-ai-founders-and-employees/)

Topics: Big Tech, Machine Learning, Microsoft, Policy And Regulation, Uk 

- Microsoft is facing an antitrust probe in the UK for hiring the core team of Inflection AI, an OpenAI rival previously invested in.  
- The Competition and Markets Authority (CMA) is conducting a "phase 1" merger inquiry to gather evidence on Microsoft's hiring practices and determine if a full probe is needed.  
- Concerns have arisen about big tech companies using new M&A tactics, like hiring key talent, to bypass regulatory scrutiny in the AI space.


[European Commission Asking About Google-Samsung AI Deal - PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2024/european-commission-asking-competitors-about-effects-of-google-samsung-ai-deal/)

Topics: Big Tech, Deals, European Union, Google, Machine Learning, Mergers And Acquisitions, Policy And Regulation, Review, Samsung 

- The European Commission is investigating the effects of Google's AI deal with Samsung on other companies' chatbots on Samsung smartphones, focusing on potential limits on the preinstallation of other AI systems and interoperability with chatbots.
- The European Commission is closely monitoring the distribution channels of big tech companies to ensure a wide range of choices among foundation models for businesses and consumers.
- Discover Financial Services reported a decrease in credit card sales but an increase in debit transaction volumes in the second quarter, with indications of a cautious consumer sentiment.


[Italy's antitrust agency launches a probe into Google and Alphabet over alleged unfair commercial practices related to consent requests Google sends to users - Reuters](https://www.reuters.com/technology/italys-antitrust-launches-probe-into-google-over-unfair-commercial-practices-2024-07-18/)

Topics: Big Tech, Data, Ethics, Google, Laws, Policy And Regulation, Privacy 

- Italy's antitrust agency has launched an investigation into Google and Alphabet for alleged unfair commercial practices related to users' personal data.
- The watchdog claims that Google's consent request for connecting services may be misleading and aggressive.
- Google is accused of limiting users' freedom of choice by encouraging combined usage of personal data across its various services.


[The EU's landmark AI Act 'rushed' out as countdown begins on compliance - The Week](https://theweek.com/law/the-eus-landmark-ai-act-rushed-out-as-countdown-begins-on-compliance)

Topics: Eu, European Union, Laws, Policy And Regulation 

- The EU's pioneering legislation, the EU Artificial Intelligence Act, aims to regulate AI to protect citizens, focusing on different risk categories and banning AI systems that threaten citizens' rights.
- There are criticisms that the legislation is incomplete and ambiguous, with concerns about the regulation's clarity, enforcement, cost of compliance, and its impact on small tech companies in Europe.
- Tech companies must comply with different rules based on risk categories by specific deadlines and there is a need for numerous secondary legislations for implementation, with concerns about the details and tight timeline.


[Metas New Multimodal AI Model May Not Launch in the EU Amidst Regulator Clash - Tech Times](https://www.techtimes.com/articles/306734/20240718/meta-s-new-multimodal-ai-model-launch-eu-amidst-regulator.htm)

Topics: AI Doom, AI Model, Big Tech, Data, Eu, European Union, Language Models, Laws, Machine Learning, Meta, Policy And Regulation, Products 

- Meta's upcoming multimodal AI model may not launch in the EU due to regulatory issues and lack of clarity on data protection rules.
- Concerns mainly revolve around challenges in training AI models with European user data while complying with the GDPR.
- Other companies like Apple have taken similar steps to withhold AI features in the EU due to regulatory uncertainties in data protection laws.


[After Apple, Meta also withholding future AI models from EU countries - 9to5Mac](https://9to5mac.com/2024/07/18/meta-withholding-future-ai-models/)

Topics: Apple, Big Tech, Data, Ethics, European Union, Machine Learning, Meta, Policy And Regulation, Privacy 

- Apple and Meta are withholding future AI models from EU countries due to regulatory uncertainties, following Apple's decision last month for the same reason.
- Apple cited the Digital Markets Act and Meta mentioned a lack of clarity from regulators in the EU as their reasons for holding back AI features.
- Both companies risk losing income and potentially alienating customers by not offering these new technologies in the EU, as the region is crucial for their global income, signaling a high-stakes strategy in hopes of influencing EU regulators.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


[Meta pulls plug on release of advanced AI model in EU - The Guardian](https://www.theguardian.com/technology/article/2024/jul/18/meta-release-advanced-ai-multimodal-llama-model-eu-facebook-owner)

Topics: AI Doom, AI Model, Artificial General Intelligence, Big Tech, Cognitive Science, Cohere, Copilot, Data, Ethics, Eu, European Union, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Meta, OpenAI, Policy And Regulation, Privacy, Products, Robots, Science 

- Meta will not release an advanced AI model in the EU due to regulatory unpredictability, particularly regarding GDPR compliance.
- The decision reflects tensions between big tech companies like Meta and the regulatory environment in the EU, including the EU AI Act and Digital Markets Act.
- Meta's Llama model will not be available in the EU, but text-based versions are accessible, and the company is working on a new text-only model for the region.


# Salesforce AI service agent 

[Salesforce debuts Einstein Service Agent, a new AI Agent for customer self-service - VentureBeat](https://venturebeat.com/ai/salesforce-debuts-einstein-service-agent-a-new-ai-agent-for-customer-self-service/)

Topics: Big Tech, Customer Service, Machine Learning, Products, Robots, Salesforce 

- Salesforce has introduced the Einstein Service Agent, an AI-powered self-service experience for customers that can handle actions such as product returns and refunds.
- The Einstein Service Agent is designed for end-customers to provide a conversational AI interface for answering questions and resolving issues, with plans for expansion beyond customer service scenarios in the future.
- The key difference between the AI agent (Einstein Service Agent) and the AI copilot (Einstein Copilot) is that the agent focuses on customer self-service while the copilot is for direct users of the Salesforce platform, chaining AI models for more autonomy in responses.


[Meet Salesforce's AI-Powered Customer Service Agent, Einstein - Inc.](https://www.inc.com/ben-sherry/meet-salesforces-ai-powered-customer-service-agent-einstein.html)

Topics: Big Tech, Customer Service, Einstein, Machine Learning, Products, Salesforce 

- Salesforce has introduced Einstein Service Agent, an autonomous AI agent designed to enhance customer service interactions, utilizing proprietary data to customize responses and interactions with customers.
- Aiming to address the limitations of traditional chatbots, Einstein Service Agent can adapt and respond to queries dynamically based on real-time data, offering personalized support and making informed decisions without the need for pre-programmed conversation trees.
- Unlike current chatbots that can be time-consuming to set up, Einstein Service Agent boasts quick implementation with out-of-the-box templates and components, offering businesses the potential to customize using Large Language Models such as OpenAI's GPT-4o.


[Salesforce Debuts Autonomous AI Service Agent Einstein - PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2024/salesforce-debuts-autonomous-ai-service-agent-einstein/)

Topics: Big Tech, Customer Service, Einstein, Machine Learning, Products, Salesforce 

- Salesforce introduces its first fully autonomous AI agent, Einstein Service Agent, to enhance customer service efficiency by autonomously handling a broad range of service issues.
- The AI agent, Einstein, distinguishes itself from traditional chatbots by interacting with customers through generative AI to provide faster solutions and offload tedious inquiries from service teams.
- Discover Financial Services experienced a decrease in card sales but an increase in debit transaction volumes, with a rise in delinquency and net charge-off rates compared to the previous year.


[Teleperformance falls on AI competition concerns as Salesforce unveils AI agent - TradingView](https://www.tradingview.com/news/reuters.com,2024:newsml_L8N3J916W:0-teleperformance-falls-on-ai-competition-concerns-as-salesforce-unveils-ai-agent/)

Topics: Customer Service, Salesforce, Stocks 

- Teleperformance shares dropped around 10% after Salesforce announced its fully autonomous AI agent, posing a threat to Teleperformance's conventional call center services.
- The Salesforce Einstein Service Agent is in the pilot phase and will be available later in the year, contributing to concerns about AI competition.
- Analysts suggest that Teleperformance shares are sensitive to rivals' advancements in AI adoption, leading to a decline of about 23.6% year-to-date and potential further losses.


[See the 9-slide pitch deck that startup Thoughtful AI used to raise $20 million from Drive Capital for its AI agents - Business Insider](https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-capital-series-a-2024-7)

Topics: Funding, Gen AI, Machine Learning, Products, Robots, Venture Capital 

- Thoughtful AI raised $20 million in Series A funding to launch AI agents for healthcare clinics and is focusing on automating revenue cycle management tasks.
- The startup identified healthcare as its key market after realizing the ROI was highest in that industry, leading to a strategic shift in its target clients.
- Thoughtful AI's AI agents can autonomously handle 100% of revenue cycle management workflows, helping healthcare clinics save costs on hiring and increasing efficiency.


[Artificial Agency raises $16M to bring AI-powered behavior to games - VentureBeat](https://venturebeat.com/ai/artificial-agency-raises-16m-to-bring-ai-powered-behavior-to-games/)

Topics: Deals, Funding, Gaming, Gen AI, Machine Learning, Rag, Reinforcement Learning, Venture Capital 

- Artificial Agency, an AI startup, raised $16 million in funding for its flagship product, an AI-powered behavior engine that allows game developers to incorporate generative AI seamlessly into various game aspects, promising more personalized and engaging gaming experiences.
- The engine enables game developers to add improvisation to scripted interactions, create full artificial players, and control game elements to enhance player immersion and entertainment, with the aim of reducing churn and keeping players engaged.
- The company, based in Edmonton, is collaborating with AAA studios to develop the engine, which is expected to be widely available for studios in 2025, offering a customizable and extensible solution for game design and development.


[Transition from solo acts to a complete symphony with AI - Fast Company](https://www.fastcompany.com/91154420/transition-from-solo-acts-to-a-complete-symphony-with-ai)

Topics: Big Tech, Cohere, Gen AI, Machine Learning, Music, OpenAI, Robots, Science 

- AI is changing the way we interact with technology and is evolving to become part of a symphony orchestrating multimodal storytelling.
- AI adoption has increased, with a focus on impacting marketing and sales through personalized storytelling and creative work.
- The future of storytelling involves multi-player ideation, deep personalization, AI agents, and an adaptive content loop, enhancing collaboration and productivity in creating tailored and impactful campaigns.


[Simplify Your Business Branding and Content With AI MagicX - TechRepublic](https://www.techrepublic.com/article/ai-magicx-lifetime-subscription/)

Topics: Gen AI, Machine Learning 

- AI MagicX is a platform that offers AI tools for content creation, logo design, creating art, generating articles, and enhancing customer engagement.
- For a discounted price of $99.99 (regularly $972), users can get a lifetime subscription to AI MagicX, enabling them to create professional designs without prior skills, saving time and resources.
- This all-in-one software can help small business owners streamline their workflow, create brand stories, and speed up the document editing process, making it a cost-effective solution for branding and content creation needs.


[Anthropic's Latest Claude AI Model Is Available. This Is What It Gives You - CNET](https://www.cnet.com/tech/services-and-software/anthropics-newest-claude-ai-model-is-available-this-is-what-it-gives-you/)

Topics: Anthropic, Artificial General Intelligence, Big Tech, Claude, Code Assistants, Data, Gen AI, Language Models, Machine Learning, Products, Recommendation Systems, Review, Science 

- Anthropic has released Claude 3.5 Sonnet, a faster and funnier language model for chatbots, available on iOS and Android, with plans for additional models later this year.
- Claude 3.5 Sonnet costs developers $3 per million input tokens and offers improved understanding of nuance, humor, code translations, and multilingual support.
- Claude 3.5 Sonnet provides notable advancements in comprehension, speed, and vision capabilities, catering to workplace tasks like real-time content editing, code migrations, and team collaboration.


[The AI Revolution Is Meeting Reality | Commentary - The Wrap](https://www.thewrap.com/ai-revolution-is-meeting-reality-commentary-alex-kantrowitz/)

Topics: Machine Learning, Opinion 

- Amazon's AI team and partners are optimistic but acknowledge the challenges of building effective generative applications.
- Companies are finding that working with generative AI is difficult due to expensive AI models, data challenges, and complexities in change management.
- According to a survey, only 21% of companies have generative AI in production, while the rest are either piloting or exploring the technology.


# Meta's AI regulatory issues 

[Metas New Multimodal AI Model May Not Launch in the EU Amidst Regulator Clash - Tech Times](https://www.techtimes.com/articles/306734/20240718/meta-s-new-multimodal-ai-model-launch-eu-amidst-regulator.htm)

Topics: AI Doom, AI Model, Big Tech, Data, Eu, European Union, Language Models, Laws, Machine Learning, Meta, Policy And Regulation, Products 

- Meta's upcoming multimodal AI model may not launch in the EU due to regulatory issues and lack of clarity on data protection rules.
- Concerns mainly revolve around challenges in training AI models with European user data while complying with the GDPR.
- Other companies like Apple have taken similar steps to withhold AI features in the EU due to regulatory uncertainties in data protection laws.


[Meta pulls plug on release of advanced AI model in EU - The Guardian](https://www.theguardian.com/technology/article/2024/jul/18/meta-release-advanced-ai-multimodal-llama-model-eu-facebook-owner)

Topics: AI Doom, AI Model, Artificial General Intelligence, Big Tech, Cognitive Science, Cohere, Copilot, Data, Ethics, Eu, European Union, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Meta, OpenAI, Policy And Regulation, Privacy, Products, Robots, Science 

- Meta will not release an advanced AI model in the EU due to regulatory unpredictability, particularly regarding GDPR compliance.
- The decision reflects tensions between big tech companies like Meta and the regulatory environment in the EU, including the EU AI Act and Digital Markets Act.
- Meta's Llama model will not be available in the EU, but text-based versions are accessible, and the company is working on a new text-only model for the region.


[After Apple, Meta also withholding future AI models from EU countries - 9to5Mac](https://9to5mac.com/2024/07/18/meta-withholding-future-ai-models/)

Topics: Apple, Big Tech, Data, Ethics, European Union, Machine Learning, Meta, Policy And Regulation, Privacy 

- Apple and Meta are withholding future AI models from EU countries due to regulatory uncertainties, following Apple's decision last month for the same reason.
- Apple cited the Digital Markets Act and Meta mentioned a lack of clarity from regulators in the EU as their reasons for holding back AI features.
- Both companies risk losing income and potentially alienating customers by not offering these new technologies in the EU, as the region is crucial for their global income, signaling a high-stakes strategy in hopes of influencing EU regulators.


[Meta wont release its multimodal Llama AI model in the EU - The Verge](https://www.theverge.com/2024/7/18/24201041/meta-multimodal-llama-ai-model-launch-eu-regulations)

Topics: AI Model, Big Tech, Data, Ethics, Eu, European Union, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Meta, Policy And Regulation, Privacy, Products, Robots 

- Meta won't release its multimodal Llama AI model in the EU citing regulatory concerns, preventing European companies from using it despite being under an open license.
- The decision follows EU's new AI Act compliance deadlines, with tech companies facing rules around copyright, transparency, and AI applications like predictive policing.
- A text-only version of Meta’s Llama 3 model will still launch in the EU, but future multimodal AI model releases will be excluded, impacting companies outside the EU.


[Meta decides to suspend its generative AI tools in Brazil - Reuters](https://www.reuters.com/technology/artificial-intelligence/meta-decides-suspend-its-generative-ai-tools-brazil-2024-07-17/)

Topics: AI Doom, Art, Design, Big Tech, Brazil, Cohere, Data, Deepfakes, Ethics, Gen AI, Generative AI Tools, Language Models, Machine Learning, Meta, OpenAI, Personal Data, Policy And Regulation, Privacy, Privacy Policy, Products, Robots 

- Meta Platforms decided to suspend the use of generative artificial intelligence tools in Brazil due to objections from the Brazilian government regarding the company's new privacy policy.
- Brazil is a significant market for Meta, with a large user base for platforms like WhatsApp. Meta launched an AI-driven ad targeting program in Sao Paulo recently.
- The National Data Protection Authority in Brazil required Meta to adapt its privacy policy excluding personal data processing for generative AI training, leading to the suspension of the tools while discussions continue.


[Self-Regulation Wont Prevent Problematic Political Uses of Generative AI - Reddit](https://www.reddit.com/r/technology/comments/1e5okgb/selfregulation_wont_prevent_problematic_political/)

Topics: AI Doom, Deepfakes, Disinformation, Ethics, Gen AI, Language Models, Machine Learning, OpenAI, Policy And Regulation, Politics, Rag, Society, Culture 

- Self-regulation is insufficient to prevent problematic political uses of generative AI.
- There are concerns regarding the potential misuse of generative AI in political contexts.
- Legal frameworks and broader governance mechanisms may be needed to address the risks associated with generative AI in politics.


[The EU's landmark AI Act 'rushed' out as countdown begins on compliance - The Week](https://theweek.com/law/the-eus-landmark-ai-act-rushed-out-as-countdown-begins-on-compliance)

Topics: Eu, European Union, Laws, Policy And Regulation 

- The EU's pioneering legislation, the EU Artificial Intelligence Act, aims to regulate AI to protect citizens, focusing on different risk categories and banning AI systems that threaten citizens' rights.
- There are criticisms that the legislation is incomplete and ambiguous, with concerns about the regulation's clarity, enforcement, cost of compliance, and its impact on small tech companies in Europe.
- Tech companies must comply with different rules based on risk categories by specific deadlines and there is a need for numerous secondary legislations for implementation, with concerns about the details and tight timeline.


[Orgs Are Finally Making Moves to Mitigate GenAI Risks - Dark Reading](https://www.darkreading.com/threat-intelligence/orgs-are-finally-making-moves-to-mitigate-genai-risks)

Topics: AI Doom, Anthropic, Artificial General Intelligence, Ethics, Gen AI, Machine Learning, OpenAI, Policy And Regulation, Singularity 

- Enterprise security teams are implementing controls to protect sensitive data from accidental exposure and leaks due to the increased adoption of AI-enabled applications.
- Organizations are focusing on blocking controls and data loss prevention tools to mitigate risks associated with users sending sensitive data to AI apps and services.
- While organizations are currently focused on mitigating risks related to data sent to GenAI services, there is a need for attention towards handling the data received from these services to address potential security and legal risks.


[EU probes Microsoft-OpenAI and Google-Samsung AI deals - Artificial Intelligence News](https://www.artificialintelligence-news.com/news/eu-probes-microsoft-openai-and-google-samsung-ai-deals/)

Topics: Big Tech, Deals, Eu, European Union, Google, Machine Learning, Mergers And Acquisitions, Microsoft, OpenAI, Policy And Regulation, Review, Samsung 

- The European Union has intensified antitrust scrutiny on AI deals involving tech giants like Microsoft-OpenAI and Google-Samsung, amid concerns of potential monopolistic power and anti-competitive practices.
- Microsoft's multibillion-dollar partnership with OpenAI and Google's collaboration with Samsung in AI technologies have drawn regulatory attention due to their implications on market dominance and competition.
- The EU Commission is investigating these partnerships to determine if exclusivity clauses could harm competition in the AI market, signaling a growing focus on fostering fair competition and ensuring smaller competitors have opportunities to compete and innovate.


# Nvidia and Mistral AI model 

[Nvidia and Mistral's new model 'Mistral-NeMo' brings enterprise-grade AI to desktop computers - VentureBeat](https://venturebeat.com/ai/nvidia-and-mistrals-new-model-mistral-nemo-brings-enterprise-grade-ai-to-desktop-computers/)

Topics: Big Tech, Data, Deals, Machine Learning, Nvidia, Products, Semiconductor Chips 

- Nvidia and Mistral AI have collaborated to release Mistral-NeMo, a 12 billion-parameter model that can run on desktop computers, aiming to democratize access to powerful AI capabilities without requiring extensive cloud resources.
- The model's standout feature is its 128,000 token context window, enabling it to process large amounts of text efficiently, which could benefit businesses dealing with complex tasks and limited internet connectivity.
- The release of Mistral-NeMo under the Apache 2.0 license and its availability on Nvidia's AI platform may potentially disrupt the AI software market by offering a local deployment option for businesses, addressing concerns like data privacy, latency, and cost.


[Mistral AI and NVIDIA Unveil Mistral NeMo 12B, a Cutting-Edge Enterprise AI Model - NVIDIA Blogs](https://blogs.nvidia.com/blog/mistral-nvidia-ai-model/)

Topics: Big Tech, Deals, Language Models, Machine Learning, Nvidia, Products 

- Mistral AI and NVIDIA launched Mistral NeMo 12B, a versatile language model for enterprise applications like chatbots, multilingual tasks, coding, and summarization.
- The model offers unprecedented accuracy, flexibility, and efficiency, excelling in diverse tasks with a 128K context length and utilizing the FP8 data format for optimized deployment.
- Mistral NeMo is available as an NVIDIA NIM inference microservice, offering high performance, easy deployment, and enterprise-grade security and support for various applications.


[Gigabyte releases AI software to help train your own AI  AI TOP utility taps Gigabyte motherboards, GPUs, SSDs, and power supplies to fine-tune local AI model training - Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai)

Topics: AI Model, Data, Gen AI, Machine Learning, Meta, Products, Taiwan, Testing 

- Gigabyte released AI TOP utility for local AI model training with up to 236B parameters, previously requiring remote data centers.
- The utility works with Gigabyte hardware, including motherboards, SSDs, GPUs, and power supplies, and supports over 70 LLM models on Hugging Face.
- Users can fine-tune AI models locally for enhanced data privacy, without needing AI programming skills, through a user-friendly graphical interface provided by Gigabyte.


[NVIDIA Partners with Indian PC Builders to Launch Made in India PCs with RTX AI - Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/nvidia-partners-with-indian-pc-builders-to-launch-made-in-india-pcs-with-rtx-ai/)

Topics: AI Doom, Big Tech, Deals, Gaming, India, Machine Learning, Manufacturing, Nvidia, Products, Science, Semiconductor Chips 

- NVIDIA collaborates with six Indian PC builders to launch 'Made in India' PCs with RTX AI technology, targeting Indian gamers, creators, and developers.
- The initiative aims to leverage India's talent pool in AI and computing, with the country's AI market projected to reach $6 billion by 2027.
- RTX AI technology in the PCs enhances performance for gamers and offers efficiency for content creators and developers, potentially reducing computing costs and improving performance by up to four times.


[Missed Out on Nvidia? Billionaires Are Buying These 2 Artificial Intelligence (AI) Stocks Hand Over Fist. - Yahoo Finance](https://finance.yahoo.com/news/missed-nvidia-billionaires-buying-2-081500476.html)

Topics: Finance, Nvidia, Stocks 

- Nvidia created significant value for investors with a ninefold increase in market capitalization over the last 18 months.
- Billionaire fund managers invested in Amazon and Oracle as potential AI opportunities post-Nvidia's success.
- Amazon and Oracle have showcased strong AI implementations within their businesses, indicating potential for positive returns for investors.


[Meta pulls plug on release of advanced AI model in EU - The Guardian](https://www.theguardian.com/technology/article/2024/jul/18/meta-release-advanced-ai-multimodal-llama-model-eu-facebook-owner)

Topics: AI Doom, AI Model, Artificial General Intelligence, Big Tech, Cognitive Science, Cohere, Copilot, Data, Ethics, Eu, European Union, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Meta, OpenAI, Policy And Regulation, Privacy, Products, Robots, Science 

- Meta will not release an advanced AI model in the EU due to regulatory unpredictability, particularly regarding GDPR compliance.
- The decision reflects tensions between big tech companies like Meta and the regulatory environment in the EU, including the EU AI Act and Digital Markets Act.
- Meta's Llama model will not be available in the EU, but text-based versions are accessible, and the company is working on a new text-only model for the region.


[Metas New Multimodal AI Model May Not Launch in the EU Amidst Regulator Clash - Tech Times](https://www.techtimes.com/articles/306734/20240718/meta-s-new-multimodal-ai-model-launch-eu-amidst-regulator.htm)

Topics: AI Doom, AI Model, Big Tech, Data, Eu, European Union, Language Models, Laws, Machine Learning, Meta, Policy And Regulation, Products 

- Meta's upcoming multimodal AI model may not launch in the EU due to regulatory issues and lack of clarity on data protection rules.
- Concerns mainly revolve around challenges in training AI models with European user data while complying with the GDPR.
- Other companies like Apple have taken similar steps to withhold AI features in the EU due to regulatory uncertainties in data protection laws.


[Nvidia Backs Networking Software Startup Arrcuss Latest Round - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-18/nvidia-backs-networking-software-startup-arrcus-s-latest-round)

Topics: Deals, Funding, Investment, Nvidia, Venture Capital 

- Arrcus Inc., a networking software startup, raised $27 million from investors like Nvidia Corp. to enhance its platform for managing data traffic, with plans to expand globally and improve technology.

- Arrcus's software is crucial for optimizing data flow as enterprises adapt to the increasing adoption of artificial intelligence and the resulting growth in network traffic.

- ElevenLabs, a startup focused on AI voice cloning, partnered with Reality Defender, a deepfake detection company, to combat the misuse of deepfake technology during critical election periods, enhancing safety measures and tools.


[Meta wont release its multimodal Llama AI model in the EU - The Verge](https://www.theverge.com/2024/7/18/24201041/meta-multimodal-llama-ai-model-launch-eu-regulations)

Topics: AI Model, Big Tech, Data, Ethics, Eu, European Union, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Meta, Policy And Regulation, Privacy, Products, Robots 

- Meta won't release its multimodal Llama AI model in the EU citing regulatory concerns, preventing European companies from using it despite being under an open license.
- The decision follows EU's new AI Act compliance deadlines, with tech companies facing rules around copyright, transparency, and AI applications like predictive policing.
- A text-only version of Meta’s Llama 3 model will still launch in the EU, but future multimodal AI model releases will be excluded, impacting companies outside the EU.


[Securing the Future of AI at the Edge: An Overview of AI Compute Security - R Street Institute](https://www.rstreet.org/research/securing-the-future-of-ai-at-the-edge-an-overview-of-ai-compute-security/)

Topics: Nvidia, Security, Semiconductor Chips 

- GenAI has significantly boosted analysis, speed, and capabilities across various sectors, leading to projections of 295 million AI-enabled PC and GenAI smartphone shipments by 2024.
- Edge computing provides a solution to challenges faced by centralized AI systems, but it also introduces new cybersecurity vulnerabilities and risks.
- AI compute security, encompassing traditional cybersecurity measures and AI-specific practices, is crucial for protecting the infrastructure, data, and integrity of AI systems within edge devices to maintain interconnected system resilience.


# TSMC AI chip demand surge 

[TSMC smashed second quarter expectations due to AI chip demand - Quartz](https://qz.com/tsmc-second-quarter-earnings-ai-chip-nvidia-apple-trade-1851597577)

Topics: Economics, Expectations, Gen AI, Machine Learning, Manufacturing, Products, Semiconductor Chips, Stocks, Taiwan, Tsmc 

- TSMC exceeded Q2 earnings expectations with a revenue of $20.82 billion, up 40% from the previous year.
- The company reported a quarterly profit of NT$247.85 billion ($7.6 billion), a 36% increase from last year, driven by high demand for AI chips from Nvidia and Apple.
- Despite the strong performance, TSMC faces supply shortages due to high demand for AI capabilities, with the CEO expecting production to catch up by 2025 or 2026.


[TSMC reports Q2 revenue up 40.1% YoY to $20.82B and net income up 36.3% YoY to $7.62B, both above estimates, as demand for advanced AI chips continue to surge - CNBC](https://www.cnbc.com/2024/07/18/tsmc-q2-2024-earnings.html)

Topics: Economics, Manufacturing, Products, Semiconductor Chips, Taiwan, Tsmc 

- TSMC exceeded revenue and profit expectations in Q2 2024 due to high demand for advanced chips used in AI applications.
- The company reported significant revenue and income growth compared to the previous year, with positive forecasts for the future supported by strong demand for 3-nanometer and 5-nanometer technologies.
- The firm expects strong performance in Q3 2024 supported by demand for smartphones and AI-related technology, aiming for continued growth throughout the year with capital investments focused on advanced technologies.


[TSMC Hikes Revenue Outlook to Reflect Heated AI Demand - Yahoo Finance](https://finance.yahoo.com/news/tsmc-profit-surpasses-estimates-ai-053450831.html)

Topics: Machine Learning, Manufacturing, Products, Semiconductor Chips, Taiwan, Tsmc 

- TSMC raised its revenue outlook for 2024 due to strong AI demand, with sales expected to grow more than previously guided.
- The company's confidence in the high AI spending is reflected in its revenue forecast and capital spending outlook, driven by global AI infrastructure investments.
- TSMC's shift in focus towards AI is paying off, with net income rising, high-performance computing contributing over half of revenue, and expectations to meet or exceed gross margin targets.


[More Than Half of TSMCs Sales Are Now High-End Chips Like AI - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-18/more-than-half-of-tsmc-s-sales-are-now-high-end-chips-like-ai)

Topics: Economics, Machine Learning, Manufacturing, Products, Semiconductor Chips, Stocks, Taiwan, Tsmc 

- TSMC now generates over half of its revenue from high-performance computing (HPC) division, with a focus on AI accelerators and processors.
- This shift is significant as the company's key sales driver has moved from smartphones to artificial intelligence technologies.
- TSMC's growth is driven by the AI boom, with HPC sales growing by 28% sequentially and a positive outlook for future sales growth due to the increasing demand for AI chips in various devices.


[TSMC rides AI demand to raise revenue forecast, says no to US joint venture - Reuters](https://www.reuters.com/technology/tsmc-set-report-strong-profit-stock-pressured-by-trump-comments-2024-07-18/)

Topics: Deals, Economics, Manufacturing, Semiconductor Chips, Taiwan, Tsmc 

- TSMC raised its full-year revenue forecast due to high demand for AI chips, benefiting from the global AI boom amid pandemic-led electronics demand slowdown.
- The company plans to continue expanding overseas, including in the U.S., Japan, and Germany, rejecting the possibility of a joint venture in the U.S. despite potential political pressure.
- TSMC expects tight supply for leading-edge nodes like 3nm and 5nm, with strong smartphone and AI-related demand supporting future business growth.


[TSMC raises hopes of a prolonged AI boom - Financial Times](https://www.ft.com/content/24c8ec54-5a9d-4e0a-b7c7-6f01fa53d049)

Topics: Semiconductor Chips, Taiwan, Tsmc 

- TSMC raises hopes for a prolonged AI boom
- Complete digital access to quality FT journalism with expert analysis
- Includes global news, expert opinion, curated newsletters, exclusive analysis


[TSMC boss predicts AI chip shortage through 2025, says Trump comments don't change his strategy - The Register](https://go.theregister.com/feed/www.theregister.com/2024/07/18/tsmc_ceo_predicts_ai_chip/)

Topics: Machine Learning, Manufacturing, Semiconductor Chips, Strategy, Taiwan, Tsmc 

- TSMC CEO predicts AI chip shortage to continue until 2025 or 2026, focusing on balancing supply and demand for advanced chips.
- Overseas expansion plans to address the shortage include building plants in Arizona and Kumamoto, Japan, with additional future expansion in Europe.
- TSMC's efforts to increase CoWoS packaging capacity as a solution to manufacturing holdups, with plans to mass-produce 2 nm chips in the second half of 2025.


[ASML: The AI play selling $370mln shovels - Investing.com](https://au.investing.com/news/stock-market-news/asml-the-ai-play-selling-370mln-shovels-3346805)

Topics: Big Tech, Economics, Investment, Products, Semiconductor Chips, Stocks 

- ASML Holding NV may not be a familiar name in the AI sector, but it plays a crucial role in the supply chain with its EUV lithography machines, which are essential for semiconductor manufacturing.
- ASML's EUV lithography machines, including the High-NA Twinscan, are high-tech, expensive, and heavy equipment that are in demand by major semiconductor manufacturers like Intel, TSMC, and Samsung.
- Despite underperforming compared to some AI peers in terms of stock performance, ASML continues to sell its lithography systems, positioning itself as a key player in the AI sector as a "pick-and-shovel play."


[San Jose-based DreamBig, which develops chiplet platforms and was founded in 2019, raised a $75M equity round co-led by Samsung Catalyst Fund and Sehat Sutardja - Crunchbase News](https://news.crunchbase.com/semiconductors-and-5g/dreambig-semiconductor-chip-startup-nvidia-halo/)

Topics: Deals, Funding, Korea, Manufacturing, Products, Samsung, Semiconductor Chips, Venture Capital 

- DreamBig Semiconductor raised $75 million in an equity round with investment from Samsung Catalyst Fund and specializes in chiplet platforms for applications like generative AI.
- The semiconductor industry is benefiting from the current AI trend, with other notable funding rounds including Halo Industries, Black Semiconductor, Etched.ai, and Celestial AI.
- In 2024, VC-backed chip startups have raised nearly $5.7 billion in 210 deals, showing strong venture funding support in the industry.


# AI search engines 

[Perplexity, Google, and the battle for AI search supremacy - Freethink](https://www.freethink.com/robots-ai/ai-search-engines-llms)

Topics: AI Search, Big Tech, Google, Language Models, Machine Learning, Perplexity 

- AI-powered "answer engines" may replace traditional search models by directly answering user queries without the need for clicking links.
- Large language models (LLMs) are enabling search engines to generate responses to user questions, potentially transforming the future of internet search.
- Challenges such as misinformation, low-quality content (slop), plagiarism, and profitability are significant hurdles that AI-powered search engines like Google and startups face in becoming the dominant search platforms.


[Research Confirms Google AIO Keyword Trends - Search Engine Journal](https://www.searchenginejournal.com/research-confirms-google-aio-keyword-trends/522365/)

Topics: Big Tech, Data, Google, Machine Learning, Research 

- Rapid and dramatic changes in AIO triggers, with shifts observed in keywords triggering AI Overviews as per BrightEdge's research findings
- BrightEdge's assessment of Google search and AI trends, emphasizing Google's dominant position while new players are emerging in an AI-driven search landscape
- Significant decrease in Reddit & Quora referrals from AI Overviews in June 2024, indicating Google's preference for expert content over user-generated discussions


[Googles AI results are showing up less frequently, a study found - The Verge](https://www.theverge.com/2024/7/17/24200544/google-ai-overviews-report)

Topics: Big Tech, Google, Language Models, Machine Learning, Study 

- Google's AI-generated search results are showing up less frequently, with a drop from 11% to 7% of queries from June 1st to June 30th.
- Google had to make adjustments to their AI Overviews feature due to displaying bizarre and nonsensical results, such as recommending putting glue on pizza and eating rocks.
- Google spokesperson defended AI Overviews but mentioned implementing changes like limiting user-generated content and improving detection of nonsensical queries to enhance the feature's usefulness.


[Japanese media say AI search infringes copyright, urge legal reform - Reddit](https://www.reddit.com/r/technology/comments/1e5zuiv/japanese_media_say_ai_search_infringes_copyright/)

Topics: AI Search, Intellectual Property, Japan, Laws, Policy And Regulation 

- Japanese media claim that AI search engines infringe copyright and call for legal reform.
- The discussion in the r/technology subreddit revolves around sharing and discussing the latest technology developments.
- Submissions to r/technology must adhere to specific guidelines related to content, behavior, titles, and moderation procedures.


[The Latest Addition to NBCs Paris Olympics Coverage? Googles AI Tech (Exclusive) - Hollywood Reporter](https://www.hollywoodreporter.com/business/digital/google-ai-features-nbc-2024-paris-olympics-1235951779/)

Topics: Big Tech, Google, Language Models, Machine Learning, Robots, Streaming 

- Google has signed a sponsorship deal with NBC and Team USA to integrate products like Google's Gemini AI, Google Search's "AI Overview," Google Lens, and Google Maps' "Immersive View" into the coverage of the Paris Olympics.
- The partnership aims to enhance viewer experience by providing assistance in learning about sports, planning schedules, showcasing Team USA athletes through social videos, and using AI tools like Google Lens and Circle to Search.
- The integration of Google's AI tech and products into NBC's coverage is seen as a natural fit that enhances the coverage instead of distracting from it, providing viewers with useful and entertaining information during the Olympics.


[Google Gemini is now a lot more helpful on Android phones - Digital Trends](https://www.digitaltrends.com/mobile/google-gemini-is-now-a-lot-more-helpful-on-android-phones/)

Topics: Big Tech, Gemini, Google, Products 

- Google Gemini AI assistant on Android phones can now answer general questions even when the device is locked, enhancing its usability.
- The Gemini feature previously only allowed control of alarms, timers, and media while the phone was locked, now expanded to include answering queries.
- Gemini has been continuously updated with new features like "Ask Photos" for image search and integration with Gmail, Docs, and Workspace applications for email summaries and document analysis.


[The Fastest Way to Lose a Court Case? Use ChatGPT - The Walrus](https://thewalrus.ca/the-fastest-way-to-lose-a-court-case-use-chatgpt/)

Topics: Chatgpt, Code Assistants, Cohere, Gen AI, Language Models, Llms, OpenAI 

- Lawyers are turning to AI, such as ChatGPT, for legal research to save time but face risks of false or inaccurate information due to the model's nature.
- Despite concerns about AI-generated errors, lawyers facing burnout and heavy workloads are still tempted to use AI for its convenience.
- Law societies are setting guidelines for AI use in legal practice, emphasizing responsible implementation, requiring consent from clients, and offering training courses for lawyers to use AI effectively.


# AI in cybersecurity 

[How AI Is Being Used to Improve Cybersecurity for Businesses of All Sizes - Entrepreneur](https://www.entrepreneur.com/science-technology/how-ai-can-improve-cybersecurity-for-businesses-of-all-sizes/476727)

Topics: Cybersecurity, Machine Learning 

- Artificial intelligence plays a dual role in cybersecurity as a protector and an antagonist.
- Threat detection and data protection are key aspects of cybersecurity that AI can enhance significantly.
- Governments and businesses must ensure that AI-based cybersecurity solutions progress faster than the growth of cyber threats in the future.


[Navigating AI Cybersecurity Risk Challenges - AI Business](https://aibusiness.com/cybersecurity/navigating-ai-cybersecurity-risk-challenges)

Topics: Cybersecurity, Risk Management 

- Cybersecurity risk is increasing due to the rapid development of AI without adequate regulations, demanding proactive strategies and robust risk management practices from companies.
- AI-driven cyber risks include vulnerabilities in AI systems, data risks such as data manipulation or loss of services, and risks of AI model theft through network attacks and social engineering techniques.
- Companies need to focus on mitigating risks associated with third parties and software dependencies, including ensuring comprehensive contracts with suppliers, understanding geopolitical risks, and implementing Software Bills of Materials (SBOMs) to maintain visibility and mitigate vulnerabilities.


[Securing the Future of AI at the Edge: An Overview of AI Compute Security - R Street Institute](https://www.rstreet.org/research/securing-the-future-of-ai-at-the-edge-an-overview-of-ai-compute-security/)

Topics: Nvidia, Security, Semiconductor Chips 

- GenAI has significantly boosted analysis, speed, and capabilities across various sectors, leading to projections of 295 million AI-enabled PC and GenAI smartphone shipments by 2024.
- Edge computing provides a solution to challenges faced by centralized AI systems, but it also introduces new cybersecurity vulnerabilities and risks.
- AI compute security, encompassing traditional cybersecurity measures and AI-specific practices, is crucial for protecting the infrastructure, data, and integrity of AI systems within edge devices to maintain interconnected system resilience.


[SAPwned: SAP AI vulnerabilities expose customers' cloud environments and privat - Wiz](https://www.wiz.io/blog/sapwned-sap-ai-vulnerabilities-ai-security)

Topics: AI Vulnerabilities, Cybersecurity, Data, Sap 

- Wiz Research discovered vulnerabilities in SAP AI Core, allowing malicious actors to access customer data and private AI artifacts.
- The vulnerabilities found in SAP AI Core included the ability to read and modify Docker images, gain cluster administrator privileges, and access customers' cloud credentials and private AI artifacts.
- The research highlighted the importance of defense in depth, tenant isolation pitfalls in managed services like Kubernetes, and the unique challenges introduced by the AI R&D process in ensuring code separation and protection of internal assets.


[SAP AI Core Vulnerabilities Expose Customer Data to Cyber Attacks - The Hacker News](https://thehackernews.com/2024/07/sap-ai-core-vulnerabilities-expose.html)

Topics: AI Vulnerabilities, Cybersecurity, Data, Products, Sap 

- Cybersecurity researchers found vulnerabilities in SAP AI Core that could allow attackers to access customer data and contaminate internal artifacts, leading to supply chain attacks.
- These vulnerabilities, collectively named SAPwned, were disclosed on January 25, 2024, and fixed by SAP on May 15, 2024, but had the potential to grant unauthorized access to private artifacts and credentials in cloud environments.
- The flaws in SAP AI Core's tenant isolation mechanisms emphasized the importance of proper tenant isolation practices in AI service providers to prevent the risk of supply chain attacks and unauthorized access to sensitive data.


[Orgs Are Finally Making Moves to Mitigate GenAI Risks - Dark Reading](https://www.darkreading.com/threat-intelligence/orgs-are-finally-making-moves-to-mitigate-genai-risks)

Topics: AI Doom, Anthropic, Artificial General Intelligence, Ethics, Gen AI, Machine Learning, OpenAI, Policy And Regulation, Singularity 

- Enterprise security teams are implementing controls to protect sensitive data from accidental exposure and leaks due to the increased adoption of AI-enabled applications.
- Organizations are focusing on blocking controls and data loss prevention tools to mitigate risks associated with users sending sensitive data to AI apps and services.
- While organizations are currently focused on mitigating risks related to data sent to GenAI services, there is a need for attention towards handling the data received from these services to address potential security and legal risks.


[AI can predict tipping points before they happen - The Economist](https://www.economist.com/science-and-technology/2024/07/17/ai-can-predict-tipping-points-before-they-happen)

Topics: AI Doom, Big Tech, Cognitive Science, Data, Gen AI, Machine Learning, Meta, OpenAI, Risk Management, Science, Singularity 

- AI has been developed to predict tipping points in complex systems before they occur, with potential applications in various fields such as economics and epidemiology.
- Computer scientists in China utilized machine-learning algorithms to accurately predict critical transitions, offering solutions for real-world issues like floods and power outages.
- The ability to foresee tipping points using AI could provide valuable time for prevention and mitigation strategies in advance.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


[Ukraine rushes to create AI-enabled war drones - Reuters](https://www.reuters.com/technology/artificial-intelligence/ukraine-rushes-create-ai-enabled-war-drones-2024-07-18/)

Topics: AI-Enabled War Drones, Gen AI, Machine Learning, Military, Robots, Science, Ukraine 

- Ukraine is rushing to develop AI-enabled drones to gain a technological edge in warfare, with startups focusing on visual systems, terrain mapping, and interconnected drone swarms.
- AI drone control systems are necessary to combat signal jamming from electronic warfare systems and increase hit rates, with drones being developed to lock onto targets through cameras and operate autonomously to bypass jamming effects.
- The development of cost-effective AI targeting systems for drones using tools like Raspberry Pi is crucial for mass deployment along the front line, where thousands of drones are used weekly, to counter the Russian threat and protect drone pilots.


[HPs new AI computer raises the stakes in the battle of tech hardware - Semafor](https://www.semafor.com/article/07/17/2024/hps-new-ai-computer-raises-the-stakes-in-the-battle-of-tech-hardware)

Topics: Big Tech, Gen AI, Hp, Machine Learning, Products 

- HP plans to release a high-performance AI PC, showcasing a trend of tech companies integrating AI into hardware.
- HP's advanced chip allows for AI tasks offline, focusing on enhancing productivity with features like a face-tracking camera and on-device AI companion.
- HP's move aligns with the rise of AI computers in the market, with an expected 60% share by 2027, signaling a shift towards AI-infused technology driven by workplace demands.


# AI in Healthcare & Precision Medicine 

[AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)

Topics: Healthcare, Machine Learning, Science 

- Artificial Intelligence is revolutionizing drug development, particularly in the early stages of drug discovery.
- Discussion between Sam Fazeli and Harren Jhoti explores the role of machine learning, AI, and AlphaFold 3 in drug discovery.
- Astex, co-founded by Harren Jhoti in 1999, has made significant contributions to oncology drugs through fragment-based drug discovery, leading to the development of drugs like Kisqali for breast cancer.


[The AI doctor will see you now: ChatGPT dominates medical exam - CNBC](https://www.cnbc.com/video/2024/07/18/the-ai-doctor-will-see-you-now-chatgpt-dominates-medical-exam.html)

Topics: Chatgpt, Language Models, Machine Learning, OpenAI 

- Former FDA Commissioner Dr. Scott Gottlieb discusses AI large language models excelling in a medical exam for doctors
- ChatGPT stands out in the medical exam for a medical license
- AI-powered healthcare is the future of healthcare


[Google's Healthcare Strategy and the Role of AI - Medscape](https://www.medscape.com/viewarticle/googles-healthcare-strategy-and-role-ai-2024a1000d49)

Topics: AI Doom, Big Tech, Gen AI, Google, Health, Fitness, Healthcare, Machine Learning, Strategy 

- Google's healthcare strategy revolves around developing digital platforms, forming partnerships, and emphasizing disease prevention, with generative AI models as a key component.
- The use of technology, particularly generative AI, is seen as instrumental in democratizing access to healthcare globally and in addressing multifaceted health challenges effectively.
- Google's involvement in healthcare encompasses providing cloud services and platforms to various stakeholders, offering devices such as fitness trackers, and collaborating with organizations like the WHO to disseminate accurate health information.


[nference Establishes AI Initiative with Takeda to Advance Precision Medicine in Inflammatory Bowel Disease - Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)

Topics: Deals, Health, Fitness, Healthcare, Machine Learning, Science 

- nference and Takeda have established a research alliance to leverage AI for optimizing patient care in inflammatory bowel disease (IBD).
- The collaboration, named Project SUCCINCT, will use nference's AI platform to analyze health records and personalize treatment options for IBD patients.
- Takeda and nference aim to advance precision medicine by utilizing AI to improve diagnostic and treatment journeys for patients with IBD.


[See the 9-slide pitch deck that startup Thoughtful AI used to raise $20 million from Drive Capital for its AI agents - Business Insider](https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-capital-series-a-2024-7)

Topics: Funding, Gen AI, Machine Learning, Products, Robots, Venture Capital 

- Thoughtful AI raised $20 million in Series A funding to launch AI agents for healthcare clinics and is focusing on automating revenue cycle management tasks.
- The startup identified healthcare as its key market after realizing the ROI was highest in that industry, leading to a strategic shift in its target clients.
- Thoughtful AI's AI agents can autonomously handle 100% of revenue cycle management workflows, helping healthcare clinics save costs on hiring and increasing efficiency.


[Healthcares AI and Robotics revolution - Euronews](https://www.euronews.com/business/2024/07/17/how-ai-and-robotics-are-transforming-the-9-trillion-healthcare-sector)

Topics: Health, Fitness, Healthcare, Machine Learning, Robotics, Robots 

- The German government endorses a draft budget for 2025 with a total spending limit of €481 billion, focusing on investments, NATO defense spending, and adherence to the debt brake for fiscal stability.
- The budget allocation highlights increased funding for defense, security, childcare benefits, tax relief, and a decrease in aid to Ukraine from €8 billion to €4 billion in 2025.
- Budget negotiations reflect challenges such as the war in Ukraine, economic constraints, and differing priorities among coalition parties, aiming for fiscal restraint while balancing investments and borrowing for stability and growth in a complex financial landscape.


[AI & robotics briefing: What is AI, really? - Nature](https://www.nature.com/articles/d41586-024-02354-0)

Topics: Machine Learning, Meta, OpenAI, Robotics, Robots 

- AI algorithm reveals sperm whales' phonetic alphabet, enhancing understanding of their communication
- AI lie detector outperforms human accuracy, leading to concerns about reliance on technology
- AI ultrasound software in Uganda improves access to pregnancy scans, potentially reducing stillbirths and complications


[AI can predict tipping points before they happen - The Economist](https://www.economist.com/science-and-technology/2024/07/17/ai-can-predict-tipping-points-before-they-happen)

Topics: AI Doom, Big Tech, Cognitive Science, Data, Gen AI, Machine Learning, Meta, OpenAI, Risk Management, Science, Singularity 

- AI has been developed to predict tipping points in complex systems before they occur, with potential applications in various fields such as economics and epidemiology.
- Computer scientists in China utilized machine-learning algorithms to accurately predict critical transitions, offering solutions for real-world issues like floods and power outages.
- The ability to foresee tipping points using AI could provide valuable time for prevention and mitigation strategies in advance.


[The Executive Playbook For Generative AI: Turning Potential Into Business Value - Forbes](https://www.forbes.com/sites/forbestechcouncil/2024/07/18/the-executive-playbook-for-generative-ai-turning-potential-into-business-value/)

Topics: Gen AI, Generative AI 

- Focus on implementing AI to automate workflows and drive immediate value while mitigating risks and ensuring compliance.
- Move quickly but be methodical when implementing GenAI to avoid disappointing results and create meaningful value through pilot AI workflows.
- Create and deploy a unified implementation plan aligned with internal end users' roles to drive near-term value and efficiency improvements in business workflows.


[World's first "MRI for cars" launched in Hawaii, diagnosing vehicles with AI-powered precision - TechSpot](https://www.techspot.com/news/103863-world-first-mri-cars-launched-hawaii-diagnosing-vehicles.html)

Topics: Machine Learning, Products, Science, Testing, Transportation 

- BMW of Honolulu in Hawaii launched the world's first AI-powered system for inspecting vehicles, referred to as an "MRI for cars," which can perform a 360-degree scan of a car in under 5 seconds without the need for human intervention.
- The system provides a visual report on a kiosk for the vehicle owner to view immediately after the scan, allowing for quick and accurate assessment of the car's condition.
- The implementation of this technology aims to improve efficiency at the service center by automating inspections, speeding up the customer check-in process, and providing high-resolution imagery of every car.


# Autonomous Drones 

[Bug brains could help drone swarms find their way home - Popular Science](https://www.popsci.com/technology/drones-ants-memory/)

Topics: Autonomous Vehicles, Cognitive Science, Navigation, Robots, Science 

- Researchers at Delft University of Technology explored using the navigation techniques of ants, specifically their "snapshot model," to help small drones navigate autonomously with minimal memory capacity.
- The insect-inspired drones were successfully able to navigate through an obstacle course using a small amount of memory and a basic microcontroller for visual computing.
- These bug-inspired drones have the potential to navigate autonomously in remote or GPS-inaccessible areas, offering applications in agriculture, stock tracking, search and rescue operations, and military uses, without the need for complex sensors or external infrastructure.


[Ukraine Builds AI Drone Fleets to Combat Russian Forces - PCMag](https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces)

Topics: Autonomous Vehicles, Machine Learning, Military, Robots, Russia, Ukraine 

- Ukrainian tech startups like Vyriy and Swarmer are developing AI-powered drones to combat Russia's ongoing invasion, with Swarmer's drones capable of fleet deployment for strikes and surveillance missions without continuous human involvement.
- AI-powered drones are resistant to Russian signal jamming, providing a strategic advantage over human-operated drones on the battlefield and aiming to eventually operate with no human connection.
- The use of autonomous drones in the war raises ethical concerns but offers the potential to reduce the human cost of the conflict, with reports indicating AI drones have already been employed in targeting Russian energy sources and plans for autonomous weaponized helicopters are in progress.


[56-gram drone gets ant-inspired AI eyes to navigate autonomously - Interesting Engineering](https://interestingengineering.com/innovation/insect-inspired-navigation-robot)

Topics: Autonomous Vehicles, Baidu, Machine Learning, Meta, Navigation, Products, Robots, Science, Testing, Transportation 

- Researchers have developed an ant-inspired navigation system for a 56-gram drone that combines visual homing with odometry for autonomous navigation, aiming at applications like warehouse stock monitoring and gas leak detection.
- Tiny drones weighing from ten to a few hundred grams have potential for real-world applications due to their lightweight design for safe navigation, especially in narrow areas, and their potential for use in scenarios like pest detection in greenhouses.
- The bio-inspired navigation approach for small drones involves capturing snapshots of surroundings, comparing visuals, and correcting odometry drift, making it memory-efficient and suitable for tasks like returning to a starting point in various indoor conditions.


[Ukraine rushes to create AI-enabled war drones - Reuters](https://www.reuters.com/technology/artificial-intelligence/ukraine-rushes-create-ai-enabled-war-drones-2024-07-18/)

Topics: AI-Enabled War Drones, Gen AI, Machine Learning, Military, Robots, Science, Ukraine 

- Ukraine is rushing to develop AI-enabled drones to gain a technological edge in warfare, with startups focusing on visual systems, terrain mapping, and interconnected drone swarms.
- AI drone control systems are necessary to combat signal jamming from electronic warfare systems and increase hit rates, with drones being developed to lock onto targets through cameras and operate autonomously to bypass jamming effects.
- The development of cost-effective AI targeting systems for drones using tools like Raspberry Pi is crucial for mass deployment along the front line, where thousands of drones are used weekly, to counter the Russian threat and protect drone pilots.


[Ukraine Rushes to Create AI-Enabled War Drones - Newsmax](https://www.newsmax.com/world/globaltalk/ukraine-war-drones/2024/07/18/id/1173032/)

Topics: AI Doom, AI-Enabled War Drones, Gen AI, Machine Learning, Military, Science, Ukraine 

- Ukrainian startups are developing AI systems to operate drones for warfare, aiming to gain a technological edge in battle and counter Russian signal jamming. 
- AI-enabled drone development includes visual systems for target identification, terrain mapping for navigation, and complex programs for drones to operate in interconnected swarms. 
- Automation in drone operations using AI not only improves efficiency but also protects drone pilots who are prime targets for enemy fire, with the use of systems like Swarmer's Styx directing a network of drones autonomously.


[Tiny drone "CrazyFlie" weighs only 56 grams (IMAGE) - EurekAlert](https://www.eurekalert.org/multimedia/1034513)

Topics: Manufacturing, Open Source, Products, Robots, Science, Testing 

- A tiny drone named "CrazyFlie" weighs only 56 grams.
- Developed by Delft University of Technology.
- The drone is capable of navigating back to its starting location using an insect-inspired navigation strategy.


[Researchers create insect-inspired autonomous navigation strategy for tiny, lightweight robots - TechXplore](https://techxplore.com/news/2024-07-insect-autonomous-strategy-tiny-lightweight.html)

Topics: Researchers, Robots, Science 

- TU Delft researchers have developed an insect-inspired autonomous navigation strategy for tiny robots based on how ants recognize their environment, enabling robots to return home after long trajectories with minimal resources.
- Tiny, lightweight robots have the potential for applications like monitoring stock in warehouses or detecting gas leaks, as they are safe, can navigate narrow areas, and be deployed in large numbers.
- The proposed navigation strategy allows small robots to cover distances up to 100 meters with minimal memory usage, enabling practical real-world applications without the need for external infrastructure like GPS or wireless beacons.


[From basement to battlefield: Ukrainian startups create low-cost robots to fight Russia - Associated Press News](https://apnews.com/article/ukraine-russia-drones-robots-artificial-intelligence-b07de714659ef859e16c51d7cda4256b)

Topics: Military, Products, Robots, Russia 

- Ukrainian startups are creating low-cost robots to aid in battlefield tasks like mine clearance, evacuations, and equipment carrying, leveraging innovation in secret workshops.
- Defense startups in Ukraine are producing economical unmanned vehicles, like the Odyssey, priced at $35,000, to counteract Russia's resources in the conflict.
- The government is supporting the development of drones and unmanned technologies, with concerns raised by experts regarding the proliferation and impact of low-cost drones and AI in warfare.


[This Is Why the Decentralization of AI Matters - Hacker Noon](https://hackernoon.com/this-is-why-the-decentralization-of-ai-matters)

Topics: Gen AI, OpenAI, Opinion 

- Decentralization of AI with blockchain technology allows for the creation of AI that is owned by and benefits users, IP creators, and other stakeholders, ensuring a fair distribution of power and value.
- The use of DAOs in developing decentralized AI presents an opportunity for inclusive governance involving builders, users, contributors, and IP holders, addressing issues of compensation and power imbalance in the digital age.
- Decentralized AI models like Beamit.space's DAO aim to involve all stakeholders, including IP holders, users generating data for AI training, micro investors, developers, and contributors, in a fair and balanced distribution of power and value, promoting a more effective and inclusive governance system.


# AI Startup Funding 

[Andreessen Horowitz-Backed 'Godmother Of AI' Fei-Fei Li Quietly Builds $1B Startup In 4 Months - Benzinga](https://www.benzinga.com/news/24/07/39833376/andreessen-horowitz-backed-godmother-of-ai-fei-fei-li-quietly-builds-1b-startup-in-4-months)

Topics: Deals, Fei-Fei Li, Funding, Investment, Machine Learning, Startup, Venture Capital 

- Fei-Fei Li, known as the "godmother of AI," has rapidly founded a billion-dollar startup named World Labs with significant funding from investors like Andreessen Horowitz and Radical Ventures.
- World Labs focuses on developing "spatial intelligence" in AI, aiming to process visual data like humans, amid increasing investor interest in AI technologies.
- The swift valuation of World Labs reflects the growing investment and competition in the AI sector, with notable funds being launched and major investments like SoftBank Group's $300 million funding in Skild AI.


[Anthropic is following OpenAI to launch its own $100 million AI startup fund - Quartz](https://qz.com/anthropic-menlo-ventures-100-million-ai-startup-fund-o-1851596353)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, OpenAI, Sam Altman, Venture Capital 

- Anthropic partners with Menlo Ventures to launch a $100 million fund for AI startups with societal benefits.
- The fund aims to support startups working on AI safety tools and apps in fields like healthcare, education, and energy.
- Anthropic's co-founders are former OpenAI engineers who left the company over concerns about scaling up AI tech too quickly.


[Godmother of A.I. Fei-Fei Li Has Created a $1B Startup in Less Than 4 Months - Observer](https://observer.com/2024/07/ai-godmother-fei-fei-li-1b-spatial-intelligence-startup/)

Topics: Fei-Fei Li, Funding, Startup, Venture Capital 

- Fei-Fei Li's startup, World Labs, reached a $1 billion valuation within four months of launching, with investments from firms like Andreessen Horowitz and Radical Ventures.
- The concept of spatial intelligence is being pursued by Li in her new venture, aiming to advance AI beyond current capabilities by linking perception with action in AI systems.
- The startup, World Labs, has successfully raised about $100 million in funding through two rounds since its launch in April.


[Anthropic and Menlo Ventures Partner on $100 Million AI Fund - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-17/anthropic-and-menlo-ventures-partner-on-100-million-ai-fund)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures have partnered on a $100 million AI fund targeting startups using Anthropic's AI models, with investments starting at $100,000. 
- The fund, managed by Menlo Ventures partner Matt Murphy, aims to help AI startups in various sectors like healthcare and cybersecurity, providing funding, advice, and networking opportunities.
- Anthropic will not take stakes in companies receiving investments but offers credits and access to AI models to portfolio startups, in addition to potentially considering them as acquisition targets in the future.


[Anthropic launches $100 million AI fund with Menlo Ventures, ramping up competition with OpenAI - CNBC](https://www.cnbc.com/2024/07/17/anthropic-menlo-ventures-launch-100-million-anthology-fund-for-ai.html)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures collaborate to establish the $100 million Anthology Fund to support early-stage startups and integrate the AI company's technology.
- The Anthology Fund is inspired by the successful iFund partnership between Apple and Kleiner Perkins, aiming to support developers and provide insights into technological needs and advancements.
- Venture capitalists are enhancing their support for AI startups amidst the rapid pace of AI development, with Anthropic offering resources like coaching, meetups, and direct communication to improve its products and assist startups in the ecosystem.


[The godmother of AI has a new startup already worth $1 billion - The Verge](https://www.theverge.com/2024/7/17/24200496/ai-fei-fei-li-world-labs-andreessen-horowitz-radical-ventures)

Topics: Funding, OpenAI, Startup 

- Fei-Fei Li, known as the "godmother of AI," founded the startup World Labs, already valued at over $1 billion within four months.
- World Labs aims to develop AI capable of advanced reasoning through human-like processing of visual data, with a focus on understanding the three-dimensional physical world.
- The startup has received significant funding rounds, with backing from investors like Andreessen Horowitz and Radical Ventures, and could impact fields such as robotics, augmented reality, virtual reality, and computer vision.


[Menlo Ventures and Anthropic team up to drop $100M on AI startups. - Ben's Bites](https://bensbites.beehiiv.com/p/menlo-ventures-anthropic-team-drop-100m-ai-startups)

Topics: Anthropic, Deals, Funding, Gen AI, Machine Learning, Menlo Ventures, Venture Capital 

- Menlo Ventures and Anthropic launched a $100M Anthology Fund to invest in AI startups, serving as matchmakers for the AI companies and VC money.
- The fund focuses on investing in AI startups from seed to expansion stage, with particular interest in AI infrastructure, cutting-edge AI apps, consumer AI, trust and safety tools, and AI for social good.
- Startups receiving investments not only get cash but also benefit from access to AI models, networking opportunities with AI experts, and cutting-edge technology, potentially birthing the next big innovation in AI.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


[San Jose-based DreamBig, which develops chiplet platforms and was founded in 2019, raised a $75M equity round co-led by Samsung Catalyst Fund and Sehat Sutardja - Crunchbase News](https://news.crunchbase.com/semiconductors-and-5g/dreambig-semiconductor-chip-startup-nvidia-halo/)

Topics: Deals, Funding, Korea, Manufacturing, Products, Samsung, Semiconductor Chips, Venture Capital 

- DreamBig Semiconductor raised $75 million in an equity round with investment from Samsung Catalyst Fund and specializes in chiplet platforms for applications like generative AI.
- The semiconductor industry is benefiting from the current AI trend, with other notable funding rounds including Halo Industries, Black Semiconductor, Etched.ai, and Celestial AI.
- In 2024, VC-backed chip startups have raised nearly $5.7 billion in 210 deals, showing strong venture funding support in the industry.


[Playground Ventures Invests in Ethical AI Firm - TipRanks](https://www.tipranks.com/news/company-announcements/playground-ventures-invests-in-ethical-ai-firm)

Topics: Deals, Ethics, Funding, Investment, Venture Capital 

- Playground Ventures Inc. has acquired a 10% stake in Asimov Nexus Inc., a leader in ethical AI technology, to enhance its presence in the AI sector.
- Asimov Nexus is recognized for its advanced AI solutions and strong ethical standards that resonate with Playground's goal of promoting responsible innovation.
- Playground Ventures is also conducting a private placement to raise capital for operational needs and to grow its investments in AI technology.


# Claude AI for Data Analysis 

[How to use Claude 3.5 Sonnet AI for Data Analysis - Geeky Gadgets](https://www.geeky-gadgets.com/ai-data-analysis-2024/)

Topics: Claude, Data, Gen AI, Machine Learning, Products 

- AI Foundations has created a guide on using Claude 3.5 Sonnet AI, a tool for data analysis with features like sourcing datasets, creating visualizations, and building dashboards.
- Claude 3.5 Sonnet enables users to explore and analyze data by understanding its structure, identifying trends, visualizing through graphs, and creating interactive dashboards.
- The tool supports quick decision-making by providing insightful visual representations, allows sharing of interactive dashboards, and offers resources through AI Foundations Community for skill development and staying updated in AI-driven data analysis.


[Anthropic's Latest Claude AI Model Is Available. This Is What It Gives You - CNET](https://www.cnet.com/tech/services-and-software/anthropics-newest-claude-ai-model-is-available-this-is-what-it-gives-you/)

Topics: Anthropic, Artificial General Intelligence, Big Tech, Claude, Code Assistants, Data, Gen AI, Language Models, Machine Learning, Products, Recommendation Systems, Review, Science 

- Anthropic has released Claude 3.5 Sonnet, a faster and funnier language model for chatbots, available on iOS and Android, with plans for additional models later this year.
- Claude 3.5 Sonnet costs developers $3 per million input tokens and offers improved understanding of nuance, humor, code translations, and multilingual support.
- Claude 3.5 Sonnet provides notable advancements in comprehension, speed, and vision capabilities, catering to workplace tasks like real-time content editing, code migrations, and team collaboration.


[I made a personalized workout plan with Claude AI  heres how to do it - Tom's Guide](https://www.tomsguide.com/claude-ai-can-improve-your-fitness-heres-how)

Topics: Claude, Gen AI, Health, Fitness, Machine Learning, Products, Recommendation Systems, Review, Robots 

- Claude AI revolutionizes personal fitness by creating tailored exercise plans based on individual needs, goals, and lifestyle, offering a personalized fitness experience accessible to anyone.
- Users can log into Claude, state fitness goals, input personal details, specify equipment and time schedule, generate initial plans, refine plans, get exercise descriptions, create progressive plans, incorporate rest and recovery, adapt as needed, and combine with nutrition advice for a comprehensive fitness plan.
- Claude's personalized approach provides detailed exercise descriptions, progression strategies, and nutrition advice, resembling a knowledgeable personal trainer experience at an affordable cost, while other interactive AI fitness solutions like the workout app and CAROL Bike offer alternative approaches to personalized fitness.


[Anthropics Claude 3.5 Sonnet ranks number 1 for business and finance in S&P AI Benchmarks by Kensho - Amazon Web Services](https://aws.amazon.com/blogs/machine-learning/anthropic-claude-3-5-sonnet-ranks-number-1-for-business-and-finance-in-sp-ai-benchmarks-by-kensho/)

Topics: Anthropic, Claude, Finance, Fintech, Machine Learning, Products, Stocks 

- Anthropic’s Claude 3.5 Sonnet is ranked number 1 for business and finance in S&P AI Benchmarks by Kensho, showcasing its capabilities in challenging financial and business tasks.
- S&P AI Benchmarks by Kensho focus on evaluating language models for finance and business based on domain knowledge, quantity extraction, and quantitative reasoning, offering a leaderboard for model performance comparison.
- Anthropic’s Claude 3.5 Sonnet excels in tasks such as quantitative reasoning, quantity extraction, and domain knowledge, demonstrating its strength in generating insights and calculations in the financial domain.


[Nvidia and Mistral's new model 'Mistral-NeMo' brings enterprise-grade AI to desktop computers - VentureBeat](https://venturebeat.com/ai/nvidia-and-mistrals-new-model-mistral-nemo-brings-enterprise-grade-ai-to-desktop-computers/)

Topics: Big Tech, Data, Deals, Machine Learning, Nvidia, Products, Semiconductor Chips 

- Nvidia and Mistral AI have collaborated to release Mistral-NeMo, a 12 billion-parameter model that can run on desktop computers, aiming to democratize access to powerful AI capabilities without requiring extensive cloud resources.
- The model's standout feature is its 128,000 token context window, enabling it to process large amounts of text efficiently, which could benefit businesses dealing with complex tasks and limited internet connectivity.
- The release of Mistral-NeMo under the Apache 2.0 license and its availability on Nvidia's AI platform may potentially disrupt the AI software market by offering a local deployment option for businesses, addressing concerns like data privacy, latency, and cost.


[The Fastest Way to Lose a Court Case? Use ChatGPT - The Walrus](https://thewalrus.ca/the-fastest-way-to-lose-a-court-case-use-chatgpt/)

Topics: Chatgpt, Code Assistants, Cohere, Gen AI, Language Models, Llms, OpenAI 

- Lawyers are turning to AI, such as ChatGPT, for legal research to save time but face risks of false or inaccurate information due to the model's nature.
- Despite concerns about AI-generated errors, lawyers facing burnout and heavy workloads are still tempted to use AI for its convenience.
- Law societies are setting guidelines for AI use in legal practice, emphasizing responsible implementation, requiring consent from clients, and offering training courses for lawyers to use AI effectively.


[AI can predict tipping points before they happen - The Economist](https://www.economist.com/science-and-technology/2024/07/17/ai-can-predict-tipping-points-before-they-happen)

Topics: AI Doom, Big Tech, Cognitive Science, Data, Gen AI, Machine Learning, Meta, OpenAI, Risk Management, Science, Singularity 

- AI has been developed to predict tipping points in complex systems before they occur, with potential applications in various fields such as economics and epidemiology.
- Computer scientists in China utilized machine-learning algorithms to accurately predict critical transitions, offering solutions for real-world issues like floods and power outages.
- The ability to foresee tipping points using AI could provide valuable time for prevention and mitigation strategies in advance.


[Meet Salesforce's AI-Powered Customer Service Agent, Einstein - Inc.](https://www.inc.com/ben-sherry/meet-salesforces-ai-powered-customer-service-agent-einstein.html)

Topics: Big Tech, Customer Service, Einstein, Machine Learning, Products, Salesforce 

- Salesforce has introduced Einstein Service Agent, an autonomous AI agent designed to enhance customer service interactions, utilizing proprietary data to customize responses and interactions with customers.
- Aiming to address the limitations of traditional chatbots, Einstein Service Agent can adapt and respond to queries dynamically based on real-time data, offering personalized support and making informed decisions without the need for pre-programmed conversation trees.
- Unlike current chatbots that can be time-consuming to set up, Einstein Service Agent boasts quick implementation with out-of-the-box templates and components, offering businesses the potential to customize using Large Language Models such as OpenAI's GPT-4o.


[Gigabyte releases AI software to help train your own AI  AI TOP utility taps Gigabyte motherboards, GPUs, SSDs, and power supplies to fine-tune local AI model training - Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai)

Topics: AI Model, Data, Gen AI, Machine Learning, Meta, Products, Taiwan, Testing 

- Gigabyte released AI TOP utility for local AI model training with up to 236B parameters, previously requiring remote data centers.
- The utility works with Gigabyte hardware, including motherboards, SSDs, GPUs, and power supplies, and supports over 70 LLM models on Hugging Face.
- Users can fine-tune AI models locally for enhanced data privacy, without needing AI programming skills, through a user-friendly graphical interface provided by Gigabyte.


[Microsoft SpreadsheetLLM AI specifically designed for spreadsheets - Geeky Gadgets](https://www.geeky-gadgets.com/spreadsheetllm-ai/)

Topics: Big Tech, Data, Deals, Llms, Machine Learning, Meta, Microsoft, Products, Review 

- Microsoft introduces SpreadsheetLLM AI aimed at simplifying and enhancing data tasks in spreadsheets.
- SpreadsheetLLM combines large language models with structured spreadsheet data to automate tasks, democratize data insights, and empower decision-making.
- The AI model optimizes performance, preserves relationships in spreadsheets, achieves high accuracy, and represents Microsoft's broader investment in enterprise AI technologies.


# AI Startup Funding 

[Andreessen Horowitz-Backed 'Godmother Of AI' Fei-Fei Li Quietly Builds $1B Startup In 4 Months - Benzinga](https://www.benzinga.com/news/24/07/39833376/andreessen-horowitz-backed-godmother-of-ai-fei-fei-li-quietly-builds-1b-startup-in-4-months)

Topics: Deals, Fei-Fei Li, Funding, Investment, Machine Learning, Startup, Venture Capital 

- Fei-Fei Li, known as the "godmother of AI," has rapidly founded a billion-dollar startup named World Labs with significant funding from investors like Andreessen Horowitz and Radical Ventures.
- World Labs focuses on developing "spatial intelligence" in AI, aiming to process visual data like humans, amid increasing investor interest in AI technologies.
- The swift valuation of World Labs reflects the growing investment and competition in the AI sector, with notable funds being launched and major investments like SoftBank Group's $300 million funding in Skild AI.


[Anthropic is following OpenAI to launch its own $100 million AI startup fund - Quartz](https://qz.com/anthropic-menlo-ventures-100-million-ai-startup-fund-o-1851596353)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, OpenAI, Sam Altman, Venture Capital 

- Anthropic partners with Menlo Ventures to launch a $100 million fund for AI startups with societal benefits.
- The fund aims to support startups working on AI safety tools and apps in fields like healthcare, education, and energy.
- Anthropic's co-founders are former OpenAI engineers who left the company over concerns about scaling up AI tech too quickly.


[Godmother of A.I. Fei-Fei Li Has Created a $1B Startup in Less Than 4 Months - Observer](https://observer.com/2024/07/ai-godmother-fei-fei-li-1b-spatial-intelligence-startup/)

Topics: Fei-Fei Li, Funding, Startup, Venture Capital 

- Fei-Fei Li's startup, World Labs, reached a $1 billion valuation within four months of launching, with investments from firms like Andreessen Horowitz and Radical Ventures.
- The concept of spatial intelligence is being pursued by Li in her new venture, aiming to advance AI beyond current capabilities by linking perception with action in AI systems.
- The startup, World Labs, has successfully raised about $100 million in funding through two rounds since its launch in April.


[Anthropic and Menlo Ventures Partner on $100 Million AI Fund - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-17/anthropic-and-menlo-ventures-partner-on-100-million-ai-fund)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures have partnered on a $100 million AI fund targeting startups using Anthropic's AI models, with investments starting at $100,000. 
- The fund, managed by Menlo Ventures partner Matt Murphy, aims to help AI startups in various sectors like healthcare and cybersecurity, providing funding, advice, and networking opportunities.
- Anthropic will not take stakes in companies receiving investments but offers credits and access to AI models to portfolio startups, in addition to potentially considering them as acquisition targets in the future.


[Anthropic launches $100 million AI fund with Menlo Ventures, ramping up competition with OpenAI - CNBC](https://www.cnbc.com/2024/07/17/anthropic-menlo-ventures-launch-100-million-anthology-fund-for-ai.html)

Topics: Anthropic, Deals, Funding, Investment, Machine Learning, Menlo Ventures, Venture Capital 

- Anthropic and Menlo Ventures collaborate to establish the $100 million Anthology Fund to support early-stage startups and integrate the AI company's technology.
- The Anthology Fund is inspired by the successful iFund partnership between Apple and Kleiner Perkins, aiming to support developers and provide insights into technological needs and advancements.
- Venture capitalists are enhancing their support for AI startups amidst the rapid pace of AI development, with Anthropic offering resources like coaching, meetups, and direct communication to improve its products and assist startups in the ecosystem.


[The godmother of AI has a new startup already worth $1 billion - The Verge](https://www.theverge.com/2024/7/17/24200496/ai-fei-fei-li-world-labs-andreessen-horowitz-radical-ventures)

Topics: Funding, OpenAI, Startup 

- Fei-Fei Li, known as the "godmother of AI," founded the startup World Labs, already valued at over $1 billion within four months.
- World Labs aims to develop AI capable of advanced reasoning through human-like processing of visual data, with a focus on understanding the three-dimensional physical world.
- The startup has received significant funding rounds, with backing from investors like Andreessen Horowitz and Radical Ventures, and could impact fields such as robotics, augmented reality, virtual reality, and computer vision.


[Menlo Ventures and Anthropic team up to drop $100M on AI startups. - Ben's Bites](https://bensbites.beehiiv.com/p/menlo-ventures-anthropic-team-drop-100m-ai-startups)

Topics: Anthropic, Deals, Funding, Gen AI, Machine Learning, Menlo Ventures, Venture Capital 

- Menlo Ventures and Anthropic launched a $100M Anthology Fund to invest in AI startups, serving as matchmakers for the AI companies and VC money.
- The fund focuses on investing in AI startups from seed to expansion stage, with particular interest in AI infrastructure, cutting-edge AI apps, consumer AI, trust and safety tools, and AI for social good.
- Startups receiving investments not only get cash but also benefit from access to AI models, networking opportunities with AI experts, and cutting-edge technology, potentially birthing the next big innovation in AI.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


[San Jose-based DreamBig, which develops chiplet platforms and was founded in 2019, raised a $75M equity round co-led by Samsung Catalyst Fund and Sehat Sutardja - Crunchbase News](https://news.crunchbase.com/semiconductors-and-5g/dreambig-semiconductor-chip-startup-nvidia-halo/)

Topics: Deals, Funding, Korea, Manufacturing, Products, Samsung, Semiconductor Chips, Venture Capital 

- DreamBig Semiconductor raised $75 million in an equity round with investment from Samsung Catalyst Fund and specializes in chiplet platforms for applications like generative AI.
- The semiconductor industry is benefiting from the current AI trend, with other notable funding rounds including Halo Industries, Black Semiconductor, Etched.ai, and Celestial AI.
- In 2024, VC-backed chip startups have raised nearly $5.7 billion in 210 deals, showing strong venture funding support in the industry.


[Playground Ventures Invests in Ethical AI Firm - TipRanks](https://www.tipranks.com/news/company-announcements/playground-ventures-invests-in-ethical-ai-firm)

Topics: Deals, Ethics, Funding, Investment, Venture Capital 

- Playground Ventures Inc. has acquired a 10% stake in Asimov Nexus Inc., a leader in ethical AI technology, to enhance its presence in the AI sector.
- Asimov Nexus is recognized for its advanced AI solutions and strong ethical standards that resonate with Playground's goal of promoting responsible innovation.
- Playground Ventures is also conducting a private placement to raise capital for operational needs and to grow its investments in AI technology.


# AI Smartphones 

[Smartphone growth continues to surge in Q2 2024 results thanks to AI - Android Central](https://www.androidcentral.com/phones/smartphone-growth-continues-to-surge-in-q2-2024-results-thanks-to-ai)

Topics: Big Tech, Products 

- The smartphone market saw a rebound in the second quarter of 2024, with IDC and Canalys reporting overall growth.
- Samsung led in shipments in Q2 2024, with 19.8% market share, partially attributed to its AI push.
- AI features are driving smartphone market excitement and growth, with Samsung's Galaxy AI suite being a key factor in its success.


[Meizus swan song is a budget-friendly phone stuffed with AI features - Phone Arena](https://www.phonearena.com/news/meizu-blue-20-5g-phone-ai-features_id160494)

Topics: AI Features, China, Gen AI, Machine Learning, Products, Review 

- Meizu's new Blue 20 is a budget-friendly 5G smartphone with AI features, signaling the company's shift towards artificial intelligence.
- The phone offers AI-powered tools for generating photos, chatting with a bot, and AI-assisted writing, despite having a basic camera and processor.
- The Meizu Blue 20 comes with a 6.52-inch display, 50MP main camera, 8MP selfie camera, 5,010 mAh battery, 3.5mm audio jack, microSD card slot, and runs on Meizu's Flyme AI OS, exclusive to China.


[Discover the Meizu Blue 20 AI: Affordable AI-Powered Smartphone - GizChina](https://www.gizchina.com/2024/07/16/discover-the-meizu-blue-20-ai-affordable-ai-powered-smartphone/)

Topics: China, Deals, Products, Review 

- Meizu Blue 20 AI is a budget-friendly smartphone with AI features, 50MP camera, and 5G support, featuring a minimalistic design with glass back cover and plastic frame.
- It comes with a 6.52-inch FHD+ display, 90Hz refresh rate, 50MP + 8MP cameras, Unisoc T765 processor, 6/128GB, and 8/128GB variants, and a 5010mAh battery with 10W charging support.
- Priced at 1000 CNY (137 USD), it will be available in the Chinese market from July 22, offering AIOS features and catering to budget-conscious consumers despite the lack of fast charging. 


[Chinas foldable smartphone market heats up with Samsung-ByteDance AI pact, Xiaomi launch - South China Morning Post](https://www.scmp.com/tech/big-tech/article/3270977/chinas-foldable-smartphone-market-heats-samsung-bytedance-ai-pact-xiaomi-launch)

Topics: Big Tech, China, Deals, Machine Learning, Products, Samsung 

- Samsung partners with ByteDance to enhance AI capabilities of Galaxy Z Fold6 and Z Flip6 models in China.
- Xiaomi enters the foldable smartphone market in China with the launch of Mix Flip and Mix Fold 4 models.
- Major Android handset vendors are focusing on foldable, AI-powered models to compete with Apple in the high-end segment of the smartphone market in China.


[Samsung buys UK AI startup to give its products the personal touchOxford Semantic could help your fridge and smartphone pick up on your proclivitiesAI + ML9 hrs|11 - The Register](https://www.theregister.com/2024/07/18/samsung_acquires_oxford_semantic/)

Topics: Big Tech, Customer Service, Deals, Machine Learning, Mergers And Acquisitions, Products, Recommendation Systems, Samsung, Smart Home, Uk 

- Samsung acquired UK knowledge graph startup Oxford Semantic Technologies to enhance its AI capabilities and offer more personalized experiences on its devices.
- Oxford Semantic specializes in knowledge graphs that store and organize data like humans, enabling rapid information retrieval and recommendations. 
- Samsung plans to integrate Oxford Semantic's technology across its range of products, such as mobiles, televisions, and home appliances.


[Meizu Blue 20 announced as ~$140-priced AI phone, key specifications revealed - GizmoChina](https://www.gizmochina.com/2024/07/16/meizu-blue-20-ai-phone-annnounced/)

Topics: China, Meizu Blue 20, Products 

- **Meizu Blue 20**: Meizu's first 1,000 Yuan (~$140) AI smartphone with a 6.52-inch display, Unisoc T765 chipset, options for 6GB/8GB of RAM, and 128GB storage.
- **Specifications**: Features a 50MP primary rear camera, 8MP front camera, 5,010mAh battery, 10W charging support, 90Hz refresh rate on the display, and Flyme AIOS.
- **Availability and Pricing**: Available in three colors, no confirmed pricing for the RAM/storage variants; unclear if it will be released outside of China.


[You can now chat with Google's Gemini AI even when your Android phone is locked - TechRadar](https://www.techradar.com/computing/artificial-intelligence/you-can-now-chat-with-googles-gemini-ai-even-when-your-android-phone-is-locked)

Topics: Big Tech, Gemini, Gen AI, Google, Language Models, Machine Learning, Products 

- Android phone owners can now interact with Google's Gemini AI assistant directly from the lock screen, asking general questions like weather updates and math problems.
- Google is expanding Gemini's capabilities on the lock screen to include extensions for services like Google Maps, Flights, Hotels, and YouTube, offering hands-free access to various features.
- The move to enhance Gemini on Android's lock screen is part of Google's strategy to integrate the AI assistant further into users' daily lives, aiming to increase its competitiveness in the digital assistant market dominated by rivals like Alexa and Siri.


[Meizu Blue 20 unveiled with 5G connectivity and AI features - GSMArena.com news - GSMArena](https://www.gsmarena.com/meizu_blue_20_unveiled_with_5g_connectivity_and_ai_features-news-63754.php)

Topics: AI Features, China, Meizu Blue 20, Products 

- Meizu Blue 20 is a budget device with a Unisoc T765 chipset, 5G connectivity, and AI features, running on Flyme AI OS with text and image generation capabilities.
- The phone features a 6.52" display with 90Hz refresh rate, 50MP rear and 8MP front camera (interpolated from 13MP and 5MP sensors), and includes smart security and privacy features.
- It has a 5,010mAh battery, USB-C charging, microSD card slot, 3.5mm headphone jack, and stereo speakers, with availability currently limited to China.


[Samsung Galaxy AI thinks Im fat, and its not wrong - SamMobile](https://www.sammobile.com/opinion/samsung-galaxy-ai-portrait-studio-thinks-i-am-fat/)

Topics: Big Tech, Data, Gen AI, Language Models, Machine Learning, Products, Recommendation Systems, Review, Samsung 

- One UI 6.1.1's Portrait Studio feature on Samsung Galaxy AI accurately detects double chins and neck fat, potentially motivating individuals to lose weight.
- The AI's renders can vary in accuracy, sometimes making individuals look quite different from their actual appearance, leading to some disappointment.
- Users can't edit the renders created by Galaxy AI, but can strategically take photos from specific angles to avoid highlighting certain features like double chins.


[Samsung's AI Makes Images That Are Uncanny Real - Analytics Insight](https://www.analyticsinsight.net/ampstories/artificial-intelligence/samsungs-ai-makes-images-that-are-uncanny-real)

Topics: Big Tech, Korea, Machine Learning, Products, Samsung, Science 

- Samsung's latest AI technology can transform sketches into hyper-realistic images with uncanny realism.
- The new Z Fold 6 integrates this AI, allowing users to turn rough sketches into detailed visuals in seconds.
- Users can sketch directly on the Z Fold 6 and witness their drawings come to life with incredible accuracy.


# Big Tech Antitrust Investigations 

[Italy's antitrust agency launches a probe into Google and Alphabet over alleged unfair commercial practices related to consent requests Google sends to users - Reuters](https://www.reuters.com/technology/italys-antitrust-launches-probe-into-google-over-unfair-commercial-practices-2024-07-18/)

Topics: Big Tech, Data, Ethics, Google, Laws, Policy And Regulation, Privacy 

- Italy's antitrust agency has launched an investigation into Google and Alphabet for alleged unfair commercial practices related to users' personal data.
- The watchdog claims that Google's consent request for connecting services may be misleading and aggressive.
- Google is accused of limiting users' freedom of choice by encouraging combined usage of personal data across its various services.


[EU probes Microsoft-OpenAI and Google-Samsung AI deals - Artificial Intelligence News](https://www.artificialintelligence-news.com/news/eu-probes-microsoft-openai-and-google-samsung-ai-deals/)

Topics: Big Tech, Deals, Eu, European Union, Google, Machine Learning, Mergers And Acquisitions, Microsoft, OpenAI, Policy And Regulation, Review, Samsung 

- The European Union has intensified antitrust scrutiny on AI deals involving tech giants like Microsoft-OpenAI and Google-Samsung, amid concerns of potential monopolistic power and anti-competitive practices.
- Microsoft's multibillion-dollar partnership with OpenAI and Google's collaboration with Samsung in AI technologies have drawn regulatory attention due to their implications on market dominance and competition.
- The EU Commission is investigating these partnerships to determine if exclusivity clauses could harm competition in the AI market, signaling a growing focus on fostering fair competition and ensuring smaller competitors have opportunities to compete and innovate.


[Microsoft faces UK antitrust probe after hiring Inflection AI founders and employees - TechCrunch](https://techcrunch.com/2024/07/16/microsoft-faces-uk-antitrust-probe-after-hiring-inflection-ai-founders-and-employees/)

Topics: Big Tech, Machine Learning, Microsoft, Policy And Regulation, Uk 

- Microsoft is facing an antitrust probe in the UK for hiring the core team of Inflection AI, an OpenAI rival previously invested in.  
- The Competition and Markets Authority (CMA) is conducting a "phase 1" merger inquiry to gather evidence on Microsoft's hiring practices and determine if a full probe is needed.  
- Concerns have arisen about big tech companies using new M&A tactics, like hiring key talent, to bypass regulatory scrutiny in the AI space.


[EU antitrust regulators want to know if Google and Samsung's chatbot deal hinders rivals - Reuters](https://www.reuters.com/technology/artificial-intelligence/eu-antitrust-regulators-want-know-if-google-samsungs-chatbot-deal-hinders-rivals-2024-07-17/)

Topics: Big Tech, Deals, Eu, European Union, Google, Policy And Regulation, Robots, Samsung 

- EU antitrust regulators are investigating if Google and Samsung's AI deal hinders rival chatbots on Samsung smartphones.
- The European Commission is gathering information to understand the effects of the deal, specifically focusing on potential anti-competitive practices.
- Industry participants are being asked about limitations on other AI systems, interoperability with other chatbots, and unsuccessful attempts to secure deals for pre-installing chatbots on devices.


[European Commission Asking About Google-Samsung AI Deal - PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2024/european-commission-asking-competitors-about-effects-of-google-samsung-ai-deal/)

Topics: Big Tech, Deals, European Union, Google, Machine Learning, Mergers And Acquisitions, Policy And Regulation, Review, Samsung 

- The European Commission is investigating the effects of Google's AI deal with Samsung on other companies' chatbots on Samsung smartphones, focusing on potential limits on the preinstallation of other AI systems and interoperability with chatbots.
- The European Commission is closely monitoring the distribution channels of big tech companies to ensure a wide range of choices among foundation models for businesses and consumers.
- Discover Financial Services reported a decrease in credit card sales but an increase in debit transaction volumes in the second quarter, with indications of a cautious consumer sentiment.


[Apple Denies Using YouTube Videos To Train Its AI Features After Report Claims Tim Cook's Company Used Creators' Content Without Consent - Benzinga](https://www.benzinga.com/news/24/07/39835993/apple-denies-using-youtube-videos-to-train-its-ai-features-after-report-claims-tim-cooks-company-use)

Topics: Apple, Big Tech, Ethics, Privacy, Privacy Concerns, Youtube 

- Apple clarified that its OpenELM model, used for research, does not power any Apple Intelligence features despite controversy over the use of YouTube subtitles for AI training.
- The OpenELM model, developed in April, was created solely for research purposes and to contribute to the open-source language model development community, with no plans for new versions.
- Tech companies using YouTube videos for AI training without creators' consent has raised concerns, with accusations against companies like Apple, leading to platform policy updates regarding web scraping rules.


[After breach, senators ask why AT&T stores call records on AI Data Cloud - Ars Technica](https://arstechnica.com/tech-policy/2024/07/after-breach-senators-ask-why-att-stores-call-records-on-ai-data-cloud/)

Topics: At&T, Big Tech, Breach, Cybersecurity, Data, Data Security, Ethics, Machine Learning, Policy And Regulation, Privacy 

- AT&T faces scrutiny from US senators over storing call and text records on a third-party analytics platform, Snowflake, following a data breach that compromised customer data.
- Snowflake's platform supposedly helped AT&T improve efficiency and customer experience by allowing the company to discard complex internal systems and process vast amounts of data more effectively.
- AT&T defends its use of Snowflake as a "trusted" cloud service for analyzing historical customer data, with data retention periods based on operational needs, legal requirements, and contractual obligations.


[An AI mafia is forming before our eyes: Heres the definitive map - Fast Company](https://www.fastcompany.com/91158379/ai-mafia-definitive-map)

Topics: AI Doom, Big Tech, Ethics, Language Models, Machine Learning 

- Former employees from AI giants like OpenAI, Google, and Apple are forming new companies and startups in the AI sector, reminiscent of the early 2000s PayPal Mafia phenomenon.
- Notable examples include Anthropic, Perplexity, Safe Superintelligence Inc., xAI, Cresta, Daedalus, Gantry, Upstart, Nuro, Lilt, Mistral, H, Humane, and Infactory, founded by ex-employees of these tech giants.
- The industry's fragmentation is attributed to a desire for independence among tech workers in a booming sector, with affiliations with prestigious companies increasing the likelihood of success for new ventures.


[Research Confirms Google AIO Keyword Trends - Search Engine Journal](https://www.searchenginejournal.com/research-confirms-google-aio-keyword-trends/522365/)

Topics: Big Tech, Data, Google, Machine Learning, Research 

- Rapid and dramatic changes in AIO triggers, with shifts observed in keywords triggering AI Overviews as per BrightEdge's research findings
- BrightEdge's assessment of Google search and AI trends, emphasizing Google's dominant position while new players are emerging in an AI-driven search landscape
- Significant decrease in Reddit & Quora referrals from AI Overviews in June 2024, indicating Google's preference for expert content over user-generated discussions


[After breach, senators ask why AT&T stores call records on AI Data Cloud | AT&T ditched internal system, stores user call logs on "trusted" cloud service. - Reddit](https://www.reddit.com/r/technews/comments/1e67t8h/after_breach_senators_ask_why_att_stores_call/)

Topics: At&T, Big Tech, Breach, Cybersecurity, Data, Data Security, Ethics, Policy And Regulation, Privacy 

- AT&T faced scrutiny from senators for storing call records on the "AI Data Cloud"
- The company shifted from an internal system to utilizing a "trusted" cloud service for storing user call logs
- The decision to store call records in this manner raised questions about security and privacy concerns


# Large Language Models 

[When To Use Large Language ModelsAnd When Not To - Forbes](https://www.forbes.com/sites/forbestechcouncil/2024/07/18/when-to-use-large-language-models-and-when-not-to/)

Topics: Chatgpt, Language Models, Large Language Models, Llms, Machine Learning, OpenAI 

- There are situations where integrating large language models (LLMs) like GPT-4o can be beneficial for tasks that don't require perfection, with examples in coding and data handling.
- Keeping humans in the loop is essential for scenarios where LLMs are used but may not be perfect, ensuring validation and human intervention where required.
- Questions to consider whether not to use LLMs include the impact on end users, the need for perfection in outcomes, and understanding the strengths and limitations of LLMs for specific tasks.


[Overcoming the Limits of Large Language Models - Sean Pedersen](https://seanpedersen.github.io/posts/overcoming-llm-limits)

Topics: Big Tech, Chatgpt, Cohere, Gen AI, Language Models, Large Language Models, Llms, Machine Learning, OpenAI, Perplexity, Research, Science 

- Large language models (LLM) have limitations including hallucinations, lack of confidence estimates, and lack of citations.
- Overcoming these limitations involves addressing hallucinations, contradictions in training data, and bootstrapping consistent LLM through curated training data.
- Consistency bootstrapping entails training a base LLM on high-quality text corpus, classifying new documents for consistency, and gradually expanding training data for a big consistent LLM.


[Large language models are rubbish at elementary level math - TechNode](https://technode.com/2024/07/18/large-language-models-are-rubbish-at-elementary-level-math/)

Topics: Chatgpt, Cognitive Science, Education, Large Language Models, Llms, Machine Learning, OpenAI, Opinion, Review 

- Large language models like GPT-4o and Kimi struggle with elementary level math questions, such as comparing 9.11 and 9.9.
- Baidu and Tencent chatbots provide correct answers to such questions by using different methods, such as comparing fractional parts or subtracting numbers.
- Clarifications like "in terms of numerical value" can help AI-powered chatbots like ChatGPT and Kimi provide correct answers to elementary math questions.


[German AI startup DeepL releases its new translation model with support for German, Japanese, and Simplified Chinese to and from English, available in DeepL Pro - SiliconANGLE](https://siliconangle.com/2024/07/17/deepl-launches-newest-dedicated-translation-llm-business-users/)

Topics: China, Data, Deals, Gen AI, Japan, Language Models, Machine Learning, Meta, Products, Review, Science 

- DeepL launched a new dedicated translation large language model specifically designed for business users, which has been significantly improved over previous models through enhanced accuracy and precision.
- The new model outperformed its predecessor and popular tools like Google Translate and ChatGPT, showing 1.7 times greater accuracy and better performance in various language translations.
- DeepL targets global enterprises with linguistic challenges, highlighting the importance of accuracy in business translations and positioning its model as more efficient and productive for translation tasks.


[OpenAI used a game to help AI models explain themselves better - VentureBeat](https://venturebeat.com/ai/openai-used-a-game-to-help-ai-models-explain-themselves-better/)

Topics: AI Models, Artificial General Intelligence, Big Tech, Gen AI, Language Models, Machine Learning, OpenAI, Reinforcement Learning, Science 

- OpenAI has developed an algorithm to improve the legibility and explainability of large language models (LLMs) like GPT-4, aiming to foster trust in AI systems, especially in critical fields.
- The algorithm is based on a Prover-Verifier Game where a powerful "prover" AI model tries to persuade a less powerful "verifier" model in providing correct answers, enhancing the models' ability to explain their reasoning.
- By engaging the models in rounds of the game and training them to be more transparent and accurate in their responses, OpenAI's method aims to enhance the correctness and understandability of AI outputs, crucial for trust and safety in real-world applications.


[Electra: Pre-Training Text Encoders as Discriminators Rather Than Generators (2020) - arXiv](https://arxiv.org/abs/2003.10555)

Topics: Language Models, Llms, Machine Learning, OpenAI 

- Proposed an efficient pre-training task called "replaced token detection" as an alternative to traditional masked language modeling (MLM) methods like BERT.
- Introduced a discriminative model that predicts whether each token in the corrupted input was replaced by a generator sample or not, showing improved efficiency over MLM by defining the task over all input tokens.
- Demonstrated that the approach outperforms BERT, GPT, RoBERTa, and XLNet in terms of contextual representation learning, achieving strong results even with smaller models and less compute resources.


[Mistral AI and NVIDIA Unveil Mistral NeMo 12B, a Cutting-Edge Enterprise AI Model - NVIDIA Blogs](https://blogs.nvidia.com/blog/mistral-nvidia-ai-model/)

Topics: Big Tech, Deals, Language Models, Machine Learning, Nvidia, Products 

- Mistral AI and NVIDIA launched Mistral NeMo 12B, a versatile language model for enterprise applications like chatbots, multilingual tasks, coding, and summarization.
- The model offers unprecedented accuracy, flexibility, and efficiency, excelling in diverse tasks with a 128K context length and utilizing the FP8 data format for optimized deployment.
- Mistral NeMo is available as an NVIDIA NIM inference microservice, offering high performance, easy deployment, and enterprise-grade security and support for various applications.


[TTT models might be the next frontier in generative AI - TechCrunch](https://techcrunch.com/2024/07/17/ttt-models-might-be-the-next-frontier-in-generative-ai/)

Topics: Art, Design, Artificial General Intelligence, Cohere, Data, Gen AI, Generative AI, Language Models, Machine Learning, Nvidia, OpenAI, Science 

- The hunt is on for new architectures in generative AI as transformers face computation-related roadblocks due to inefficient data processing and analysis.
- Test-time training (TTT) models, developed by a research team from Stanford, UC San Diego, UC Berkeley, and Meta, offer a promising alternative to transformers by processing more data efficiently without high compute power demands.
- TTT models utilize a machine learning model within a model to encode data into representative variables, making them highly performant and capable of processing billions of data pieces across various formats, potentially revolutionizing generative AI.


[Maths test stumps AI models: which number is bigger, 9.90 or 9.11? - South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3270851/maths-test-stumps-ai-models-which-number-bigger-990-or-911)

Topics: AI Models, Cognitive Science, Education, Gen AI, Machine Learning, OpenAI, Perplexity, Robots, Testing 

- Large language models struggle with basic maths knowledge.
- AI models can only predict answers based on training data.
- AI technology's evolution is highlighted by the math issue faced by models.


[Meta wont release its multimodal Llama AI model in the EU - The Verge](https://www.theverge.com/2024/7/18/24201041/meta-multimodal-llama-ai-model-launch-eu-regulations)

Topics: AI Model, Big Tech, Data, Ethics, Eu, European Union, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Meta, Policy And Regulation, Privacy, Products, Robots 

- Meta won't release its multimodal Llama AI model in the EU citing regulatory concerns, preventing European companies from using it despite being under an open license.
- The decision follows EU's new AI Act compliance deadlines, with tech companies facing rules around copyright, transparency, and AI applications like predictive policing.
- A text-only version of Meta’s Llama 3 model will still launch in the EU, but future multimodal AI model releases will be excluded, impacting companies outside the EU.


# AI-Powered Customer Service Agents 

[Salesforce debuts Einstein Service Agent, a new AI Agent for customer self-service - VentureBeat](https://venturebeat.com/ai/salesforce-debuts-einstein-service-agent-a-new-ai-agent-for-customer-self-service/)

Topics: Big Tech, Customer Service, Machine Learning, Products, Robots, Salesforce 

- Salesforce has introduced the Einstein Service Agent, an AI-powered self-service experience for customers that can handle actions such as product returns and refunds.
- The Einstein Service Agent is designed for end-customers to provide a conversational AI interface for answering questions and resolving issues, with plans for expansion beyond customer service scenarios in the future.
- The key difference between the AI agent (Einstein Service Agent) and the AI copilot (Einstein Copilot) is that the agent focuses on customer self-service while the copilot is for direct users of the Salesforce platform, chaining AI models for more autonomy in responses.


[Meet Salesforce's AI-Powered Customer Service Agent, Einstein - Inc.](https://www.inc.com/ben-sherry/meet-salesforces-ai-powered-customer-service-agent-einstein.html)

Topics: Big Tech, Customer Service, Einstein, Machine Learning, Products, Salesforce 

- Salesforce has introduced Einstein Service Agent, an autonomous AI agent designed to enhance customer service interactions, utilizing proprietary data to customize responses and interactions with customers.
- Aiming to address the limitations of traditional chatbots, Einstein Service Agent can adapt and respond to queries dynamically based on real-time data, offering personalized support and making informed decisions without the need for pre-programmed conversation trees.
- Unlike current chatbots that can be time-consuming to set up, Einstein Service Agent boasts quick implementation with out-of-the-box templates and components, offering businesses the potential to customize using Large Language Models such as OpenAI's GPT-4o.


[Salesforce Debuts Autonomous AI Service Agent Einstein - PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2024/salesforce-debuts-autonomous-ai-service-agent-einstein/)

Topics: Big Tech, Customer Service, Einstein, Machine Learning, Products, Salesforce 

- Salesforce introduces its first fully autonomous AI agent, Einstein Service Agent, to enhance customer service efficiency by autonomously handling a broad range of service issues.
- The AI agent, Einstein, distinguishes itself from traditional chatbots by interacting with customers through generative AI to provide faster solutions and offload tedious inquiries from service teams.
- Discover Financial Services experienced a decrease in card sales but an increase in debit transaction volumes, with a rise in delinquency and net charge-off rates compared to the previous year.


[Teleperformance falls on AI competition concerns as Salesforce unveils AI agent - TradingView](https://www.tradingview.com/news/reuters.com,2024:newsml_L8N3J916W:0-teleperformance-falls-on-ai-competition-concerns-as-salesforce-unveils-ai-agent/)

Topics: Customer Service, Salesforce, Stocks 

- Teleperformance shares dropped around 10% after Salesforce announced its fully autonomous AI agent, posing a threat to Teleperformance's conventional call center services.
- The Salesforce Einstein Service Agent is in the pilot phase and will be available later in the year, contributing to concerns about AI competition.
- Analysts suggest that Teleperformance shares are sensitive to rivals' advancements in AI adoption, leading to a decline of about 23.6% year-to-date and potential further losses.


[See the 9-slide pitch deck that startup Thoughtful AI used to raise $20 million from Drive Capital for its AI agents - Business Insider](https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-capital-series-a-2024-7)

Topics: Funding, Gen AI, Machine Learning, Products, Robots, Venture Capital 

- Thoughtful AI raised $20 million in Series A funding to launch AI agents for healthcare clinics and is focusing on automating revenue cycle management tasks.
- The startup identified healthcare as its key market after realizing the ROI was highest in that industry, leading to a strategic shift in its target clients.
- Thoughtful AI's AI agents can autonomously handle 100% of revenue cycle management workflows, helping healthcare clinics save costs on hiring and increasing efficiency.


[Simplify Your Business Branding and Content With AI MagicX - TechRepublic](https://www.techrepublic.com/article/ai-magicx-lifetime-subscription/)

Topics: Gen AI, Machine Learning 

- AI MagicX is a platform that offers AI tools for content creation, logo design, creating art, generating articles, and enhancing customer engagement.
- For a discounted price of $99.99 (regularly $972), users can get a lifetime subscription to AI MagicX, enabling them to create professional designs without prior skills, saving time and resources.
- This all-in-one software can help small business owners streamline their workflow, create brand stories, and speed up the document editing process, making it a cost-effective solution for branding and content creation needs.


[Playing the Long Game: How AI Can Optimize Enablement in Longtail Software Partnerships - Hacker Noon](https://hackernoon.com/playing-the-long-game-how-ai-can-optimize-enablement-in-longtail-software-partnerships)

Topics: Opinion 

- Larger partners receive personalized support, while smaller long-tail partners often face disparities in enablement processes, hindering growth for both.
- Generative AI can offer tier-one treatment affordably to long-tail partners by providing necessary support.
- Over 50% of SaaS licenses remain unused for more than 90 days, highlighting efficiency challenges that AI can help address.


[Transition from solo acts to a complete symphony with AI - Fast Company](https://www.fastcompany.com/91154420/transition-from-solo-acts-to-a-complete-symphony-with-ai)

Topics: Big Tech, Cohere, Gen AI, Machine Learning, Music, OpenAI, Robots, Science 

- AI is changing the way we interact with technology and is evolving to become part of a symphony orchestrating multimodal storytelling.
- AI adoption has increased, with a focus on impacting marketing and sales through personalized storytelling and creative work.
- The future of storytelling involves multi-player ideation, deep personalization, AI agents, and an adaptive content loop, enhancing collaboration and productivity in creating tailored and impactful campaigns.


[Anthropic's Latest Claude AI Model Is Available. This Is What It Gives You - CNET](https://www.cnet.com/tech/services-and-software/anthropics-newest-claude-ai-model-is-available-this-is-what-it-gives-you/)

Topics: Anthropic, Artificial General Intelligence, Big Tech, Claude, Code Assistants, Data, Gen AI, Language Models, Machine Learning, Products, Recommendation Systems, Review, Science 

- Anthropic has released Claude 3.5 Sonnet, a faster and funnier language model for chatbots, available on iOS and Android, with plans for additional models later this year.
- Claude 3.5 Sonnet costs developers $3 per million input tokens and offers improved understanding of nuance, humor, code translations, and multilingual support.
- Claude 3.5 Sonnet provides notable advancements in comprehension, speed, and vision capabilities, catering to workplace tasks like real-time content editing, code migrations, and team collaboration.


[Artificial Agency raises $16M to bring AI-powered behavior to games - VentureBeat](https://venturebeat.com/ai/artificial-agency-raises-16m-to-bring-ai-powered-behavior-to-games/)

Topics: Deals, Funding, Gaming, Gen AI, Machine Learning, Rag, Reinforcement Learning, Venture Capital 

- Artificial Agency, an AI startup, raised $16 million in funding for its flagship product, an AI-powered behavior engine that allows game developers to incorporate generative AI seamlessly into various game aspects, promising more personalized and engaging gaming experiences.
- The engine enables game developers to add improvisation to scripted interactions, create full artificial players, and control game elements to enhance player immersion and entertainment, with the aim of reducing churn and keeping players engaged.
- The company, based in Edmonton, is collaborating with AAA studios to develop the engine, which is expected to be widely available for studios in 2025, offering a customizable and extensible solution for game design and development.


# AI Ethics and Regulation 

[Orgs Are Finally Making Moves to Mitigate GenAI Risks - Dark Reading](https://www.darkreading.com/threat-intelligence/orgs-are-finally-making-moves-to-mitigate-genai-risks)

Topics: AI Doom, Anthropic, Artificial General Intelligence, Ethics, Gen AI, Machine Learning, OpenAI, Policy And Regulation, Singularity 

- Enterprise security teams are implementing controls to protect sensitive data from accidental exposure and leaks due to the increased adoption of AI-enabled applications.
- Organizations are focusing on blocking controls and data loss prevention tools to mitigate risks associated with users sending sensitive data to AI apps and services.
- While organizations are currently focused on mitigating risks related to data sent to GenAI services, there is a need for attention towards handling the data received from these services to address potential security and legal risks.


[Self-Regulation Wont Prevent Problematic Political Uses of Generative AI - Reddit](https://www.reddit.com/r/technology/comments/1e5okgb/selfregulation_wont_prevent_problematic_political/)

Topics: AI Doom, Deepfakes, Disinformation, Ethics, Gen AI, Language Models, Machine Learning, OpenAI, Policy And Regulation, Politics, Rag, Society, Culture 

- Self-regulation is insufficient to prevent problematic political uses of generative AI.
- There are concerns regarding the potential misuse of generative AI in political contexts.
- Legal frameworks and broader governance mechanisms may be needed to address the risks associated with generative AI in politics.


[Metas New Multimodal AI Model May Not Launch in the EU Amidst Regulator Clash - Tech Times](https://www.techtimes.com/articles/306734/20240718/meta-s-new-multimodal-ai-model-launch-eu-amidst-regulator.htm)

Topics: AI Doom, AI Model, Big Tech, Data, Eu, European Union, Language Models, Laws, Machine Learning, Meta, Policy And Regulation, Products 

- Meta's upcoming multimodal AI model may not launch in the EU due to regulatory issues and lack of clarity on data protection rules.
- Concerns mainly revolve around challenges in training AI models with European user data while complying with the GDPR.
- Other companies like Apple have taken similar steps to withhold AI features in the EU due to regulatory uncertainties in data protection laws.


[Playground Ventures Invests in Ethical AI Firm - TipRanks](https://www.tipranks.com/news/company-announcements/playground-ventures-invests-in-ethical-ai-firm)

Topics: Deals, Ethics, Funding, Investment, Venture Capital 

- Playground Ventures Inc. has acquired a 10% stake in Asimov Nexus Inc., a leader in ethical AI technology, to enhance its presence in the AI sector.
- Asimov Nexus is recognized for its advanced AI solutions and strong ethical standards that resonate with Playground's goal of promoting responsible innovation.
- Playground Ventures is also conducting a private placement to raise capital for operational needs and to grow its investments in AI technology.


[Navigating AI Cybersecurity Risk Challenges - AI Business](https://aibusiness.com/cybersecurity/navigating-ai-cybersecurity-risk-challenges)

Topics: Cybersecurity, Risk Management 

- Cybersecurity risk is increasing due to the rapid development of AI without adequate regulations, demanding proactive strategies and robust risk management practices from companies.
- AI-driven cyber risks include vulnerabilities in AI systems, data risks such as data manipulation or loss of services, and risks of AI model theft through network attacks and social engineering techniques.
- Companies need to focus on mitigating risks associated with third parties and software dependencies, including ensuring comprehensive contracts with suppliers, understanding geopolitical risks, and implementing Software Bills of Materials (SBOMs) to maintain visibility and mitigate vulnerabilities.


[After Apple, Meta also withholding future AI models from EU countries - 9to5Mac](https://9to5mac.com/2024/07/18/meta-withholding-future-ai-models/)

Topics: Apple, Big Tech, Data, Ethics, European Union, Machine Learning, Meta, Policy And Regulation, Privacy 

- Apple and Meta are withholding future AI models from EU countries due to regulatory uncertainties, following Apple's decision last month for the same reason.
- Apple cited the Digital Markets Act and Meta mentioned a lack of clarity from regulators in the EU as their reasons for holding back AI features.
- Both companies risk losing income and potentially alienating customers by not offering these new technologies in the EU, as the region is crucial for their global income, signaling a high-stakes strategy in hopes of influencing EU regulators.


[The EU's landmark AI Act 'rushed' out as countdown begins on compliance - The Week](https://theweek.com/law/the-eus-landmark-ai-act-rushed-out-as-countdown-begins-on-compliance)

Topics: Eu, European Union, Laws, Policy And Regulation 

- The EU's pioneering legislation, the EU Artificial Intelligence Act, aims to regulate AI to protect citizens, focusing on different risk categories and banning AI systems that threaten citizens' rights.
- There are criticisms that the legislation is incomplete and ambiguous, with concerns about the regulation's clarity, enforcement, cost of compliance, and its impact on small tech companies in Europe.
- Tech companies must comply with different rules based on risk categories by specific deadlines and there is a need for numerous secondary legislations for implementation, with concerns about the details and tight timeline.


[EU probes Microsoft-OpenAI and Google-Samsung AI deals - Artificial Intelligence News](https://www.artificialintelligence-news.com/news/eu-probes-microsoft-openai-and-google-samsung-ai-deals/)

Topics: Big Tech, Deals, Eu, European Union, Google, Machine Learning, Mergers And Acquisitions, Microsoft, OpenAI, Policy And Regulation, Review, Samsung 

- The European Union has intensified antitrust scrutiny on AI deals involving tech giants like Microsoft-OpenAI and Google-Samsung, amid concerns of potential monopolistic power and anti-competitive practices.
- Microsoft's multibillion-dollar partnership with OpenAI and Google's collaboration with Samsung in AI technologies have drawn regulatory attention due to their implications on market dominance and competition.
- The EU Commission is investigating these partnerships to determine if exclusivity clauses could harm competition in the AI market, signaling a growing focus on fostering fair competition and ensuring smaller competitors have opportunities to compete and innovate.


[Secretive China Agency Backtracks on Aggressive Policies After $1 Trillion Rout - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-18/china-regulator-overseeing-ai-overseas-ipos-relaxes-policies-to-promote-growth)

Topics: Bubble, China, Economics, Finance, Policy And Regulation, Stocks 

- The Cyberspace Administration of China (CAC) has evolved into a powerful regulatory body overseeing data, AI, and overseas listings, with implications for business and the economy.
- The CAC's policies have created confusion and caused concern among investors due to perceived risks of policy flip-flops and challenges in managing economic growth and national security priorities.
- The CAC's recent relaxations of some data transfer rules have alleviated concerns for businesses, but doubts persist about the agency's direction and impact on tech regulations in China.


[Meta pulls plug on release of advanced AI model in EU - The Guardian](https://www.theguardian.com/technology/article/2024/jul/18/meta-release-advanced-ai-multimodal-llama-model-eu-facebook-owner)

Topics: AI Doom, AI Model, Artificial General Intelligence, Big Tech, Cognitive Science, Cohere, Copilot, Data, Ethics, Eu, European Union, Gen AI, Intellectual Property, Language Models, Laws, Llms, Machine Learning, Meta, OpenAI, Policy And Regulation, Privacy, Products, Robots, Science 

- Meta will not release an advanced AI model in the EU due to regulatory unpredictability, particularly regarding GDPR compliance.
- The decision reflects tensions between big tech companies like Meta and the regulatory environment in the EU, including the EU AI Act and Digital Markets Act.
- Meta's Llama model will not be available in the EU, but text-based versions are accessible, and the company is working on a new text-only model for the region.


# AI Art by Microsoft 

[Microsoft wants you to pay $300 per year for AI art in Word - PC World](https://www.pcworld.com/article/2403020/microsoft-wants-you-to-pay-300-per-year-for-ai-art-in-word.html)

Topics: Art, Design, Big Tech, Machine Learning, Microsoft, Products 

- Microsoft is embedding AI art capabilities from Microsoft Designer into Word and PowerPoint.
- To generate AI art within Word, users need to pay an additional $20 per user per month for the Copilot Pro subscription.
- Microsoft 365 subscription is required, with a Personal user paying $6.99 per month or $69.99 annually, while Family users pay $9.99 per month or $99.99 annually.


[Microsoft has let its AI-powered Designer app out of preview mode - Mashable](https://sea.mashable.com/tech/33474/microsoft-has-let-its-ai-powered-designer-app-out-of-preview-mode)

Topics: Art, Design, Big Tech, Microsoft, Products 

- Microsoft's AI-powered Designer app is now available for free to all users with a Microsoft account, allowing for template-based image editing, creation of greeting cards, invitations, and basic designs.
- The app features a Restyle Image function for creating AI-powered avatars in different styles like pop art and cubism, similar to offerings from other tech giants.
- Users can integrate Designer with Microsoft 365 software, access Boosts for faster AI-powered image editing, and Microsoft emphasizes responsible AI use in its blog post.


[Microsofts Designer AI image generator comes to iOS, Android with new features - VentureBeat](https://venturebeat.com/ai/microsofts-designer-ai-image-generator-comes-to-ios-android-with-new-features/)

Topics: Android, Art, Design, Big Tech, Gen AI, Machine Learning, Microsoft, Products 

- Microsoft's Designer AI image generator, powered by OpenAI's DALL-E 3 image generation model, is now available as a free app on Android, iOS, and Windows devices.
- New capabilities for Designer include the ability to restyle existing images, prompts to help users start on a blank canvas, and categories like avatars, emojis, stickers, greeting cards, and invitations.
- Microsoft plans to integrate Designer into the Windows 11 Photos app and offers a Copilot Pro subscription for advanced features like 100 daily boosts for image creation and editing.


[Microsoft brings controversial AI app to iPhone and Android devices - Laptop Mag](https://www.laptopmag.com/software/creative-media-apps/microsoft-brings-controversial-ai-app-to-iphone-and-android-devices)

Topics: Android, Big Tech, Copilot, Ethics, Facial Recognition, Language Models, Machine Learning, Microsoft, Privacy, Products 

- Microsoft announced the release of a free mobile app for Designer, offering AI-enhanced graphic design and image editing capabilities similar to Adobe Express or Canva.
- The app integrates seamlessly with Microsoft Word, PowerPoint, and Photos, supports over 80 languages, and is available on Windows, iOS, and Android devices.
- Despite a controversial history involving the generation of inappropriate content, Microsoft has enhanced security measures and now offers Designer as a powerful and intuitive graphic design tool on multiple platforms.


[Microsoft releases iOS and Android apps for Designer, its AI-powered Canva competitor - Engadget](https://www.engadget.com/microsoft-releases-ios-and-android-apps-for-designer-its-ai-powered-canva-competitor-203028855.html)

Topics: Art, Design, Big Tech, Machine Learning, Microsoft, Products 

- Microsoft has officially released its AI-powered Designer platform for AI image generation, available on the web, as a mobile app for iOS and Android, and as a Windows app, offering templates and customization options.
- Designer integrates with Microsoft's Copilot AI chatbot, allowing for easy linkage with Microsoft Word and PowerPoint projects, though full utilization requires a Copilot Pro subscription.
- Designer provides a user experience similar to Canva, with integration with OpenAI's image generator DALL-E and Copilot's existing services like DALL-E 3 integration and ChatGPT 4 Turbo.


[Simplify Your Business Branding and Content With AI MagicX - TechRepublic](https://www.techrepublic.com/article/ai-magicx-lifetime-subscription/)

Topics: Gen AI, Machine Learning 

- AI MagicX is a platform that offers AI tools for content creation, logo design, creating art, generating articles, and enhancing customer engagement.
- For a discounted price of $99.99 (regularly $972), users can get a lifetime subscription to AI MagicX, enabling them to create professional designs without prior skills, saving time and resources.
- This all-in-one software can help small business owners streamline their workflow, create brand stories, and speed up the document editing process, making it a cost-effective solution for branding and content creation needs.


[Samsung's AI Makes Images That Are Uncanny Real - Analytics Insight](https://www.analyticsinsight.net/ampstories/artificial-intelligence/samsungs-ai-makes-images-that-are-uncanny-real)

Topics: Big Tech, Korea, Machine Learning, Products, Samsung, Science 

- Samsung's latest AI technology can transform sketches into hyper-realistic images with uncanny realism.
- The new Z Fold 6 integrates this AI, allowing users to turn rough sketches into detailed visuals in seconds.
- Users can sketch directly on the Z Fold 6 and witness their drawings come to life with incredible accuracy.


[Artificial Agency raises $16M to bring AI-powered behavior to games - VentureBeat](https://venturebeat.com/ai/artificial-agency-raises-16m-to-bring-ai-powered-behavior-to-games/)

Topics: Deals, Funding, Gaming, Gen AI, Machine Learning, Rag, Reinforcement Learning, Venture Capital 

- Artificial Agency, an AI startup, raised $16 million in funding for its flagship product, an AI-powered behavior engine that allows game developers to incorporate generative AI seamlessly into various game aspects, promising more personalized and engaging gaming experiences.
- The engine enables game developers to add improvisation to scripted interactions, create full artificial players, and control game elements to enhance player immersion and entertainment, with the aim of reducing churn and keeping players engaged.
- The company, based in Edmonton, is collaborating with AAA studios to develop the engine, which is expected to be widely available for studios in 2025, offering a customizable and extensible solution for game design and development.


[Microsoft SpreadsheetLLM AI specifically designed for spreadsheets - Geeky Gadgets](https://www.geeky-gadgets.com/spreadsheetllm-ai/)

Topics: Big Tech, Data, Deals, Llms, Machine Learning, Meta, Microsoft, Products, Review 

- Microsoft introduces SpreadsheetLLM AI aimed at simplifying and enhancing data tasks in spreadsheets.
- SpreadsheetLLM combines large language models with structured spreadsheet data to automate tasks, democratize data insights, and empower decision-making.
- The AI model optimizes performance, preserves relationships in spreadsheets, achieves high accuracy, and represents Microsoft's broader investment in enterprise AI technologies.


[From reality to fantasy: Live2Diff AI brings instant video stylization to life - VentureBeat](https://venturebeat.com/ai/from-reality-to-fantasy-live2diff-ai-brings-instant-video-stylization-to-life/)

Topics: Art, Design, Gen AI, Machine Learning, Products, Science, Streaming 

- Live2Diff AI system can stylize live video streams in near real-time, processing at 16 frames per second without needing access to future frames.
- The technology enables real-time video style transfer without future frame data, offering possibilities for entertainment, augmented reality experiences, and creative expression.
- Live2Diff's implications span various industries, including entertainment, augmented reality, and virtual reality, with the potential for applications in live events, gaming, tourism, and professional fields.


# AI Chip Demand Surge 

[TSMC smashed second quarter expectations due to AI chip demand - Quartz](https://qz.com/tsmc-second-quarter-earnings-ai-chip-nvidia-apple-trade-1851597577)

Topics: Economics, Expectations, Gen AI, Machine Learning, Manufacturing, Products, Semiconductor Chips, Stocks, Taiwan, Tsmc 

- TSMC exceeded Q2 earnings expectations with a revenue of $20.82 billion, up 40% from the previous year.
- The company reported a quarterly profit of NT$247.85 billion ($7.6 billion), a 36% increase from last year, driven by high demand for AI chips from Nvidia and Apple.
- Despite the strong performance, TSMC faces supply shortages due to high demand for AI capabilities, with the CEO expecting production to catch up by 2025 or 2026.


[TSMC reports Q2 revenue up 40.1% YoY to $20.82B and net income up 36.3% YoY to $7.62B, both above estimates, as demand for advanced AI chips continue to surge - CNBC](https://www.cnbc.com/2024/07/18/tsmc-q2-2024-earnings.html)

Topics: Economics, Manufacturing, Products, Semiconductor Chips, Taiwan, Tsmc 

- TSMC exceeded revenue and profit expectations in Q2 2024 due to high demand for advanced chips used in AI applications.
- The company reported significant revenue and income growth compared to the previous year, with positive forecasts for the future supported by strong demand for 3-nanometer and 5-nanometer technologies.
- The firm expects strong performance in Q3 2024 supported by demand for smartphones and AI-related technology, aiming for continued growth throughout the year with capital investments focused on advanced technologies.


[TSMC Hikes Revenue Outlook to Reflect Heated AI Demand - Yahoo Finance](https://finance.yahoo.com/news/tsmc-profit-surpasses-estimates-ai-053450831.html)

Topics: Machine Learning, Manufacturing, Products, Semiconductor Chips, Taiwan, Tsmc 

- TSMC raised its revenue outlook for 2024 due to strong AI demand, with sales expected to grow more than previously guided.
- The company's confidence in the high AI spending is reflected in its revenue forecast and capital spending outlook, driven by global AI infrastructure investments.
- TSMC's shift in focus towards AI is paying off, with net income rising, high-performance computing contributing over half of revenue, and expectations to meet or exceed gross margin targets.


[More Than Half of TSMCs Sales Are Now High-End Chips Like AI - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-18/more-than-half-of-tsmc-s-sales-are-now-high-end-chips-like-ai)

Topics: Economics, Machine Learning, Manufacturing, Products, Semiconductor Chips, Stocks, Taiwan, Tsmc 

- TSMC now generates over half of its revenue from high-performance computing (HPC) division, with a focus on AI accelerators and processors.
- This shift is significant as the company's key sales driver has moved from smartphones to artificial intelligence technologies.
- TSMC's growth is driven by the AI boom, with HPC sales growing by 28% sequentially and a positive outlook for future sales growth due to the increasing demand for AI chips in various devices.


[TSMC rides AI demand to raise revenue forecast, says no to US joint venture - Reuters](https://www.reuters.com/technology/tsmc-set-report-strong-profit-stock-pressured-by-trump-comments-2024-07-18/)

Topics: Deals, Economics, Manufacturing, Semiconductor Chips, Taiwan, Tsmc 

- TSMC raised its full-year revenue forecast due to high demand for AI chips, benefiting from the global AI boom amid pandemic-led electronics demand slowdown.
- The company plans to continue expanding overseas, including in the U.S., Japan, and Germany, rejecting the possibility of a joint venture in the U.S. despite potential political pressure.
- TSMC expects tight supply for leading-edge nodes like 3nm and 5nm, with strong smartphone and AI-related demand supporting future business growth.


[TSMC boss predicts AI chip shortage through 2025, says Trump comments don't change his strategy - The Register](https://go.theregister.com/feed/www.theregister.com/2024/07/18/tsmc_ceo_predicts_ai_chip/)

Topics: Machine Learning, Manufacturing, Semiconductor Chips, Strategy, Taiwan, Tsmc 

- TSMC CEO predicts AI chip shortage to continue until 2025 or 2026, focusing on balancing supply and demand for advanced chips.
- Overseas expansion plans to address the shortage include building plants in Arizona and Kumamoto, Japan, with additional future expansion in Europe.
- TSMC's efforts to increase CoWoS packaging capacity as a solution to manufacturing holdups, with plans to mass-produce 2 nm chips in the second half of 2025.


[TSMC raises hopes of a prolonged AI boom - Financial Times](https://www.ft.com/content/24c8ec54-5a9d-4e0a-b7c7-6f01fa53d049)

Topics: Semiconductor Chips, Taiwan, Tsmc 

- TSMC raises hopes for a prolonged AI boom
- Complete digital access to quality FT journalism with expert analysis
- Includes global news, expert opinion, curated newsletters, exclusive analysis


[San Jose-based DreamBig, which develops chiplet platforms and was founded in 2019, raised a $75M equity round co-led by Samsung Catalyst Fund and Sehat Sutardja - Crunchbase News](https://news.crunchbase.com/semiconductors-and-5g/dreambig-semiconductor-chip-startup-nvidia-halo/)

Topics: Deals, Funding, Korea, Manufacturing, Products, Samsung, Semiconductor Chips, Venture Capital 

- DreamBig Semiconductor raised $75 million in an equity round with investment from Samsung Catalyst Fund and specializes in chiplet platforms for applications like generative AI.
- The semiconductor industry is benefiting from the current AI trend, with other notable funding rounds including Halo Industries, Black Semiconductor, Etched.ai, and Celestial AI.
- In 2024, VC-backed chip startups have raised nearly $5.7 billion in 210 deals, showing strong venture funding support in the industry.


[Smartphone growth continues to surge in Q2 2024 results thanks to AI - Android Central](https://www.androidcentral.com/phones/smartphone-growth-continues-to-surge-in-q2-2024-results-thanks-to-ai)

Topics: Big Tech, Products 

- The smartphone market saw a rebound in the second quarter of 2024, with IDC and Canalys reporting overall growth.
- Samsung led in shipments in Q2 2024, with 19.8% market share, partially attributed to its AI push.
- AI features are driving smartphone market excitement and growth, with Samsung's Galaxy AI suite being a key factor in its success.


# Text-To-Image Diffusion Models 

[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Related Work - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-related-work)

Topics: Text-To-Image Diffusion Models 

- Text-to-image diffusion models are powerful tools for generating high-quality images from text descriptions, commonly used for guided image synthesis and editing tasks.
- Diffusion models have been applied in various tasks beyond image generation, including video editing, 3D synthesis, and motion generation.
- The discussed method aims to generate consistent and diverse images of the same character solely based on a text description, avoiding the need for input images and manual interventions in the generation process.


[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Implementation Details - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-implementation-details)

Topics: Machine Learning, Text-To-Image Diffusion Models 

- Researchers introduce an iterative procedure to identify a coherent set of images sharing a similar identity
- Implementation details for the text-to-image model method and baselines used are provided in the appendices
- Evaluations through prompts generation and user studies were conducted to assess prompt similarity and identity consistency of the method and baselines


[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: References - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-references)

Topics: Text-To-Image Diffusion Models 

- Study introduces an iterative procedure to identify images sharing similar identities
- Authors include researchers from Google, Hebrew University of Jerusalem, Tel Aviv University, and Reichman University
- Several references to related works in text-to-image diffusion models and personalization techniques


[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Additional Experiments - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-additional-experiments)

Topics: Machine Learning, OpenAI, Science, Text-To-Image Diffusion Models 

- Researchers introduce an iterative procedure to identify a coherent set of images sharing a similar identity in text-to-image diffusion models.
- Additional experiments demonstrate the non-deterministic nature of the method and comparisons against baselines and feature extractors.
- The method is able to generate a wide range of consistent characters and life stories, showcasing its versatility and effectiveness in character generation.


[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Experiments - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-experiments)

Topics: Cohere, Language Models, Machine Learning, OpenAI, Science, Text-To-Image Diffusion Models 

- Researchers introduce an iterative procedure to identify a coherent set of images sharing a similar identity
- A comparison is made between the proposed method and related personalization techniques in generating consistent characters 
- An ablation study highlights the significance of key components for achieving a consistent identity in the final result


[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Societal Impact - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-societal-impact)

Topics: Machine Learning, Society, Culture, Text-To-Image Diffusion Models 

- Introduction of an iterative procedure to identify a coherent set of images with consistent characters in text-to-image diffusion models for various creative applications and democratizing character design
- Potential impact of technology for empowering storytellers, reducing advertising costs, democratizing character design, while also highlighting the risk of misuse for deceptive purposes requiring the development of content detection systems
- Demonstration of consistent generation of various objects beyond human characters, such as paintings, animations, stickers, and vector art, showcasing the method's applicability for story generation and comparison to baselines and feature extractors.


[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Abstract and Introduction - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-abstract-and-introduction)

Topics: OpenAI, Text-To-Image Diffusion Models 

- Researchers introduce an iterative procedure for consistent character generation in text-to-image models, improving identity consistency without the need for pre-existing images.
- Maintaining consistency in generated visual content is crucial for various creative endeavors like storytelling, branding, and communication.
- The proposed method for consistent character generation is fully automated, based on clustering images with shared characteristics to extract a representation that enhances identity coherence.


[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: Limitations and Conclusions - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-limitations-and-conclusions)

Topics: Machine Learning, OpenAI, Review, Science, Text-To-Image Diffusion Models 

- Researchers introduced an iterative procedure to identify coherent sets of images with consistent identity but faced limitations like inconsistent identity, supporting characters, and spurious attributes.
- The study took about 20 minutes per iteration due to generating a large number of images, highlighting the significant computational cost of the method.
- The paper concluded by presenting the first fully-automated solution for consistent character generation, aiming to impact various sectors like education, entertainment, and advertising.


[TTT models might be the next frontier in generative AI - TechCrunch](https://techcrunch.com/2024/07/17/ttt-models-might-be-the-next-frontier-in-generative-ai/)

Topics: Art, Design, Artificial General Intelligence, Cohere, Data, Gen AI, Generative AI, Language Models, Machine Learning, Nvidia, OpenAI, Science 

- The hunt is on for new architectures in generative AI as transformers face computation-related roadblocks due to inefficient data processing and analysis.
- Test-time training (TTT) models, developed by a research team from Stanford, UC San Diego, UC Berkeley, and Meta, offer a promising alternative to transformers by processing more data efficiently without high compute power demands.
- TTT models utilize a machine learning model within a model to encode data into representative variables, making them highly performant and capable of processing billions of data pieces across various formats, potentially revolutionizing generative AI.


# ChatGPT in AI Dominance 

[ChatGPT has eyes - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e60f7p/chatgpt_has_eyes/)

Topics: Chatgpt, OpenAI 

- Subreddit dedicated to discussing ChatGPT and AI, highlighting use cases and features
- Discussion on interrupting ChatGPT and its behavior in response to interruptions
- Expressions of skepticism and frustration towards OpenAI's demos and releases


[The AI doctor will see you now: ChatGPT dominates medical exam - CNBC](https://www.cnbc.com/video/2024/07/18/the-ai-doctor-will-see-you-now-chatgpt-dominates-medical-exam.html)

Topics: Chatgpt, Language Models, Machine Learning, OpenAI 

- Former FDA Commissioner Dr. Scott Gottlieb discusses AI large language models excelling in a medical exam for doctors
- ChatGPT stands out in the medical exam for a medical license
- AI-powered healthcare is the future of healthcare


[The Fastest Way to Lose a Court Case? Use ChatGPT - The Walrus](https://thewalrus.ca/the-fastest-way-to-lose-a-court-case-use-chatgpt/)

Topics: Chatgpt, Code Assistants, Cohere, Gen AI, Language Models, Llms, OpenAI 

- Lawyers are turning to AI, such as ChatGPT, for legal research to save time but face risks of false or inaccurate information due to the model's nature.
- Despite concerns about AI-generated errors, lawyers facing burnout and heavy workloads are still tempted to use AI for its convenience.
- Law societies are setting guidelines for AI use in legal practice, emphasizing responsible implementation, requiring consent from clients, and offering training courses for lawyers to use AI effectively.


[TTT models might be the next frontier in generative AI - TechCrunch](https://techcrunch.com/2024/07/17/ttt-models-might-be-the-next-frontier-in-generative-ai/)

Topics: Art, Design, Artificial General Intelligence, Cohere, Data, Gen AI, Generative AI, Language Models, Machine Learning, Nvidia, OpenAI, Science 

- The hunt is on for new architectures in generative AI as transformers face computation-related roadblocks due to inefficient data processing and analysis.
- Test-time training (TTT) models, developed by a research team from Stanford, UC San Diego, UC Berkeley, and Meta, offer a promising alternative to transformers by processing more data efficiently without high compute power demands.
- TTT models utilize a machine learning model within a model to encode data into representative variables, making them highly performant and capable of processing billions of data pieces across various formats, potentially revolutionizing generative AI.


[OpenAI used a game to help AI models explain themselves better - VentureBeat](https://venturebeat.com/ai/openai-used-a-game-to-help-ai-models-explain-themselves-better/)

Topics: AI Models, Artificial General Intelligence, Big Tech, Gen AI, Language Models, Machine Learning, OpenAI, Reinforcement Learning, Science 

- OpenAI has developed an algorithm to improve the legibility and explainability of large language models (LLMs) like GPT-4, aiming to foster trust in AI systems, especially in critical fields.
- The algorithm is based on a Prover-Verifier Game where a powerful "prover" AI model tries to persuade a less powerful "verifier" model in providing correct answers, enhancing the models' ability to explain their reasoning.
- By engaging the models in rounds of the game and training them to be more transparent and accurate in their responses, OpenAI's method aims to enhance the correctness and understandability of AI outputs, crucial for trust and safety in real-world applications.


[Gigabyte releases AI software to help train your own AI  AI TOP utility taps Gigabyte motherboards, GPUs, SSDs, and power supplies to fine-tune local AI model training - Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai)

Topics: AI Model, Data, Gen AI, Machine Learning, Meta, Products, Taiwan, Testing 

- Gigabyte released AI TOP utility for local AI model training with up to 236B parameters, previously requiring remote data centers.
- The utility works with Gigabyte hardware, including motherboards, SSDs, GPUs, and power supplies, and supports over 70 LLM models on Hugging Face.
- Users can fine-tune AI models locally for enhanced data privacy, without needing AI programming skills, through a user-friendly graphical interface provided by Gigabyte.


[The AI Revolution Is Meeting Reality | Commentary - The Wrap](https://www.thewrap.com/ai-revolution-is-meeting-reality-commentary-alex-kantrowitz/)

Topics: Machine Learning, Opinion 

- Amazon's AI team and partners are optimistic but acknowledge the challenges of building effective generative applications.
- Companies are finding that working with generative AI is difficult due to expensive AI models, data challenges, and complexities in change management.
- According to a survey, only 21% of companies have generative AI in production, while the rest are either piloting or exploring the technology.


[The Executive Playbook For Generative AI: Turning Potential Into Business Value - Forbes](https://www.forbes.com/sites/forbestechcouncil/2024/07/18/the-executive-playbook-for-generative-ai-turning-potential-into-business-value/)

Topics: Gen AI, Generative AI 

- Focus on implementing AI to automate workflows and drive immediate value while mitigating risks and ensuring compliance.
- Move quickly but be methodical when implementing GenAI to avoid disappointing results and create meaningful value through pilot AI workflows.
- Create and deploy a unified implementation plan aligned with internal end users' roles to drive near-term value and efficiency improvements in business workflows.


[When To Use Large Language ModelsAnd When Not To - Forbes](https://www.forbes.com/sites/forbestechcouncil/2024/07/18/when-to-use-large-language-models-and-when-not-to/)

Topics: Chatgpt, Language Models, Large Language Models, Llms, Machine Learning, OpenAI 

- There are situations where integrating large language models (LLMs) like GPT-4o can be beneficial for tasks that don't require perfection, with examples in coding and data handling.
- Keeping humans in the loop is essential for scenarios where LLMs are used but may not be perfect, ensuring validation and human intervention where required.
- Questions to consider whether not to use LLMs include the impact on end users, the need for perfection in outcomes, and understanding the strengths and limitations of LLMs for specific tasks.


[OpenAI unveils GPT-4o mini, calling it the most capable and cost-efficient small model available, and plans to integrate image, video, and audio into it later - CNBC](https://www.cnbc.com/2024/07/18/openai-4o-mini-model-announced.html)

Topics: Big Tech, Gen AI, Language Models, Machine Learning, OpenAI, Products, Science 

- OpenAI launched a new AI model, "GPT-4o mini," to expand the use of its chatbot.
- The mini model is a smaller version of the powerful GPT-4o model, offering improved audio, video, and text capabilities.
- The release is part of OpenAI's focus on multimodality, aiming to provide various types of AI-generated media within one tool like ChatGPT.




In [114]:
docid_list

[1,
 155,
 134,
 100,
 93,
 3,
 92,
 78,
 107,
 160,
 2,
 0,
 3,
 59,
 12,
 13,
 56,
 1,
 67,
 109,
 30,
 58,
 4,
 6,
 35,
 36,
 34,
 127,
 92,
 5,
 38,
 99,
 122,
 7,
 9,
 11,
 67,
 83,
 14,
 19,
 20,
 131,
 8,
 160,
 14,
 19,
 83,
 18,
 20,
 11,
 7,
 9,
 12,
 10,
 13,
 101,
 160,
 83,
 14,
 20,
 19,
 18,
 83,
 11,
 7,
 177,
 9,
 67,
 21,
 22,
 53,
 23,
 136,
 159,
 112,
 169,
 190,
 147,
 30,
 29,
 33,
 31,
 32,
 162,
 49,
 95,
 67,
 46,
 43,
 45,
 42,
 44,
 160,
 8,
 84,
 185,
 17,
 68,
 49,
 46,
 95,
 50,
 48,
 81,
 162,
 158,
 30,
 63,
 94,
 159,
 64,
 131,
 46,
 49,
 182,
 50,
 98,
 61,
 62,
 56,
 55,
 59,
 58,
 57,
 165,
 112,
 148,
 191,
 189,
 106,
 124,
 187,
 77,
 115,
 152,
 98,
 116,
 82,
 158,
 93,
 67,
 35,
 172,
 1,
 78,
 164,
 3,
 160,
 114,
 92,
 93,
 163,
 108,
 6,
 36,
 4,
 35,
 34,
 5,
 127,
 38,
 86,
 7,
 20,
 9,
 14,
 19,
 11,
 83,
 67,
 112,
 113,
 15,
 17,
 192,
 16,
 63,
 77,
 93,
 45,
 159,
 161,
 7,
 20,
 9,
 14,
 19,
 11,
 83,
 67,
 112,
 113,
 136,
 28,
 2

In [115]:
# write sections individually

mail_md_str = ""

for current_topic, cat in enumerate(my_cats):

    section_prompt = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input.

Input:
I will provide a markdown list of today's news articles on the topic: {my_cats[current_topic]}.
The input will be in the format
[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Instructions:

Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Write an engaging summary consisting of a title and at least 1 and no more than 5 bullet points.
Use as few bullet points as you need to provide the most significant facts.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.
DO NOT REPEAT LINKS FROM PREVIOUS BULLET POINTS.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.

Please check carefully that you only use information provided in the following input, and that any bullet point
does not repeat information or links prevously provided.

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging title

- bullet point a - [site name a](site url a)
- bullet point b - [site name b ](site url b)

Input:

{doc_list[current_topic]}
"""

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
                  {"role": "user", "content": section_prompt
                  }],
        n=1,   
        temperature=0.2
    )

    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\$")
    mail_md_str += response_str + " \n\n"
    display(Markdown(response_str))


2024-07-18 11:11:54,661 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI in Drug Discovery: Today's Highlights

- Artificial Intelligence is revolutionizing drug development, particularly in the early stages of drug discovery - [Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)
- ICL and Lavie Bio have used AI to identify novel microbes for row crops, aiming to increase yields by 5% to 10% - [PR Newswire](https://www.prnewswire.com/news-releases/icl-and-lavie-bio-leverage-artificial-intelligence-to-make-significant-advancement-in-development-of-yield-increasing-bio-stimulants-302199316.html)
- nference and Takeda have established a research alliance to leverage AI for optimizing patient care in inflammatory bowel disease - [Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)
- AI has been developed to predict tipping points in complex systems before they occur, offering solutions for real-world issues - [The Economist](https://www.economist.com/science-and-technology/2024/07/17/ai-can-predict-tipping-points-before-they-happen)
- Thoughtful AI raised \$20 million in Series A funding to launch AI agents for healthcare clinics, focusing on automating revenue cycle management tasks - [Business Insider](https://www.businessinsider.com/pitch-deck-thoughtful-ai-20-million-drive-capital-series-a-2024-7)

2024-07-18 11:11:57,041 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Advances in Machine Learning Propel Structural Biology

- Structural biologist Lucas Farnung uses AI to predict protein interactions, accelerating research in gene expression regulation - [Harvard Medical School](https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology)
- Ex-Meta scientists developed the ESM3 model, an AI capable of creating proteins not found in nature, revolutionizing synthetic biology - [Live Science](https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature)

2024-07-18 11:11:58,926 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

# AI Chip Demand Surge

- TSMC exceeded Q2 earnings expectations with a revenue of \$20.82 billion, up 40% from the previous year, driven by high demand for AI chips from Nvidia and Apple - [Quartz](https://qz.com/tsmc-second-quarter-earnings-ai-chip-nvidia-apple-trade-1851597577)
- TSMC raised its revenue outlook for 2024 due to strong AI demand, with sales expected to grow more than previously guided - [Yahoo Finance](https://finance.yahoo.com/news/tsmc-profit-surpasses-estimates-ai-053450831.html)
- TSMC now generates over half of its revenue from high-performance computing (HPC) division, with a focus on AI accelerators and processors - [Bloomberg](https://www.bloomberg.com/news/articles/2024-07-18/more-than-half-of-tsmc-s-sales-are-now-high-end-chips-like-ai)
- DreamBig Semiconductor raised \$75 million in an equity round with investment from Samsung Catalyst Fund and specializes in chiplet platforms for applications like generative AI - [Crunchbase News](https://news.crunchbase.com/semiconductors-and-5g/dreambig-semiconductor-chip-startup-nvidia-halo/)
- The smartphone market saw a rebound in the second quarter of 2024, with IDC and Canalys reporting overall growth, driven by AI features - [Android Central](https://www.androidcentral.com/phones/smartphone-growth-continues-to-surge-in-q2-2024-results-thanks-to-ai)

2024-07-18 11:15:02,286 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Advances in Text-To-Image Diffusion Models

- Researchers introduce an iterative procedure to generate consistent characters in text-to-image diffusion models, avoiding the need for input images - [Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-related-work)
- Implementation details and evaluations of the text-to-image model method are provided, assessing prompt similarity and identity consistency - [Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-implementation-details)
- Additional experiments demonstrate the method's versatility in generating a wide range of consistent characters and life stories - [Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-additional-experiments)
- The societal impact of this technology includes democratizing character design and reducing advertising costs, while also posing risks of misuse - [Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-societal-impact)
- Test-time training (TTT) models, developed by top universities and Meta, offer a promising alternative to transformers in generative AI by processing data more efficiently - [TechCrunch](https://techcrunch.com/2024/07/17/ttt-models-might-be-the-next-frontier-in-generative-ai/)

2024-07-18 11:15:12,969 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# ChatGPT in AI Dominance

- A subreddit on Reddit discusses ChatGPT's use cases, interruptions, and skepticism towards OpenAI's demos - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1e60f7p/chatgpt_has_eyes/).
- Former FDA Commissioner Dr. Scott Gottlieb highlights ChatGPT's success in a medical exam for doctors on CNBC - [CNBC](https://www.cnbc.com/video/2024/07/18/the-ai-doctor-will-see-you-now-chatgpt-dominates-medical-exam.html).
- Lawyers are using ChatGPT for legal research despite risks, with law societies setting guidelines for responsible AI use - [The Walrus](https://thewalrus.ca/the-fastest-way-to-lose-a-court-case-use-chatgpt/).
- TechCrunch reports on TTT models as a promising alternative to transformers in generative AI, developed by a team from Stanford, UC San Diego, UC Berkeley, and Meta - [TechCrunch](https://techcrunch.com/2024/07/17/ttt-models-might-be-the-next-frontier-in-generative-ai/).
- OpenAI's new algorithm improves the explainability of large language models through a Prover-Verifier Game, enhancing trust in AI systems - [VentureBeat](https://venturebeat.com/ai/openai-used-a-game-to-help-ai-models-explain-themselves-better/).
- Gigabyte's AI TOP utility allows local AI model training with up to 236B parameters, supporting over 70 LLM models on Hugging Face - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gigabyte-releases-ai-software-to-help-train-your-own-ai).
- Amazon's AI team acknowledges challenges in generative AI, with only 21% of companies having it in production - [The Wrap](https://www.thewrap.com/ai-revolution-is-meeting-reality-commentary-alex-kantrowitz/).
- Forbes advises on implementing GenAI to automate workflows and drive value while mitigating risks and ensuring compliance - [Forbes](https://www.forbes.com/sites/forbestechcouncil/2024/07/18/the-executive-playbook-for-generative-ai-turning-potential-into-business-value/).
- OpenAI's new "GPT-4o mini" model offers improved audio, video, and text capabilities, focusing on multimodality - [CNBC](https://www.cnbc.com/2024/07/18/openai-4o-mini-model-announced.html).

In [116]:
print(mail_md_str)

# AI in Drug Discovery: Today's Highlights

- Artificial Intelligence is revolutionizing drug development, particularly in the early stages of drug discovery - [Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)
- ICL and Lavie Bio have used AI to identify novel microbes for row crops, aiming to increase yields by 5% to 10% - [PR Newswire](https://www.prnewswire.com/news-releases/icl-and-lavie-bio-leverage-artificial-intelligence-to-make-significant-advancement-in-development-of-yield-increasing-bio-stimulants-302199316.html)
- nference and Takeda have established a research alliance to leverage AI for optimizing patient care in inflammatory bowel disease - [Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)
- AI has been developed to predict tipping points in complex systems before they oc

In [117]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(mail_md_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-18 11:15:13,030 - AInewsbot - INFO - Sending full summary email 


# Final Summary

In [118]:
# summarize by just giving selected stories in semantic order and hinting how to write it
AIdf.loc[AIdf['id'].isin(set(docid_list))]

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,'ChatGPT moment for biology': Ex-Meta scientis...,https://news.google.com/articles/CBMioQFodHRwc...,True,2024-07-18,https://www.livescience.com/technology/artific...,www.livescience.com,Live Science,"Innovation, Machine Learning, Meta, Science",'ChatGPT moment for biology': Ex-Meta scientis...,66,0,AI in Healthcare & Precision Medicine,htmldata/_ChatGPT_moment_for_biology___Ex-Meta...
1,1,Feedly AI,AI & Drug Discovery With Harren Jhoti: Vanguar...,https://www.bloomberg.com/news/audio/2024-07-1...,True,2024-07-18,https://www.bloomberg.com/news/audio/2024-07-1...,www.bloomberg.com,Bloomberg,"Healthcare, Machine Learning, Science",AI & Drug Discovery With Harren Jhoti: Vanguar...,94,0,AI in Healthcare & Precision Medicine,htmldata/AI___Drug_Discovery_With_Harren_Jhoti...
2,2,Google News,How Machine Learning Is Propelling Structural ...,https://news.google.com/articles/CBMiT2h0dHBzO...,True,2024-07-18,https://hms.harvard.edu/news/how-machine-learn...,hms.harvard.edu,Harvard Medical School,"Data, Machine Learning, Science",How Machine Learning Is Propelling Structural ...,156,0,AI in Healthcare & Precision Medicine,htmldata/How_Machine_Learning_Is_Propelling_St...
3,3,Google News,nference Establishes AI Initiative with Takeda...,https://news.google.com/articles/CBMiowFodHRwc...,True,2024-07-18,https://www.businesswire.com/news/home/2024071...,www.businesswire.com,Business Wire,"Deals, Health, Fitness, Healthcare, Machine Le...",nference Establishes AI Initiative with Takeda...,168,0,AI in Healthcare & Precision Medicine,htmldata/nference_Establishes_AI_Initiative_wi...
4,4,Google News,56-gram drone gets ant-inspired AI eyes to nav...,https://news.google.com/articles/CBMiTmh0dHBzO...,True,2024-07-18,https://interestingengineering.com/innovation/...,interestingengineering.com,Interesting Engineering,"Autonomous Vehicles, Baidu, Machine Learning, ...",56-gram drone gets ant-inspired AI eyes to nav...,22,1,Autonomous Drones,htmldata/56-gram_drone_gets_ant-inspired_AI_ey...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,190,Feedly AI,Mastering Photoshops Generative AI for Perfect...,https://fstoppers.com/photoshop/mastering-phot...,True,2024-07-18,https://fstoppers.com/photoshop/mastering-phot...,fstoppers.com,Fstoppers,"Art, Design, Gen AI, Generative AI, Machine Le...",Mastering Photoshops Generative AI for Perfect...,185,999,,htmldata/Mastering_Photoshops_Generative_AI_fo...
191,191,Google News,Research Confirms Google AIO Keyword Trends,https://news.google.com/articles/CBMiV2h0dHBzO...,True,2024-07-18,https://www.searchenginejournal.com/research-c...,www.searchenginejournal.com,Search Engine Journal,"Big Tech, Data, Google, Machine Learning, Rese...",Research Confirms Google AIO Keyword Trends (T...,188,999,,htmldata/Research_Confirms_Google_AIO_Keyword_...
192,192,Feedly AI,I made a personalized workout plan with Claude...,https://www.tomsguide.com/claude-ai-can-improv...,True,2024-07-18,https://www.tomsguide.com/claude-ai-can-improv...,www.tomsguide.com,Tom's Guide,"Claude, Gen AI, Health, Fitness, Machine Learn...",I made a personalized workout plan with Claude...,189,999,,htmldata/I_made_a_personalized_workout_plan_wi...
194,194,Google News,Maths test stumps AI models: which number is b...,https://news.google.com/articles/CBMicGh0dHBzO...,True,2024-07-18,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"AI Models, Cognitive Science, Education, Gen A...",Maths test stumps AI models: which number is b...,192,999,,htmldata/Maths_test_stumps_AI_models__which_nu...


In [119]:
docs[16].page_content

'[Anthropics Claude 3.5 Sonnet ranks number 1 for business and finance in S&P AI Benchmarks by Kensho - Amazon Web Services](https://aws.amazon.com/blogs/machine-learning/anthropic-claude-3-5-sonnet-ranks-number-1-for-business-and-finance-in-sp-ai-benchmarks-by-kensho/)\n\nTopics: Anthropic, Claude, Finance, Fintech, Machine Learning, Products, Stocks \n\n- Anthropic’s Claude 3.5 Sonnet is ranked number 1 for business and finance in S&P AI Benchmarks by Kensho, showcasing its capabilities in challenging financial and business tasks.\n- S&P AI Benchmarks by Kensho focus on evaluating language models for finance and business based on domain knowledge, quantity extraction, and quantitative reasoning, offering a leaderboard for model performance comparison.\n- Anthropic’s Claude 3.5 Sonnet excels in tasks such as quantitative reasoning, quantity extraction, and domain knowledge, demonstrating its strength in generating insights and calculations in the financial domain.\n\n'

In [120]:
# make text for email and also collect data for vector store
markdown_str = ''
print()

for i, row in enumerate(AIdf.loc[AIdf['id'].isin(set(docid_list))].itertuples()):
    mdstr = docs[row.id].page_content
    display(Markdown(mdstr.replace('$', '\\$')))
    markdown_str += mdstr
    

['ChatGPT moment for biology': Ex-Meta scientists develop AI model that creates proteins 'not found in nature' - Live Science](https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature)

Topics: Innovation, Machine Learning, Meta, Science 

- Ex-Meta scientists developed the ESM3 model, an AI model that can create proteins not found in nature, opening up new possibilities for synthetic biology.
- The ESM3 model is similar to OpenAI's GPT-4 and is trained on 2.78 billion proteins, enabling it to predict missing information about protein sequences, structures, and functions.
- ESM3 can generate new proteins with specific functions, such as creating a novel green fluorescent protein called "esmGPF," which would take 500 million years of evolution to achieve naturally.



[AI & Drug Discovery With Harren Jhoti: Vanguards of Health Care - Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)

Topics: Healthcare, Machine Learning, Science 

- Artificial Intelligence is revolutionizing drug development, particularly in the early stages of drug discovery.
- Discussion between Sam Fazeli and Harren Jhoti explores the role of machine learning, AI, and AlphaFold 3 in drug discovery.
- Astex, co-founded by Harren Jhoti in 1999, has made significant contributions to oncology drugs through fragment-based drug discovery, leading to the development of drugs like Kisqali for breast cancer.



[How Machine Learning Is Propelling Structural Biology - Harvard Medical School](https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology)

Topics: Data, Machine Learning, Science 

- Structural biologist Lucas Farnung studies how molecular machines regulate gene expression during transcription.
- Farnung uses visualization techniques such as specific microscopes to study molecular machines involved in transcription.
- Artificial intelligence, specifically machine learning, is revolutionizing structural biology by predicting protein interactions, accelerating research, and enabling nonbiased testing of protein interactions.



[nference Establishes AI Initiative with Takeda to Advance Precision Medicine in Inflammatory Bowel Disease - Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)

Topics: Deals, Health, Fitness, Healthcare, Machine Learning, Science 

- nference and Takeda have established a research alliance to leverage AI for optimizing patient care in inflammatory bowel disease (IBD).
- The collaboration, named Project SUCCINCT, will use nference's AI platform to analyze health records and personalize treatment options for IBD patients.
- Takeda and nference aim to advance precision medicine by utilizing AI to improve diagnostic and treatment journeys for patients with IBD.



[56-gram drone gets ant-inspired AI eyes to navigate autonomously - Interesting Engineering](https://interestingengineering.com/innovation/insect-inspired-navigation-robot)

Topics: Autonomous Vehicles, Baidu, Machine Learning, Meta, Navigation, Products, Robots, Science, Testing, Transportation 

- Researchers have developed an ant-inspired navigation system for a 56-gram drone that combines visual homing with odometry for autonomous navigation, aiming at applications like warehouse stock monitoring and gas leak detection.
- Tiny drones weighing from ten to a few hundred grams have potential for real-world applications due to their lightweight design for safe navigation, especially in narrow areas, and their potential for use in scenarios like pest detection in greenhouses.
- The bio-inspired navigation approach for small drones involves capturing snapshots of surroundings, comparing visuals, and correcting odometry drift, making it memory-efficient and suitable for tasks like returning to a starting point in various indoor conditions.



...

[Mastering Photoshops Generative AI for Perfect Edits - Fstoppers](https://fstoppers.com/photoshop/mastering-photoshops-generative-ai-perfect-edits-673085)

Topics: Art, Design, Gen AI, Generative AI, Machine Learning, Products 

- Photoshop's generative AI can save time but may yield unexpected results; knowing techniques is crucial.
- Anthony Morganti's video demonstrates using Photoshop's generative AI to remove objects from photos.
- Tips include expanding selections, avoiding detailed selections that confuse AI, using the Remove tool for simple backgrounds, and using the Quick Mask tool for precise selections.



[Research Confirms Google AIO Keyword Trends - Search Engine Journal](https://www.searchenginejournal.com/research-confirms-google-aio-keyword-trends/522365/)

Topics: Big Tech, Data, Google, Machine Learning, Research 

- Rapid and dramatic changes in AIO triggers, with shifts observed in keywords triggering AI Overviews as per BrightEdge's research findings
- BrightEdge's assessment of Google search and AI trends, emphasizing Google's dominant position while new players are emerging in an AI-driven search landscape
- Significant decrease in Reddit & Quora referrals from AI Overviews in June 2024, indicating Google's preference for expert content over user-generated discussions



[I made a personalized workout plan with Claude AI  heres how to do it - Tom's Guide](https://www.tomsguide.com/claude-ai-can-improve-your-fitness-heres-how)

Topics: Claude, Gen AI, Health, Fitness, Machine Learning, Products, Recommendation Systems, Review, Robots 

- Claude AI revolutionizes personal fitness by creating tailored exercise plans based on individual needs, goals, and lifestyle, offering a personalized fitness experience accessible to anyone.
- Users can log into Claude, state fitness goals, input personal details, specify equipment and time schedule, generate initial plans, refine plans, get exercise descriptions, create progressive plans, incorporate rest and recovery, adapt as needed, and combine with nutrition advice for a comprehensive fitness plan.
- Claude's personalized approach provides detailed exercise descriptions, progression strategies, and nutrition advice, resembling a knowledgeable personal trainer experience at an affordable cost, while other interactive AI fitness solutions like the workout app and CAROL Bike offer alternative approaches to personalized fitness.



[Maths test stumps AI models: which number is bigger, 9.90 or 9.11? - South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3270851/maths-test-stumps-ai-models-which-number-bigger-990-or-911)

Topics: AI Models, Cognitive Science, Education, Gen AI, Machine Learning, OpenAI, Perplexity, Robots, Testing 

- Large language models struggle with basic maths knowledge.
- AI models can only predict answers based on training data.
- AI technology's evolution is highlighted by the math issue faced by models.



[The Chosen One: Consistent Characters in Text-to-Image Diffusion Models: References - Hacker Noon](https://hackernoon.com/the-chosen-one-consistent-characters-in-text-to-image-diffusion-models-references)

Topics: Text-To-Image Diffusion Models 

- Study introduces an iterative procedure to identify images sharing similar identities
- Authors include researchers from Google, Hebrew University of Jerusalem, Tel Aviv University, and Reichman University
- Several references to related works in text-to-image diffusion models and personalization techniques



In [121]:
# clean up the list of topics in human in the loop workflow
# loop though each topic and summarize, then 
# then combine the summaries for a final prompt

In [122]:
TESTPROMPT = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input. You are able to categorize information, 
and identify trends from large volumes of news.

Objective: 
I will provide the text of today's news articles about AI and summary bullet points in markdown format.
Bullet points will contain a title and URL, a list of topics discussed, and a bullet-point summary of
the article. You are tasked with identifying and summarizing the most important news, recurring themes,
common facts and items. Your job is to create a concise summary of today's topics and developments.
You will write an engaging summary of today's news encompassing the most important and frequently 
mentioned topics and themes.
You will write in the professional but engaging, narrative style of a tech reporter for a national publication.
You will be balanced, professional, informative, providing accurate, clear, concise summaries in a neutral tone.
You will group stories into related topics

Input Format Template:

[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging-topic-title-1

- bullet-point-1a - [site-name-1a](site-url-1a)
- bullet-point-1b - [site-name-1b](site-url-1b)

# Engaging-topic-title-2

- bullet-point-2a - [site-name-2a](site-url-2a)
- bullet-point-2b - [site-name-2b](site-url-2b)

Instructions:

Read the input closely.
Very important: USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.

Please check carefully that you only use information provided in the following input, that you include
all links in the input, and that any bullet point does not repeat information or links prevously provided.

Input:

"""



In [123]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TESTPROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)

response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


2024-07-18 11:18:43,994 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Innovations in Biology and Healthcare

- Ex-Meta scientists developed the ESM3 model, an AI that can create proteins not found in nature, opening new possibilities for synthetic biology - [Live Science](https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature)
- AI is revolutionizing drug development, particularly in early-stage drug discovery, with significant contributions to oncology drugs - [Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)
- Structural biology is being transformed by AI, predicting protein interactions and accelerating research - [Harvard Medical School](https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology)
- nference and Takeda have established an AI initiative to advance precision medicine in inflammatory bowel disease - [Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)

# AI and Autonomous Navigation

- Researchers developed an ant-inspired navigation system for a 56-gram drone, enabling autonomous navigation for applications like warehouse monitoring - [Interesting Engineering](https://interestingengineering.com/innovation/insect-inspired-navigation-robot)
- A tiny drone named "CrazyFlie" developed by Delft University of Technology can navigate back to its starting location using an insect-inspired navigation strategy - [EurekAlert](https://www.eurekalert.org/multimedia/1034513)
- Bug-inspired drones navigate autonomously in remote areas, offering applications in agriculture, stock tracking, and search and rescue - [Popular Science](https://www.popsci.com/technology/drones-ants-memory/)

# AI Startups and Investments

- Fei-Fei Li's startup, World Labs, reached a \$1 billion valuation within four months, focusing on developing AI with spatial intelligence - [Benzinga](https://www.benzinga.com/news/24/07/39833376/andreessen-horowitz-backed-godmother-of-ai-fei-fei-li-quietly-builds-1b-startup-in-4-months)
- Artificial Agency raised \$16 million for its AI-powered behavior engine to enhance gaming experiences - [VentureBeat](https://venturebeat.com/ai/artificial-agency-raises-16m-to-bring-ai-powered-behavior-to-games/)
- CTERA secured \$80 million in funding to expand global presence and enhance AI capabilities - [CRN](https://www.crn.com/news/storage/2024/ctera-ceo-on-new-80m-growth-funding-plans-for-ai-intelligence-in-file-services)

# AI in Consumer Electronics

- Samsung's AI technology can transform sketches into hyper-realistic images, integrated into the new Z Fold 6 - [Analytics Insight](https://www.analyticsinsight.net/ampstories/artificial-intelligence/samsungs-ai-makes-images-that-are-uncanny-real)
- Samsung acquired UK startup Oxford Semantic Technologies to enhance AI capabilities and offer personalized experiences on its devices - [The Register](https://www.theregister.com/2024/07/18/samsung_acquires_oxford_semantic/)
- Samsung partners with ByteDance to enhance AI capabilities of Galaxy Z Fold6 and Z Flip6 models in China - [South China Morning Post](https://www.scmp.com/tech/big-tech/article/3270977/chinas-foldable-smartphone-market-heats-samsung-bytedance-ai-pact-xiaomi-launch)

# AI and Regulatory Challenges

- EU antitrust regulators are investigating if Google and Samsung's AI deal hinders rival chatbots on Samsung smartphones - [Reuters](https://www.reuters.com/technology/artificial-intelligence/eu-antitrust-regulators-want-know-if-google-samsungs-chatbot-deal-hinders-rivals-2024-07-17/)
- The European Union is scrutinizing AI deals involving tech giants like Microsoft-OpenAI and Google-Samsung for potential anti-competitive practices - [Artificial Intelligence News](https://www.artificialintelligence-news.com/news/eu-probes-microsoft-openai-and-google-samsung-ai-deals/)
- Italy's antitrust agency launched a probe into Google and Alphabet over alleged unfair commercial practices related to user consent requests - [Reuters](https://www.reuters.com/technology/italys-antitrust-launches-probe-into-google-over-unfair-commercial-practices-2024-07-18/)

# AI in Warfare

- Ukrainian startups are developing AI-enabled drones for warfare, focusing on visual systems, terrain mapping, and interconnected drone swarms - [Newsmax](https://www.newsmax.com/world/globaltalk/ukraine-war-drones/2024/07/18/id/1173032/)
- AI-powered drones in Ukraine are being developed to combat signal jamming and increase hit rates, with cost-effective targeting systems using tools like Raspberry Pi - [Reuters](https://www.reuters.com/technology/artificial-intelligence/ukraine-rushes-create-ai-enabled-war-drones-2024-07-18/)
- Ukrainian tech startups are creating AI-powered drones for strikes and surveillance missions, resistant to Russian signal jamming - [PCMag](https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces)

# AI in Financial and Business Applications

- Anthropic and Menlo Ventures partnered on a \$100 million AI fund to support startups using Anthropic's AI models - [Bloomberg](https://www.bloomberg.com/news/articles/2024-07-17/anthropic-and-menlo-ventures-partner-on-100-million-ai-fund)
- Anthropic's Claude 3.5 Sonnet is ranked number 1 for business and finance in S&P AI Benchmarks by Kensho - [Amazon Web Services](https://aws.amazon.com/blogs/machine-learning/anthropic-claude-3-5-sonnet-ranks-number-1-for-business-and-finance-in-sp-ai-benchmarks-by-kensho/)
- TSMC now generates over half of its revenue from high-performance computing, driven by strong AI demand - [Bloomberg](https://www.bloomberg.com/news/articles/2024-07-18/more-than-half-of-tsmc-s-sales-are-now-high-end-chips-like-ai)

# AI in Creative and Design Tools

- Microsoft released its AI-powered Designer platform for AI image generation, available on web, mobile, and Windows apps - [Engadget](https://www.engadget.com/microsoft-releases-ios-and-android-apps-for-designer-its-ai-powered-canva-competitor-203028855.html)
- Photoshop's generative AI can save time on edits, with techniques demonstrated for removing objects from photos - [Fstoppers](https://fstoppers.com/photoshop/mastering-photoshops-generative-ai-perfect-edits-673085)

# AI and Cybersecurity

- Vulnerabilities in SAP AI Core could allow attackers to access customer data and contaminate internal artifacts, leading to supply chain attacks - [The Hacker News](https://thehackernews.com/2024/07/sap-ai-core-vulnerabilities-expose.html)
- Enterprise security teams are implementing controls to protect sensitive data from accidental exposure due to AI-enabled applications - [Dark Reading](https://www.darkreading.com/threat-intelligence/orgs-are-finally-making-moves-to-mitigate-genai-risks)

# AI in Media and Entertainment

- Google signed a sponsorship deal with NBC and Team USA to integrate AI products into the coverage of the Paris Olympics - [Hollywood Reporter](https://www.hollywoodreporter.com/business/digital/google-ai-features-nbc-2024-paris-olympics-1235951779/)
- Live2Diff AI system can stylize live video streams in near real-time, offering applications in entertainment and augmented reality - [VentureBeat](https://venturebeat.com/ai/from-reality-to-fantasy-live2diff-ai-brings-instant-video-stylization-to-life/)

In [124]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


# AI Innovations in Biology and Healthcare

- Ex-Meta scientists developed the ESM3 model, an AI that can create proteins not found in nature, opening new possibilities for synthetic biology - [Live Science](https://www.livescience.com/technology/artificial-intelligence/chatgpt-moment-for-biology-ex-meta-scientists-develop-ai-that-creates-proteins-not-found-in-nature)
- AI is revolutionizing drug development, particularly in early-stage drug discovery, with significant contributions to oncology drugs - [Bloomberg](https://www.bloomberg.com/news/audio/2024-07-18/ai-drug-discovery-with-harren-jhoti-vanguards-of-health-care)
- Structural biology is being transformed by AI, predicting protein interactions and accelerating research - [Harvard Medical School](https://hms.harvard.edu/news/how-machine-learning-propelling-structural-biology)
- nference and Takeda have established an AI initiative to advance precision medicine in inflammatory bowel disease - [Business Wire](https://www.businesswire.com/news/home/20240717546296/en/nference-Establishes-AI-Initiative-with-Takeda-to-Advance-Precision-Medicine-in-Inflammatory-Bowel-Disease)

# AI and Autonomous Navigation

- Researchers developed an ant-inspired navigation system for a 56-gram drone, enabling autonomous navigation for applications like warehouse monitoring - [Interesting Engineering](https://interestingengineering.com/innovation/insect-inspired-navigation-robot)
- A tiny drone named "CrazyFlie" developed by Delft University of Technology can navigate back to its starting location using an insect-inspired navigation strategy - [EurekAlert](https://www.eurekalert.org/multimedia/1034513)
- Bug-inspired drones navigate autonomously in remote areas, offering applications in agriculture, stock tracking, and search and rescue - [Popular Science](https://www.popsci.com/technology/drones-ants-memory/)

# AI Startups and Investments

- Fei-Fei Li's startup, World Labs, reached a \$1 billion valuation within four months, focusing on developing AI with spatial intelligence - [Benzinga](https://www.benzinga.com/news/24/07/39833376/andreessen-horowitz-backed-godmother-of-ai-fei-fei-li-quietly-builds-1b-startup-in-4-months)
- Artificial Agency raised \$16 million for its AI-powered behavior engine to enhance gaming experiences - [VentureBeat](https://venturebeat.com/ai/artificial-agency-raises-16m-to-bring-ai-powered-behavior-to-games/)
- CTERA secured \$80 million in funding to expand global presence and enhance AI capabilities - [CRN](https://www.crn.com/news/storage/2024/ctera-ceo-on-new-80m-growth-funding-plans-for-ai-intelligence-in-file-services)

# AI in Consumer Electronics

- Samsung's AI technology can transform sketches into hyper-realistic images, integrated into the new Z Fold 6 - [Analytics Insight](https://www.analyticsinsight.net/ampstories/artificial-intelligence/samsungs-ai-makes-images-that-are-uncanny-real)
- Samsung acquired UK startup Oxford Semantic Technologies to enhance AI capabilities and offer personalized experiences on its devices - [The Register](https://www.theregister.com/2024/07/18/samsung_acquires_oxford_semantic/)
- Samsung partners with ByteDance to enhance AI capabilities of Galaxy Z Fold6 and Z Flip6 models in China - [South China Morning Post](https://www.scmp.com/tech/big-tech/article/3270977/chinas-foldable-smartphone-market-heats-samsung-bytedance-ai-pact-xiaomi-launch)

# AI and Regulatory Challenges

- EU antitrust regulators are investigating if Google and Samsung's AI deal hinders rival chatbots on Samsung smartphones - [Reuters](https://www.reuters.com/technology/artificial-intelligence/eu-antitrust-regulators-want-know-if-google-samsungs-chatbot-deal-hinders-rivals-2024-07-17/)
- The European Union is scrutinizing AI deals involving tech giants like Microsoft-OpenAI and Google-Samsung for potential anti-competitive practices - [Artificial Intelligence News](https://www.artificialintelligence-news.com/news/eu-probes-microsoft-openai-and-google-samsung-ai-deals/)
- Italy's antitrust agency launched a probe into Google and Alphabet over alleged unfair commercial practices related to user consent requests - [Reuters](https://www.reuters.com/technology/italys-antitrust-launches-probe-into-google-over-unfair-commercial-practices-2024-07-18/)

# AI in Warfare

- Ukrainian startups are developing AI-enabled drones for warfare, focusing on visual systems, terrain mapping, and interconnected drone swarms - [Newsmax](https://www.newsmax.com/world/globaltalk/ukraine-war-drones/2024/07/18/id/1173032/)
- AI-powered drones in Ukraine are being developed to combat signal jamming and increase hit rates, with cost-effective targeting systems using tools like Raspberry Pi - [Reuters](https://www.reuters.com/technology/artificial-intelligence/ukraine-rushes-create-ai-enabled-war-drones-2024-07-18/)
- Ukrainian tech startups are creating AI-powered drones for strikes and surveillance missions, resistant to Russian signal jamming - [PCMag](https://www.pcmag.com/news/ukraine-building-ai-war-drone-fleet-to-combat-russian-forces)

# AI in Financial and Business Applications

- Anthropic and Menlo Ventures partnered on a \$100 million AI fund to support startups using Anthropic's AI models - [Bloomberg](https://www.bloomberg.com/news/articles/2024-07-17/anthropic-and-menlo-ventures-partner-on-100-million-ai-fund)
- Anthropic's Claude 3.5 Sonnet is ranked number 1 for business and finance in S&P AI Benchmarks by Kensho - [Amazon Web Services](https://aws.amazon.com/blogs/machine-learning/anthropic-claude-3-5-sonnet-ranks-number-1-for-business-and-finance-in-sp-ai-benchmarks-by-kensho/)
- TSMC now generates over half of its revenue from high-performance computing, driven by strong AI demand - [Bloomberg](https://www.bloomberg.com/news/articles/2024-07-18/more-than-half-of-tsmc-s-sales-are-now-high-end-chips-like-ai)

# AI in Creative and Design Tools

- Microsoft released its AI-powered Designer platform for AI image generation, available on web, mobile, and Windows apps - [Engadget](https://www.engadget.com/microsoft-releases-ios-and-android-apps-for-designer-its-ai-powered-canva-competitor-203028855.html)
- Photoshop's generative AI can save time on edits, with techniques demonstrated for removing objects from photos - [Fstoppers](https://fstoppers.com/photoshop/mastering-photoshops-generative-ai-perfect-edits-673085)

# AI and Cybersecurity

- Vulnerabilities in SAP AI Core could allow attackers to access customer data and contaminate internal artifacts, leading to supply chain attacks - [The Hacker News](https://thehackernews.com/2024/07/sap-ai-core-vulnerabilities-expose.html)
- Enterprise security teams are implementing controls to protect sensitive data from accidental exposure due to AI-enabled applications - [Dark Reading](https://www.darkreading.com/threat-intelligence/orgs-are-finally-making-moves-to-mitigate-genai-risks)

# AI in Media and Entertainment

- Google signed a sponsorship deal with NBC and Team USA to integrate AI products into the coverage of the Paris Olympics - [Hollywood Reporter](https://www.hollywoodreporter.com/business/digital/google-ai-features-nbc-2024-paris-olympics-1235951779/)
- Live2Diff AI system can stylize live video streams in near real-time, offering applications in entertainment and augmented reality - [VentureBeat](https://venturebeat.com/ai/from-reality-to-fantasy-live2diff-ai-brings-instant-video-stylization-to-life/)

In [125]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-18 11:18:44,160 - AInewsbot - INFO - Sending full summary email 


In [ ]:
log("Finished")


In [ ]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at refining and enhancing written materials, specializing in
ensuring clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Edit the markdown newsletter provided below by removing any redundant
sentences or bullet points that restate previous points and contain the same link.
Leave intact bullet points that are unique and provide distinct information.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information or provide identical links.
Remove all redundant sentences and bullet points that do not contribute new information or unique links.
Ensure that the remaining content flows logically and maintains the intended message and tone of the newsletter.
Double-check the final edited version for any inconsistencies or errors introduced during the editing process.
Take a deep breath and work on this problem step-by-step.
"""

In [ ]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below and advise on ways to improve it.
Note any links which are repeated, any sections which are similar and could be combined,
and any copy edits. You will only provide suggestions, and not rewrite the copy.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information and provide identical links and should be removed.
Identify any sections which could be combined because they contain similar but not identical content.
Suggest improvements to any sections which are not clear, concise, and coherent.
Take a deep breath and work on this problem step-by-step.
"""

In [15]:
display(Markdown(mail_md_str.replace("$", "\\$")))


NameError: name 'Markdown' is not defined

In [ ]:
edit_prompt1 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below.
It consists of a series of sections, each of which contains several bullet points.
For each section, review each bullet point and advise if it should be moved to a different section.
You will only provide suggestions, and not rewrite the newsletter or provide other comments except
instructions regarding moving bullet points between sections.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections.
Identify sentences and bullet points that should be moved to a different section. Write the
bullet point and the section in should be moved to.
If no bullet points should be moved for a given section, state that no action is required for that section.

Check carefully to make sure all similar bullet points end up grouped together in the same section.

Take a deep breath and work on this problem step-by-step.

Newsletter to edit: 
{mail_md_str}
"""
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt1
              }],
    n=1,   
    temperature=0.2
)

response_str1 = response.choices[0].message.content
display(Markdown(response_str1.replace("$", "\\$")))


In [ ]:
edit_prompt2 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Below are editing instructions followed by a markdown newsletter.
Carefully review the editing instructions and the markdown newsletter provided below.
The newsletter consists of a series of sections, each of which contains several bullet points.
Move bullet points according to the editing instructions below from one section to another 
If there is no change to a specific section, include it unchanged in the response as it appears in the input.
Respond with the updated newsletter in markdown format.

Editing instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections. Then make only the following changes:
{response_str1}

Newsletter to edit: 
{mail_md_str}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt2
              }],
    n=1,   
    temperature=0.2
)
response_str2 = response.choices[0].message.content
display(Markdown(response_str2.replace("$", "\\$")))


In [16]:
edit_prompt3 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Carefully review each section of the markdown newsletter provided below. 
Each section consists of several bullet points. 

For each section, identify and combine redundant bullet points:

Instructions: 
For each section, identify bullet points containing identical URLs to other bullet points in the same section 
Rewrite the section, combining these similar bullet points to eliminate duplication.
Do not duplicate any URLs within a section.
Check the response carefully and ensure that no links are duplicated within a section.

Newsletter to edit: 
{response_str2}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt3
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


NameError: name 'response_str2' is not defined

In [ ]:
display(Markdown(response_str3[11:].replace("$", "\\$")))


In [ ]:
response_str3.replace("$", "\\$")